In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

from sklearn.model_selection import train_test_split

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, 'C:/Users/Administrator/Desktop/mine/1_参考/Class_Model')
# module_dir = os.path.join(current_dir, 'Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean, Risk_Data
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()
risk = Risk_Data()

In [2]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [3]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(48050, 63)

## 1_12. 剔除商家数据

In [4]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(45200, 63)

## 1_13.历史订单渠道名称补充

In [5]:
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('E:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

0

## 1_14.渠道归属

In [6]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物" 
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "八派信息" in a:
        return "八派信息"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "社群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a

In [7]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"]),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [8]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [9]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    44000
二手     1200
Name: count, dtype: int64

In [10]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2023&2024年台账20250117.xlsx"
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/0_参数表/'
dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 

# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(27824, 2)

是否带锁
未带锁    25728
带锁      2096
Name: count, dtype: int64

(45200, 70)

是否带锁,带锁,未带锁
是否二手,,
二手,174,1026
全新,1603,42397


## 1_17.日期处理

In [11]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [12]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
2季度    14838
4季度    11205
3季度     9679
1季度     9478
Name: count, dtype: int64

## 1_21.获取_买断数据

In [13]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(5050, 7)

## 1_22.租后数据 关联 买断数据

In [14]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(45200, 78)

(45200, 74)

In [15]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

6284

In [16]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      40360
已买断      4738
部分买断      102
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [17]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(2707, 7)

## 1_32.租后数据关联分期取消数据

In [18]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(45200, 82)

(45200, 79)

## 1_41.按风控策略调整时间节点划分时间段

In [19]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [20]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(595771, 23)

In [21]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4872, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(595771, 26)

(595771, 23)

In [22]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [23]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp

## 1_52.重命名

In [24]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [25]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [26]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

595920

(595920, 104)

(595771, 23)

(45200, 82)

## 1_62.去重

In [27]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
146271    869632   1.0
146283    869632   1.0
434571    869632   1.0
434583    869632   1.0
146272    869632   2.0
...          ...   ...
484139   2040621  23.0
431363   2040621  24.0
431387   2040621  24.0
484116   2040621  24.0
484140   2040621  24.0

[672 rows x 2 columns]


In [28]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(595416, 104)

In [29]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 2.计算逾期天数

## 2_11.异常查询

In [30]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 2_12.实还日期修正

In [31]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [32]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
346385,A2024110708472957,1692436,457.98,457.98,2025-04-10,2025-04-06,3.0,6.0,还款中,NaT,4,NaT,NaN,NaT
346386,A2024110708472957,1692436,457.98,457.98,2025-05-10,2025-05-10,3.0,7.0,还款中,NaT,4,NaT,NaN,NaT
477793,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
477794,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
477795,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT


In [33]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
383465,A20241210113506313,1839860,695.21,665.65,2025-05-13,NaT,1.0,6.0,还款中,NaT,4,NaT,NaN,NaT
451008,A20250223144805320,2133904,974.62,300.00,2025-05-26,NaT,1.0,4.0,已逾期,NaT,4,NaT,8.0,NaT
477793,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
477794,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
477795,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT


In [34]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60964,2024-07-22


In [35]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [36]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
222771,A2024060716074819,1111255,1085.68,0.0,2025-05-10,NaT,1.0,12.0,已买断,2024-12-14 14:59:21,8,2024-12-14,7.0,2024-12-14
230911,A20240620152351112,1150772,770.50,0.0,2026-02-24,NaT,1.0,21.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01
230912,A20240620152351112,1150772,514.32,0.0,2026-03-24,NaT,1.0,22.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01
446491,A2025021919082939,2114551,974.62,0.0,2025-12-22,NaT,1.0,11.0,已买断,2025-05-22 10:15:51,8,2025-05-22,9.0,2025-05-22
504912,A20250411105344370,2386604,1082.94,0.0,2026-02-14,NaT,1.0,11.0,已买断,2025-04-19 12:35:48,8,2025-04-19,10.0,2025-04-19


In [37]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [38]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [39]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [40]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [41]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [42]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [43]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [44]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 2_13.获取逾期截止时间

In [45]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-06-02
1   2025-06-02
2   2025-06-02
3   2025-06-02
4   2025-06-02
Name: now_date, dtype: datetime64[s]

## 2_14.重新定义状态

In [46]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    102722
已逾期                     59674
未到首期还款日                 42459
续租中                     14316
检测中                        24
Name: count, dtype: int64

订单状态 
租赁中      210714
已退款        6672
待归还        6104
待收货        2092
已完成         217
待发货         145
订单取消         84
检测中          21
待商家收货         5
Name: count, dtype: int64

In [47]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     288083
未到还款日    217566
已逾期支付     41682
已逾期       41196
已取消        6889
Name: count, dtype: int64

In [48]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 2_15.重新定义逾期天数

In [49]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

226084

Series([], Name: order_number, dtype: object)

In [50]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(595416, 108)

(595416, 108)

In [51]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        59950
还款中         8417
已买断         7509
续租中         5709
已完成         1097
检测中          135
未到首期还款日       48
Name: count, dtype: int64

In [52]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 2_16.逾期天数处理

In [53]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 2_17.稽查

In [54]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 2_18.确认是否到表现期

In [55]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [56]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [57]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [58]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [59]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-06-02,1
1,2022-09-04,2022-08-05,2025-06-02,1
2,2022-10-05,2022-09-05,2025-06-02,1
3,2022-11-04,2022-10-05,2025-06-02,1
4,2022-12-05,2022-11-05,2025-06-02,1


In [60]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 2_19.观察日定义 每月月底时间

In [61]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)


# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

In [62]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

## 2_20.更新逾期天数

In [63]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(1076, 2)

label
0.0    582588
1.0     12828
Name: count, dtype: int64

(1055, 160)

# 3.逾期加工

In [64]:
# （1）历史最大逾期--续租的含续租，但未续租的（即归还逾期的）没有统计在内
df_maxoverdue = df_xz_concat.groupby(['order_number','order_id','true_name','id_card_num','mobile','下单日期','search_time','归属渠道','是否二手','是否带锁']).agg({'逾期天数':'max'}).reset_index()
df_maxoverdue.shape

df_tmp = df_xz_concat[df_xz_concat.逾期天数 > 0].groupby(['order_number']).agg({'当前期数':'min'}).reset_index()
df_tmp = df_tmp.rename(columns = {'当前期数': '首次逾期期次'})

df_maxoverdue = df_maxoverdue.merge(df_tmp, on = 'order_number', how = 'left')
df_maxoverdue[:1]
del df_tmp

(45181, 11)

,order_number,order_id,true_name,id_card_num,mobile,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,首次逾期期次
0,A202207050851373,8924,胡水飘,362202199805282550,13027948681,2022-07-05,2022-07,未知渠道,二手,未带锁,4,4.0


In [65]:
# （2）FPD SPD TPD
df_xz_concat['当前期数'] = df_xz_concat['当前期数'].astype(int)

df_fstq = df_xz_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
df_fstq = df_fstq.reset_index()
df_fstq.shape

# df_fstq = df_fstq.rename(columns = {2: 'FPD'})
df_fstq['FPD'] = df_fstq[2]
df_fstq.loc[:,"SPD"] = np.where( df_fstq['FPD'] == 0, df_fstq[3], 0)
df_fstq.loc[:,"TPD"] = np.where((df_fstq['FPD'] == 0) & (df_fstq[3] == 0), df_fstq[4], 0)

# df_fstq = df_fstq[['order_id','FPD','SPD','TPD']]
df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])][['order_id','FPD','SPD','TPD']]
# df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# A2023073114301076 508404
# df_fstq.FPD.max()

(45182, 37)

当前期数,order_id,FPD,SPD,TPD
28,12449,0.0,11.0,0.0
3719,508404,0.0,0.0,0.0
24300,1678492,178.0,0.0,0.0
24564,1692916,8.0,0.0,0.0


In [66]:
# # （3）第2-4期逾期天数
# df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数'})
# df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数']]
# tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
# df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
# df_fstq = df_fstq.fillna(0)
# df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])]
# # df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# # A2023073114301076 508404
# # df_fstq.FPD.max()


# # df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数', 5:'第五期逾期天数',  6:'第六期逾期天数', 7:'第七期逾期天数'})
# # df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数','第五期逾期天数','第六期逾期天数','第七期逾期天数']]
# # tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
# # df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
# # df_fstq = df_fstq.fillna(0)

# （3）
# （3-1）第2-12期逾期天数
df_fstq = df_fstq.fillna(0)
for var in range(2,13):
    if var == 2:
        df_fstq['mob12dpd_max'] = df_fstq[var]
    else:
        # print (var)
        df_fstq['mob12dpd_max'] = np.where(df_fstq.mob12dpd_max >= df_fstq[var], df_fstq.mob12dpd_max, df_fstq[var])

for var in range(2,8):
    if var == 2:
        df_fstq['mob6dpd_max'] = df_fstq[var]
    else:
        df_fstq['mob6dpd_max'] = np.where(df_fstq.mob6dpd_max >= df_fstq[var], df_fstq.mob6dpd_max, df_fstq[var])

for var in range(2,5):
    if var == 2:
        df_fstq['mob3dpd_max'] = df_fstq[var]
    else:
        df_fstq['mob3dpd_max'] = np.where(df_fstq.mob3dpd_max >= df_fstq[var], df_fstq.mob3dpd_max, df_fstq[var])

# （3-2）第2-4期逾期天数
df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数'})
# df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数']]
df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数','mob3dpd_max','mob6dpd_max','mob12dpd_max']]
tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])]
# df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# A2023073114301076 508404
# df_fstq.FPD.max()

,order_id,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,mob3dpd_max,mob6dpd_max,mob12dpd_max,应付日期
28,12449,0.0,11.0,0.0,0.0,11.0,9.0,11.0,19.0,19.0,2022-08-05
3719,508404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2023-08-03
24300,1678492,178.0,0.0,0.0,178.0,147.0,116.0,178.0,178.0,178.0,2024-11-06
24564,1692916,8.0,0.0,0.0,8.0,0.0,40.0,40.0,48.0,48.0,2024-11-10


In [67]:
# # （4）表关联 
# df_analy_data = df_maxoverdue.merge(df_fstq, on = 'order_id', how = 'left')
# df_analy_data = df_analy_data.rename(columns = {'逾期天数':'历史最大逾期天数','应付日期': '首期应还日期'})
# df_analy_data = df_analy_data.merge(df_zhys[['order_number','overdue_type']], on = 'order_number', how = 'left')

# df_analy_data.shape
# df_analy_data = df_analy_data.sort_values(by = ['order_number','overdue_type'])
# df_analy_data = df_analy_data.drop_duplicates(subset = 'order_number')
# # 以下6个订单号重复,overdue_type 不一样，保留未逾期的，因为 历史最大逾期天数 为0，P站显示为0
# # A2024020821520518    2
# # A2024010116302721    2
# # A2024012715442754    2
# # A202401030606401     2
# # A2024010216293470    2
# # A202402110902322     2
# df_analy_data.shape

# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

# （4）表关联 
df_analy_data = df_maxoverdue.merge(df_fstq, on = 'order_id', how = 'left')
df_analy_data = df_analy_data.rename(columns = {'逾期天数':'历史最大逾期天数','应付日期': '首期应还日期'})
df_analy_data = df_analy_data.merge(df_zhys[['order_number','overdue_type']], on = 'order_number', how = 'left')

df_analy_data.shape
df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

(45194, 23)

,order_number,order_id,true_name,id_card_num,mobile,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,首次逾期期次,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,mob3dpd_max,mob6dpd_max,mob12dpd_max,首期应还日期,overdue_type
28,A202208052103231,12449,赵旭,230184199803042615,13591343043,2022-08-05,2022-08,搜索渠道,全新,未带锁,19,3.0,0.0,11.0,0.0,0.0,11.0,9.0,11.0,19.0,19.0,2022-08-05,M6
3719,A2023073114301076,508404,邵光峰,371482199409260337,15136842042,2023-07-31,2023-07,芝麻租物,全新,未带锁,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2023-08-03,
24310,A20241103155713186,1678492,张驰,152201200201300017,16223593777,2024-11-03,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,178,2.0,178.0,0.0,0.0,178.0,147.0,116.0,178.0,178.0,178.0,2024-11-06,M3
24574,A20241107103123201,1692916,李星华,360427199610221519,15870818200,2024-11-07,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,48,2.0,8.0,0.0,0.0,8.0,0.0,40.0,40.0,48.0,48.0,2024-11-10,M0


In [68]:
# （5）打标签
# df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)

df_analy_data['mob2dpd'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['mob3dpd'] = np.where(df_analy_data.mob2dpd >= df_analy_data.第四期逾期天数, df_analy_data.mob2dpd, df_analy_data.第四期逾期天数)

df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
                                               else 0 if x.FPD >= 0 and x.FPD <= 7
                                               else 999, axis = 1)
df_analy_data['y_fraud_fpd15'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 16
                                               else 0 if x.FPD >= 0 and x.FPD <= 15
                                               else 999, axis = 1)
df_analy_data['y_fraud_fpd30'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 31 
                                               else 0 if x.FPD >= 0 and x.FPD <= 30
                                               else 999, axis = 1)

df_analy_data['y_fraud_mob2dpd7'] = df_analy_data.apply(lambda x: 1 if x.mob2dpd >= 8 
                                               else 0 if x.mob2dpd >= 0 and x.mob2dpd <= 7
                                               else 999, axis = 1)

df_analy_data['y_fraud_mob3dpd7'] = df_analy_data.apply(lambda x: 1 if x.mob3dpd_max >= 8 
                                               else 0 if x.mob3dpd_max >= 0 and x.mob3dpd_max <= 7
                                               else 999, axis = 1)
df_analy_data['y_credit_mob3dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob3dpd_max >= 31 
                                               else 0 if x.mob3dpd_max >= 0 and x.mob3dpd_max <= 30
                                               else 999, axis = 1)

df_analy_data['y_credit_mob6dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob6dpd_max >= 31 
                                               else 0 if x.mob6dpd_max >= 0 and x.mob6dpd_max <= 30
                                               else 999, axis = 1)

df_analy_data['y_credit_mob12dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob12dpd_max >= 31 
                                               else 0 if x.mob12dpd_max >= 0 and x.mob12dpd_max <= 30
                                               else 999, axis = 1)

df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
                                               else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
                                               else 999, axis = 1)
df_analy_data.shape

# # （5）打标签
# df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)
# df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f2pd7'] = df_analy_data.apply(lambda x: 1 if x.f2pd7 >= 8 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f3pd7'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 8 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_credit_mob3pd30'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 31 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 30
#                                                else 999, axis = 1)
# df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
#                                                else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
#                                                else 999, axis = 1)
# df_analy_data.shape
# var_1 = ['order_number','下单日期','历史最大逾期天数','FPD','SPD','TPD','第二期逾期天数','第三期逾期天数','第四期逾期天数','overdue_type','f2pd7','f3pd7','y_fraud_fpd7','y_fraud_f2pd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])][var_1]



# df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)
# # 新增
# df_analy_data['mob4dpd'] = np.where(df_analy_data.f3pd7 >= df_analy_data.第五期逾期天数, df_analy_data.f3pd7, df_analy_data.第五期逾期天数)
# df_analy_data['mob5dpd'] = np.where(df_analy_data.mob4dpd >= df_analy_data.第六期逾期天数, df_analy_data.mob4dpd, df_analy_data.第六期逾期天数)
# df_analy_data['mob6dpd'] = np.where(df_analy_data.mob5dpd >= df_analy_data.第七期逾期天数, df_analy_data.mob5dpd, df_analy_data.第七期逾期天数)

# df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f2pd7'] = df_analy_data.apply(lambda x: 1 if x.f2pd7 >= 8 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f3pd7'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 8 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_credit_mob3dpd30'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 31 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 30
#                                                else 999, axis = 1)
# df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
#                                                else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
#                                                else 999, axis = 1)
# # 新增
# df_analy_data['y_credit_mob3dpd15'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 15
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 14
#                                                else 999, axis = 1)
# df_analy_data['y_credit_mob6dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob6dpd >= 31
#                                                else 0 if x.mob6dpd >= 0 and x.mob6dpd <= 30
#                                                else 999, axis = 1)


# df_analy_data.shape

(45194, 33)

In [69]:
# （6）数据检查
df_analy_data[df_analy_data.历史最大逾期天数 >0].shape
df_analy_data[df_analy_data.历史最大逾期天数 >30].shape
df_analy_data[df_analy_data.历史最大逾期天数 >90].shape
df_analy_data[df_analy_data.历史最大逾期天数 >180].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 <= '2023-12-31')].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2024-01-01')&(df_analy_data.下单日期 <= '2024-12-31')].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2025-01-01')].shape

(17621, 33)

(8751, 33)

(6500, 33)

(4516, 33)

(2841, 33)

(1675, 33)

(0, 33)

# 4.规则命中数据整理

## 4_1.取规则命中数据

In [70]:
# （1）大表
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status,om.order_method,om.order_type
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  

#2025.3.10
# 判断拒量是机审拒绝还是被出库前强拒，无拒绝理由： qvt_risk = '1'为机审强拒，qvt_risk='0' 且qvt_result='1'的为出库前强拒

,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result 

# is_vip = '1' & status_result = '0' 为免人审的单

,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tprm.purchase_amount, tprm.all_rental, tprm.order_number order_number2
,om.buy_service_product,tso.status as service_status 
, tprm.contract_price, tor.update_time, tomt.reason
,tor.jd_user_level_risk  -- 京享租评分等级

from db_digua_business.t_order  om 
LEFT JOIN db_digua_business.t_postlease_receivables_monitoring tprm on om.id=tprm.order_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
;
'''
df = query(sql1)
df.shape

(3222753, 43)

In [71]:
# （2）机审
# db_credit.risk：机审的订单，机审命中的
sql2 = '''
SELECT 
id_card, mobile, time, trace_id
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_risk  
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.rejected') end,'"','') as rejected  

# 2025.3.18 识别联合运营订单 union_rent_tag = '1' 为联合运营的，前置拦截的表从 2025.3.25 开始也加上 union_rent_tag 的标签，存在"union_rent_tag"字段，且union_rent_tag==1，为联营订单
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected
FROM `db_credit`.`risk`
'''
df_risk = query(sql2)
df_risk.shape

(358167, 8)

In [72]:
# （3）出库前-风控流
# 出库前风控流强拒
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_re = query(sql3)
# df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
df_re.shape
df_re[:1]

(103898, 5)

,trace_id,id_card_re,time,status_re,time_re
0,53ab4a0a13e7f82b5fc594a852900ff7,43072619940323311X,2024-01-25 17:11:18,0,2024-01-25


In [73]:
# （4）出库前-数控
# 蚂蚁数控强拒
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)
df_ra.shape
df_ra[:1]

(104085, 4)

,order_id,time,status_ra,time_ra
104084,2796665,2025-06-02 20:56:05,0,2025-06-02


In [74]:
# （5）表合并
# df_risk 的订单 包含 df_re、df_ra 的，出现在 df_re、df_ra 的话，以 df_re、df_ra 的结果为准
# 前置拦截的不会出现在 df_risk
df = df.merge(df_risk[['trace_id', 'status_risk', 'union_rent_tag', 'union_rent_rejected']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], on='order_id', how='left')
df.shape
df[:2]

(3222753, 49)

,create_date,create_time,order_id,order_number,all_money,status,order_method,order_type,status2,channel_type,sku_attributes,product_name,new_actual_money,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,qvt_risk,qvt_result,is_vip,status_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,purchase_amount,all_rental,order_number2,buy_service_product,service_status,contract_price,update_time,reason,jd_user_level_risk,status_risk,union_rent_tag,union_rent_rejected,status_re,time_ra,status_ra
0,2022-05-31,2022-05-31 08:55:44,8742,A202205310855441,7796.4,13,0,ZFB_ORDER,订单取消,其他渠道,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",全新国行苹果iPhone13 全网通 双卡双待 租完即送,7798.0,租完即送,18301579950,陈园园,342923199007100712,None,None,过期系统自动取消,None,None,None,None,None,None,None,None,None,NaN,None,13,自营店铺,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN
1,2022-05-31,2022-05-31 08:56:19,8743,A202205310856192,7796.4,13,0,ZFB_ORDER,订单取消,其他渠道,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",全新国行苹果iPhone13 全网通 双卡双待 租完即送,7798.0,租完即送,18301579950,陈园园,342923199007100712,None,None,过期系统自动取消,None,None,None,None,None,None,None,None,None,NaN,None,13,自营店铺,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN


In [75]:
# （6）
# 前置拦截
# df_reject_before = df[((df['status2']=='订单取消')&(df['拒绝理由'].notna()))|((df['status2']=='订单取消')&(df['total_describes'].notna()))]
# df_reject_before['拒绝理由'] = np.where(df_reject_before['拒绝理由'].isna(),   
#                                         df_reject_before['total_describes'].str.split('拒绝理由：').str[-1].str.split('"').str[0],df_reject_before['拒绝理由'])
# df_reject_before = df_reject_before[['create_time', 'order_number', '拒绝理由']]

## 4_2.数据处理

In [76]:
# （1）信息处理
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'] = np.where(df['拒绝理由']=='', df['total_describes'].str.split('原因：').str[-1].str.split('"').str[0], df['rejected'])

In [77]:
# （2）判断进件
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")

In [78]:
# （3）日期处理
df['下单日期'] = df['create_time'].dt.date
df["下单日期"] = pd.to_datetime(df["下单日期"],errors="coerce")

In [79]:
# （4）订单预排序去重 剔除内部订单
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去除刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除订单状态空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
# note 20241209 删除此条件
# df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

df.shape

(2659637, 53)

In [80]:
# （5）剔除商家数据--只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)

df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="线下小店"].index,inplace=True)

df.drop(df[df['merchant_name'].str.contains(pat='探路者',regex=False)==True].index,inplace=True) 
df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)

df.shape

(2541410, 53)

In [81]:
# （6）定义审核状态
# 处理原因
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))|(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)


# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)

# 2025.3.10
df['是否机审强拒'] = np.where(df.status_risk=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)

# df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
# df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)

df.shape

(2541228, 60)

In [82]:
# （7）加工渠道、是否免审、是否拒量
# 2025.3.10
df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df['是否免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1 , 0)

# df['是否拒量'] = np.where(df.qvt_risk == '1', 1 , 0)
# df['是否拒量'] = np.where((df.qvt_risk == '0') & (df.qvt_result == '1'), 1 , df.是否拒量)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)

In [83]:
# （8）添加拒量被拒理由
# 判断拒量是机审拒绝还是被出库前强拒，无拒绝理由： qvt_risk = '1'为机审强拒，qvt_risk='0' 且qvt_result='1'的为出库前强拒
# 有些为真实拒绝原因
# 回捞命中订单恢复记录豁免的强拒项，2025.3.6 上线
df['拒绝理由'] = np.where((df.拒绝理由 == '[') & (df.qvt_risk == '1'), '命中机审强拒进回捞池', df.拒绝理由)
df['拒绝理由'] = np.where((df.拒绝理由 == '[') & (df.qvt_risk == '0') & (df.qvt_result == '1'), '命中出库前强拒进回捞池', df.拒绝理由)
df.shape
df[:1]

# 以下3个订单检查到是拒量回捞，但未识别出来
# A20241212093821140
# A20241214161430122
# A20241220184558214
# df[df.order_number == 'A20241220184558214'][['拒绝理由','qvt_risk','qvt_result']]

# df[(df.qvt_risk == '1')&(df.拒绝理由 != '命中机审强拒进回捞池')].下单日期.min()
# df[(df.qvt_risk == '0') & (df.qvt_result == '1')&(df.拒绝理由 != '命中出库前强拒进回捞池')].下单日期.min()
# df[(df.qvt_risk == '1')][['order_number','拒绝理由','status_risk']].拒绝理由.value_counts()
# df[(df.qvt_risk == '0') & (df.qvt_result == '1')][['order_number','拒绝理由','status_risk']].拒绝理由.value_counts()

(2541228, 66)

,create_date,create_time,order_id,order_number,all_money,status,order_method,order_type,status2,channel_type,sku_attributes,product_name,new_actual_money,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,qvt_risk,qvt_result,is_vip,status_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,purchase_amount,all_rental,order_number2,buy_service_product,service_status,contract_price,update_time,reason,jd_user_level_risk,status_risk,union_rent_tag,union_rent_rejected,status_re,time_ra,status_ra,拒绝理由,进件,下单日期,状态编码,取消原因,电审拒绝原因,无法联系原因,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,订单号,渠道名称,来源渠道,归属渠道,是否免人审,是否拒量
0,2022-05-31,2022-05-31 08:55:44,8742,A202205310855441,7796.4,13,0,ZFB_ORDER,订单取消,其他渠道,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",全新国行苹果iPhone13 全网通 双卡双待 租完即送,7798.0,租完即送,18301579950,陈园园,342923199007100712,None,None,过期系统自动取消,None,None,None,None,None,None,None,None,None,NaN,None,13,自营店铺,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN,None,未进件,2022-05-31,1.0,None,None,None,0,0,0,进件前取消,NaN,NaN,未知渠道,未知渠道,0,0


In [84]:
# （9）添加商家拒量被拒理由
# 添加小蚂蚁租机被拒理由
# 小蚂蚁租机 的用 df_risk 的 拒绝理由填充
df2 = df.copy()
# df2.drop(df2[df2['merchant_name']=="小蚂蚁租机"].index,inplace=True)
df2.drop(df2[df2['merchant_name'].isin(["小蚂蚁租机",'人人享租','崇胜数码','兴鑫兴通讯'])].index,inplace=True)

# 匹配小蚂蚁数据
# df_xmy = df[df['merchant_name'] == '小蚂蚁租机']
df_xmy = df[df['merchant_name'].isin(["小蚂蚁租机",'人人享租','崇胜数码','兴鑫兴通讯'])]

# 机审强拒数据重命名
# df_risk.rename(columns={'time': 'time_risk', 'status': 'status_risk', 'rejected': 'rejected_risk'}, inplace=True)
df_risk.rename(columns={'time': 'time_risk', 'rejected': 'rejected_risk'}, inplace=True)
df_risk['拒绝理由_risk'] = df_risk["rejected_risk"].str.replace("[","").str.replace("]","").str.replace('"','')
# 对小蚂蚁数据和机审强拒数据进行拼接
df_risk_all = pd.merge(df_xmy, df_risk, left_on='id_card_num', right_on='id_card', how='inner')
# 计算出库前强拒的订单数
df_risk_all2 = df_risk_all[(df_risk_all['time_risk'] < df_risk_all['update_time'])]
# 进行排序并取到最近的一个订单
df_risk_all2 = df_risk_all2.sort_values(['order_id', 'time_risk'], ascending=[True, False]).groupby('order_id').head(1)
# # df_risk_all2 = df_risk_all2[df_risk_all2['status_risk'] == '1']
df_risk_all2 = df_risk_all2[df_risk_all2['status_risk_y'] == '1']
df_risk_all2.shape
# 拼接小蚂蚁数据获取全新的拒绝理由
# status_risk 为空 但 拒绝理由不为空
df_risk_new = df.merge(df_risk_all2[['order_number', '拒绝理由_risk']], on='order_number', how='left')
df_risk_new['拒绝理由'] = np.where((df_risk_new['拒绝理由'].isna()) & df_risk_new['拒绝理由_risk'].notna(), df_risk_new['拒绝理由_risk'], df_risk_new['拒绝理由'])
df_risk_new['search_time'] = pd.to_datetime(df_risk_new["下单日期"]).dt.strftime('%Y-%m')
df_risk_new = df_risk_new.drop(columns = ['拒绝理由_risk'])
df_risk_new.shape
df_risk_new[:1]

# 860459
# df[df.拒绝理由.notnull()].shape
# 868083
# df_risk_new[df_risk_new.拒绝理由.notnull()].shape
# df_risk_new[df_risk_new.order_number == 'A2024110614100051'][['拒绝理由','status_risk']]

(27539, 75)

(2541228, 67)

,create_date,create_time,order_id,order_number,all_money,status,order_method,order_type,status2,channel_type,sku_attributes,product_name,new_actual_money,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,qvt_risk,qvt_result,is_vip,status_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,purchase_amount,all_rental,order_number2,buy_service_product,service_status,contract_price,update_time,reason,jd_user_level_risk,status_risk,union_rent_tag,union_rent_rejected,status_re,time_ra,status_ra,拒绝理由,进件,下单日期,状态编码,取消原因,电审拒绝原因,无法联系原因,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,订单号,渠道名称,来源渠道,归属渠道,是否免人审,是否拒量,search_time
0,2022-05-31,2022-05-31 08:55:44,8742,A202205310855441,7796.4,13,0,ZFB_ORDER,订单取消,其他渠道,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",全新国行苹果iPhone13 全网通 双卡双待 租完即送,7798.0,租完即送,18301579950,陈园园,342923199007100712,None,None,过期系统自动取消,None,None,None,None,None,None,None,None,None,NaN,None,13,自营店铺,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN,None,未进件,2022-05-31,1.0,None,None,None,0,0,0,进件前取消,NaN,NaN,未知渠道,未知渠道,0,0,2022-05


In [85]:
# （10）
#进件的数据
# 机审强拒&出库前强拒规则明细，1代表命中
df_jj = df_risk_new[df_risk_new['进件']=='进件']
# df_reject = df_jj[(df_jj['机审强拒']==1)|(df_jj['出库前风控强拒']==1)][['create_time', 'order_id', 'order_number', '拒绝理由', 'total_describes', 'result']]
df_reject = df_jj[['create_time', 'order_id', 'order_number', '拒绝理由', 'total_describes', 'result','trace_id','审核状态']]
df_reject_risk = risk.model_reject(df_reject, 'order_number')
# df_reject_risk_merge = df_reject[['order_number','order_id','trace_id','拒绝理由','审核状态']].merge(df_reject_risk, on='order_number', how='left')
# 2025.3.25
var_need = ['order_number','order_id','trace_id','下单日期','search_time','是否免人审','是否拒量','归属渠道','union_rent_tag','union_rent_rejected','拒绝理由','审核状态']
df_reject_risk_merge = df_risk_new[df_risk_new['进件']=='进件'][var_need].merge(df_reject_risk, on='order_number', how='left')
df_reject_risk_merge.shape
df_reject_risk_merge.tail(1)

# 拒绝理由 == '[' 的可能 可能是拒件 也可能是出库， 看 审核状态
# df_reject_risk_merge[df_reject_risk_merge.拒绝理由 == '['][:2]
# df_reject_risk_merge[df_reject_risk_merge.拒绝理由 == '['].tail(2)
# 295034

(373843, 38)

,order_number,order_id,trace_id,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected,拒绝理由,审核状态,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中融安分低于680,命中高风险名单,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒
373842,A20250602094830188,2790138,225fde8241f661cc51e9b49b001a1d0b,2025-06-02,2025-06,0,0,芝麻租物,1,[命中自有模型强拒],[,出库,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# （11）
# df_reject_before：前置命中的数据
# 前置拦截拒规则明细，1代表命中
df_reject_before = df[((df['status2']=='订单取消')&(df['拒绝理由'].notna()))|((df['status2']=='订单取消')&(df['total_describes'].notna()))]
df_reject_before['拒绝理由'] = np.where(df_reject_before['拒绝理由'].isna(), np.where(df_reject_before['total_describes'].str.contains('拒绝理由：'),df_reject_before['total_describes'].str.split('拒绝理由：').str[-1].str.split('"').str[0], df_reject_before['拒绝理由'])  
                                        ,df_reject_before['拒绝理由'])
# df_reject_before = df_reject_before[['create_time', 'order_id', 'order_number','trace_id', 'status2', '拒绝理由', 'total_describes', 'result','审核状态']]
# 2025.3.25
var_1 = ['create_time', 'order_id', 'order_number', 'status2', '拒绝理由', 'total_describes', 'result','trace_id','审核状态','下单日期','是否免人审','是否拒量','归属渠道','union_rent_tag','union_rent_rejected']
df_reject_before = df_reject_before[var_1]
df_reject_before_risk = risk.pre_reject(df_reject_before, 'order_number')

# 2025.3.25
# df_reject_before_risk_merge = df_reject_before[['order_number','order_id','trace_id','拒绝理由','审核状态']].merge(df_reject_before_risk, on='order_number', how='right')
var_need = ['order_number','order_id','trace_id','下单日期','search_time','是否免人审','是否拒量','归属渠道','union_rent_tag','union_rent_rejected','拒绝理由','审核状态']
df_reject_before['search_time'] = pd.to_datetime(df_reject_before["下单日期"]).dt.strftime('%Y-%m')
df_reject_before_risk_merge = df_reject_before[var_need].merge(df_reject_before_risk, on='order_number', how='right')

df_reject_before_risk_merge.shape
df_reject_before_risk_merge[:1]   
# 643314
# 699375

(772327, 24)

,order_number,order_id,trace_id,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected,拒绝理由,审核状态,年龄超过49岁或低于18岁,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中融安分低于680,命中高风险名单,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏
0,A2023010917234825,89368,,2023-01-09,2023-01,0,0,未知渠道,NaN,NaN,[年龄超过49岁或低于18岁],进件前取消,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# （12）进件和前置拦截-合并
# df_rule_data：前置被拒绝的和进件的样本数据
df_rule_data = df_reject_risk_merge.merge(df_reject_before_risk_merge, on = ['order_number','order_id','trace_id'], how = 'outer')
df_rule_data = df_rule_data.rename(columns = {'拒绝理由_x':'机审拒绝','拒绝理由_y':'前置拒绝'})

# df_rule_data[df_rule_data.审核状态_x == df_rule_data.审核状态_y].shape
# df_reject_before_risk_merge.审核状态.value_counts()
df_rule_data['审核状态'] = np.where(pd.isnull(df_rule_data.审核状态_x), df_rule_data.审核状态_y, df_rule_data.审核状态_x)
df_rule_data = df_rule_data.drop(columns = ['审核状态_x','审核状态_y'])

df_reject_risk_merge.shape
df_reject_before_risk_merge.shape
df_rule_data.shape
# df_rule_data[:1]

# df_rule_data[df_rule_data.机审拒绝.notnull()][:20]
# df_rule_data[df_rule_data.order_number == 'A2023010909584423']

df_rule_data['下单日期'] = np.where(pd.isnull(df_rule_data.下单日期_x), df_rule_data.下单日期_y, df_rule_data.下单日期_x)
df_rule_data['search_time'] = np.where(pd.isnull(df_rule_data.search_time_x), df_rule_data.search_time_y, df_rule_data.search_time_x)
df_rule_data['是否免人审'] = np.where(pd.isnull(df_rule_data.是否免人审_x), df_rule_data.是否免人审_y, df_rule_data.是否免人审_x)
df_rule_data['是否拒量'] = np.where(pd.isnull(df_rule_data.是否拒量_x), df_rule_data.是否拒量_y, df_rule_data.是否拒量_x)
df_rule_data['归属渠道'] = np.where(pd.isnull(df_rule_data.归属渠道_x), df_rule_data.归属渠道_y, df_rule_data.归属渠道_x)
df_rule_data['union_rent_tag'] = np.where(pd.isnull(df_rule_data.union_rent_tag_x), df_rule_data.union_rent_tag_y, df_rule_data.union_rent_tag_x)
df_rule_data['union_rent_rejected'] = np.where(pd.isnull(df_rule_data.union_rent_rejected_x), df_rule_data.union_rent_rejected_y, df_rule_data.union_rent_rejected_x)

df_rule_data = df_rule_data.drop(columns = ['下单日期_x','下单日期_y','search_time_x','search_time_y','是否免人审_x','是否免人审_y','是否拒量_x','是否拒量_y','归属渠道_x','归属渠道_y','union_rent_tag_x','union_rent_tag_y','union_rent_rejected_x','union_rent_rejected_y'])
df_rule_data.shape
df_rule_data[:1]

(373843, 38)

(772327, 24)

(1146170, 58)

(1146170, 51)

,order_number,order_id,trace_id,机审拒绝,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人_x,命中刑事案件_x,命中借贷纠纷_x,命中融安分低于680_x,命中高风险名单_x,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒,前置拒绝,年龄超过49岁或低于18岁,命中法院失信/限高被执行人_y,命中刑事案件_y,命中借贷纠纷_y,命中融安分低于680_y,命中高风险名单_y,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏,审核状态,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected
0,A202206131732031,8747,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,人审拒绝,2022-06-13,2022-06,0.0,0.0,未知渠道,NaN,NaN


In [88]:
# （13）进件和前置拦截-明细

df_rule_data = df_rule_data.fillna(0)
df_rule_data[:1]

df_rule_data.shape
# df_rule_data.drop_duplicates(subset=['order_id']).shape
# df_rule_data.drop_duplicates(subset=['trace_id']).shape
# df_rule_data.drop_duplicates(subset=['order_id','trace_id']).shape
# # (395509, 44)
# df_rule_data[df_rule_data.trace_id == 0].shape
# df_rule_data[df_rule_data.order_id == 0].shape
# df_rule_data[(df_rule_data.trace_id == 0) & (df_rule_data.order_id == 0)].shape
# df_rule_data[(df_rule_data.trace_id != 0) & (df_rule_data.order_id != 0)][['order_id','trace_id']][:2]
# trace_id 重复， order_id不重复，中间表有缓存
# df_rule_data[df_rule_data.trace_id == '441581199811230010'][['order_id','trace_id']]
# 441581199811230010                     10
# 947affface19108aada119d330bac244        7

,order_number,order_id,trace_id,机审拒绝,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人_x,命中刑事案件_x,命中借贷纠纷_x,命中融安分低于680_x,命中高风险名单_x,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒,前置拒绝,年龄超过49岁或低于18岁,命中法院失信/限高被执行人_y,命中刑事案件_y,命中借贷纠纷_y,命中融安分低于680_y,命中高风险名单_y,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏,审核状态,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected
0,A202206131732031,8747,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,人审拒绝,2022-06-13,2022-06,0.0,0.0,未知渠道,0,0


(1146170, 51)

In [89]:
# （14）进件数据
# df_risk_new：申请数据
# 仅取进件的数据， 拒绝理由 == '[' 的可能 可能是拒件 也可能是出库， 看 审核状态
df_jj = df_risk_new[df_risk_new['进件']=='进件']
df_jj = df_jj[['order_number','下单日期','search_time','归属渠道','是否免人审','审核状态','拒绝理由','是否前置拦截','是否机审强拒','是否出库前风控强拒','是否拒量','union_rent_tag','union_rent_rejected','user_mobile','id_card_num','jd_user_level_risk']]
df_jj = df_jj.rename(columns = {'union_rent_tag': '是否联合运营', 'union_rent_rejected': '联合运营命中风控'})
df_jj.shape
df_jj.tail()

# 释放空间
del df
del df2
del df_reject_risk_merge
del df_reject_before_risk_merge
del df_risk 
del df_re 
del df_ra 
del df_xmy 
del df_risk_all2 
del df_reject_before 
del df_reject_before_risk

(373843, 16)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk
2541223,A20250602193255202,2025-06-02,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,18760021819,350824199802244177,None
2541224,A20250602203051146,2025-06-02,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,13161797383,612525199708145418,None
2541225,A20250602203547183,2025-06-02,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,17769715431,342501199104244216,None
2541226,A20250602205309278,2025-06-02,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,15185961146,522226199512300031,None
2541227,A20250602094830188,2025-06-02,2025-06,芝麻租物,0,出库,[,0,0,0,0,1,[命中自有模型强拒],13660885012,51138119970509565X,None


In [90]:
# （15）数据检查
# df_jj.shape
# df_jj[:1]
# df_jj.是否拒量.value_counts()
# df_jj.是否联合运营.value_counts()
# df_jj[df_jj.是否联合运营 == '1']
# df_rule_data[:1]

# df_risk.shape
# df_risk[:3]
# df_risk.union_rent_tag.value_counts()
# df_risk[df_risk.union_rent_tag == '1'].shape

# df[df.union_rent_tag == '1'].shape

# df_risk[df_risk.union_rent_tag == '1'][['trace_id']]
# df[df.union_rent_tag == '1'][['trace_id']]

# 以下三个 trace_id 在 df_risk，但不在 df
# 8f80fc9a5c1b864e9e41260de3a86273
# 8ff42b701a73228a59f360e851b630aa
# 47cf27a705f026fadcefef09090ef2df

# df.shape
# df[df.trace_id == '8ff42b701a73228a59f360e851b630aa']

# 免押数据
# df_jj.拒绝理由.value_counts()
# tmp = df_jj.groupby(['拒绝理由']).agg({'order_number': 'count'}).reset_index()
# tmp = tmp.sort_values(by = 'order_number', ascending = False)
# tmp.to_excel('tmp.xlsx', index = False)

# df_jj[df_jj.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝'])].shape

# tmp = df_jj[(df_jj.search_time == '2025-02') & (~df_jj.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))][['order_number','下单日期','search_time','归属渠道']]
# tmp.shape
# tmp.to_excel('免押进件_202502.xlsx', index = False)
# tmp[:5]

# 17135 - 1351= 15784

# # 联合运营订单漏斗
# sql = '''
# SELECT  id as order_id, order_number, create_time, union_rent_tag, status
# from  db_digua_business.t_order
# where union_rent_tag = 'Y'
# ;
# '''
# tmp1 = query(sql)
# tmp1.shape

# # 有3单未关联到，为测试白名单
# # 拒绝理由 为前置拒绝理由，union_rent_rejected 为机审拒绝，因为联合运营的机审豁免了，所以单独存放机审拒绝
# # ,om.user_mobile,tmu.true_name,tmu.id_card_num
# var_1 = ['order_number','拒绝理由','审核状态','下单日期','search_time','union_rent_rejected','union_rent_tag','product_name','进件','user_mobile','id_card_num']
# df_lhyy_1 = tmp1.merge(df_risk_new[var_1], on = 'order_number', how = 'inner')
# df_lhyy_1 = df_lhyy_1[df_lhyy_1.下单日期 >= '2025-03-18']
# df_lhyy_1.shape
# df_lhyy_1[:2]
# df_lhyy_1 = df_lhyy_1.merge(df_out_data[['order_number','grade','bj_qy_score','br_ra_score','rentOnlineDetailLevelOverall']], on = 'order_number', how = 'left')
# df_lhyy_1['grade_new'] = df_lhyy_1.apply(lambda x: 'F' if x.bj_qy_score <= 400 and x.br_ra_score <= 799
#                                          else 'E' if x.bj_qy_score <= 400 and x.br_ra_score > 799

#                                          else 'F' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score <= 679
#                                          else 'D' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 679 and x.br_ra_score <= 799
#                                          else 'C' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'B' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 839 and x.br_ra_score <= 879
#                                          else 'A' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 879

#                                          else 'F' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score <= 679
#                                          else 'D' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 679 and x.br_ra_score <= 799
#                                          else 'C' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'B' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 839 and x.br_ra_score <= 879
#                                          else 'A' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 879
                                         
#                                          else 'F' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score <= 679
#                                          else 'D' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 679 and x.br_ra_score <= 709
#                                          else 'C' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 709 and x.br_ra_score <= 769
#                                          else 'B' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 769 and x.br_ra_score <= 879
#                                          else 'A' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 879 and x.br_ra_score <= 910
#                                          else 'S' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 910

#                                          else 'F' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 679 and x.br_ra_score <= 769
#                                          else 'B' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 769 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 799 and x.br_ra_score <= 879
#                                          else 'S' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 879

#                                          else 'F' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 679 and x.br_ra_score <= 769
#                                          else 'B' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 769 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 839
                                         
#                                          else 'F' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 679 and x.br_ra_score <= 739
#                                          else 'B' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 739 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 839

#                                          else 'F' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 679 and x.br_ra_score <= 709
#                                          else 'B' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 709 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 839

#                                          else 'F' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score <= 679
#                                          else 'B' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score > 679 and x.br_ra_score <= 769
#                                          else 'A' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score > 769 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score > 839

#                                          else 'F' if x.bj_qy_score > 680 and x.br_ra_score <= 679
#                                          else 'B' if x.bj_qy_score > 680 and x.br_ra_score > 679 and x.br_ra_score <= 709
#                                          else 'A' if x.bj_qy_score > 680 and x.br_ra_score > 709 and x.br_ra_score <= 799
#                                          else 'S' if x.bj_qy_score > 680 and x.br_ra_score > 799

#                                          else 'other', axis = 1)
# df_analy_data['是否出库'] = 1
# df_lhyy_1 = df_lhyy_1.merge(df_analy_data[['order_number','是否出库']], on = 'order_number', how = 'left')
# df_lhyy_1.to_excel('llyy_tmp.xlsx', index = False)


# # 拒量
# df_risk_new[df_risk_new['是否拒量'] == 1].shape
# df_risk_new[(df_risk_new['是否拒量'] == 1) & (df_risk_new.下单日期>='2025-03-17') & (df_risk_new.下单日期<='2025-03-23')].审核状态.value_counts()
# # df_risk_new[(df_risk_new['是否拒量'] == 1) & (df_risk_new.下单日期>='2025-03-17') & (df_risk_new.下单日期<='2025-03-23')].drop_duplicates(subset = ['id_card_num']).拒绝理由.value_counts()
# df_risk_new[(df_risk_new['是否拒量'] == 1) & (df_risk_new.下单日期>='2025-03-17') & (df_risk_new.下单日期<='2025-03-23')].拒绝理由.value_counts()

# tmp = df_risk_new[df_risk_new['是否拒量'] == 1][['下单日期','order_number','tips','审核状态']]
# tmp['是否出库'] = np.where(tmp.审核状态 == '出库', 1, 0)
# # tmp['策略命中等级'] = tmp['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# tmp['策略命中等级'] = tmp['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# tmp = tmp[['下单日期','order_number','策略命中等级','是否出库']]
# # tmp.to_excel('拒量数据_20250422.xlsx', index = False)
# tmp.to_excel('拒量数据_20250507.xlsx', index = False)

#八派
# var_1 = ['order_number','拒绝理由','审核状态','下单日期','search_time','union_rent_rejected','union_rent_tag','product_name','进件','user_mobile','id_card_num']
# condition_1 = (df_risk_new.归属渠道== '八派信息')
# df_risk_new[condition_1][var_1].to_excel('八派.xlsx', index = False)

# 聚量派
# var_1 = ['order_number','拒绝理由','审核状态','下单日期','search_time','union_rent_rejected','union_rent_tag','product_name','进件','user_mobile','id_card_num']
# condition_1 = (df_risk_new.归属渠道== '聚量派')
# df_risk_new[condition_1][var_1].to_excel('聚量派.xlsx', index = False)

# 蚁盾分
# df_risk_new[(df_risk_new.拒绝理由.isin(['[命中蚁盾分>50]','[蚁盾分>=80]','[命中蚁盾分>50, 蚁盾分>=80]']))&(df_risk_new.下单日期>='2025-03-01')].groupby(['下单日期']).agg({'order_number':'count'})




# 5.自变量

## （5-1-1）中间表

In [91]:
# （5-1-1）中间表
# trace_df 关联其他表，有缓存的用 trace_id 关联，没缓存的用 order_trace_id 关联
# 2025.3.19：目前存在以下情况，各数据源的表有数据，但中间表没有，主要是美宜租、小蚂蚁或者23年及之前的数据，非免押也会流到小蚂蚁，到小蚂蚁之后会调贷前风控，中间表只含自营订单，故会出现数据偏差
sql1='''-- trace关联表 
SELECT * 
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape
trace_df[trace_df.order_id.isnull()].shape
trace_df[trace_df.order_number.isnull()].shape
trace_df[trace_df.trace_id.isnull()].shape
# trace_df[trace_df.trace_id.isnull()].tail()
# trace_df.drop_duplicates(subset=['trace_id']).shape

# trace_id 有重复
# order_trace_id 有重复，没有空
# order_id 无重复，没有空
# trace_df[trace_df['order_trace_id'] == '441581199811230010']

# 三者非空的时候，order_trace_id = risk_trace_id，但 trace_id 不一定和risk_trace_id/order_trace_id相等
# tmp1 = trace_df[(trace_df.trace_id.notna()) & (trace_df.order_trace_id.notna()) & (trace_df.risk_trace_id.notna())]
# tmp1.shape
# tmp1[tmp1.trace_id== tmp1.order_trace_id].shape
# tmp1[tmp1.trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.order_trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.trace_id!= tmp1.risk_trace_id][:1]

# （2）区别用哪个模型
# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
# 4: ascore2501
sql1='''-- risk关联表 
SELECT trace_id, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
from db_credit.risk
;
'''
risk_df_model = query(sql1)
risk_df_model.shape
# risk_df_model[:2]

# 这个关联方式有问题
# trace_df = trace_df.merge(risk_df_model, on = 'trace_id', how = 'left')
# trace_df.shape
# trace_df[:1]

(2269442, 5)

(0, 5)

(0, 5)

(26703, 5)

(358167, 2)

## （5-2-1）火眸分

In [92]:
# （5-2-1）火眸分
sql1='''-- 风控-冰鉴-火眸分
SELECT * 
from db_credit.risk_icekredit_cds
;
'''

bj_hm_df =query(sql1)
bj_hm_df.shape

bj_hm_df['bj_hm_score'] = bj_hm_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('card_device_score22', {}).get('bj_score', None))  
bj_hm_df['bj_hm_score'] = bj_hm_df['bj_hm_score'].astype(int)
bj_hm_df = bj_hm_df[['trace_id', 'bj_hm_score']]



df_out_data = trace_df.merge(bj_hm_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del bj_hm_df

(1229505, 5)

(2269442, 6)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0


## （5-3-1）芝麻租物-风控专业版/风控进阶版

In [93]:
# （5-3-1）芝麻租物-风控专业版/风控进阶版 --调用周期-7天
# 细化综合风险等级 rentOnlineDetailLevelOverall  1-2024.9-2024.12切流调用，2025.1.1开始切流100%；2-线下测试 203.10-2024.10，仅租物渠道
# 综合风险等级 rentOnlineRiskLevelOverall
# 共租行为等级 rentOnlineRiskLevelMultiRent
# 履约历史等级 rentOnlineRiskLevelFulfill

sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT *
from db_credit.risk_alipay_auth_query
;
'''

zm_fk_df = query(sql1)
zm_fk_df = zm_fk_df[['trace_id', 'rentOnlineRiskLevelOverall', 'rentOnlineRiskLevelMultiRent', 'rentOnlineRiskLevelFulfill','rentOnlineDetailLevelOverall']]
zm_fk_df.shape
zm_fk_df[zm_fk_df.trace_id.isnull()].shape
# zm_fk_df[:1]

# outer 关联出来的 order_number 都是空值，没意义
# df_out_data = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'outer')
df_out_data = df_out_data.merge(zm_fk_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del zm_fk_df

(296764, 5)

(0, 5)

(2269442, 10)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN


## （5-3-2）细化综合风险等级

In [94]:
# （5-3-2）细化综合风险等级-补充
# 2023.10及之后的样本，本次选取 2023.10-2024.10 的进件（出库），trace_id、逾期标签（M1+、M3+、FPD31+）、品类（全新、二手、非手机）
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

# 2025.1.1全面调用，2024.11-2024.12的样本为随机线上调用的（2024.09-2024.12线上随机调用）

# 返回结果 outer_order_id 去重
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/5_联合运营/2_数据返回/'
df_lhyy_in = pd.read_excel(path + 'lhyy_sample_data_diguazuji_return.xlsx')
df_lhyy_in = df_lhyy_in.rename(columns = {'outer_order_id': 'out_order_no'})
df_lhyy_in.shape
df_lhyy_in.drop_duplicates(subset = ['out_order_no']).shape

# # （2-3-1）取 trace_id
# sql1='''-- trace关联表 
# SELECT trace_id, order_number
# from db_credit.order_association
# ;
# '''

# trace_df =query(sql1)
# trace_df.shape

# （2-3-2）关联 out_order_no
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT trace_id,out_order_no
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df.shape

# （2-3-3）表关联
trace_df = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
trace_df.shape

# 发散，有些没有租后表现
df_lhyy_in = df_lhyy_in.merge(trace_df, on = 'out_order_no', how = 'inner')
df_lhyy_in.shape
df_lhyy_in.drop_duplicates(subset = ['trace_id']).shape
# df_lhyy_in[df_lhyy_in.out_order_no == 'A2024082909142423']
df_lhyy_in.drop_duplicates(subset = ['out_order_no']).shape
df_lhyy_in.drop_duplicates(subset = ['order_number']).shape

df_lhyy_in = df_lhyy_in[['order_number','zm_lvl']]
df_lhyy_in['zm_lvl'] = df_lhyy_in.apply(lambda x: '0' if x.zm_lvl == 'R0' 
                                        else '1' if x.zm_lvl == 'R1'
                                        else '2' if x.zm_lvl == 'R2'
                                        else '3' if x.zm_lvl == 'R3'
                                        else '4' if x.zm_lvl == 'R4'
                                        else '5' if x.zm_lvl == 'R5'
                                        else '6' if x.zm_lvl == 'R6'
                                        else x.zm_lvl, axis = 1)
# df_lhyy_in[:1]

# 关联到主表
df_out_data = df_out_data.merge(df_lhyy_in, on = 'order_number', how = 'left')
# 同时存在,以线上调用的结果为准
df_out_data['rentOnlineDetailLevelOverall'] = np.where(pd.isnull(df_out_data.rentOnlineDetailLevelOverall), df_out_data.zm_lvl, df_out_data.rentOnlineDetailLevelOverall)
df_out_data = df_out_data.drop(columns = ['zm_lvl'])
df_out_data.shape
df_out_data[:1]

# del trace_df
del zm_fk_df
del df_lhyy_in

# 17941
# tmp = df_lhyy_in.merge(df_analy_data, on = 'order_number', how = 'inner')
# tmp.shape

(17902, 2)

(17902, 2)

(296764, 2)

(2269442, 6)

(24751, 7)

(17902, 7)

(17902, 7)

(24751, 7)

(2269442, 10)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN


## （5-3-3）蚂蚁链-蚁盾分1.0

In [95]:
# （5-3-3）蚂蚁链-蚁盾分1.0
sql1='''-- 风控-蚂蚁链租赁风险分
-- 122905	122905	122905
SELECT trace_id, score as my_zl_score
from db_credit.risk_antchain
;
'''

my_zl_df=query(sql1)
my_zl_df['my_zl_score'] = my_zl_df['my_zl_score'].astype(float)
my_zl_df.shape

# 11454 条关联不到 	order_id	order_number	
# df_out_data = df_out_data.merge(my_zl_df, on = 'trace_id', how = 'outer')
df_out_data = df_out_data.merge(my_zl_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1] 

del my_zl_df 

(334047, 2)

(2269442, 11)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419


## （5-3-4）蚂蚁链-蚁盾分2.0

In [96]:
# （5-3-4）蚂蚁链-蚁盾分2.0
# 1-2024.12线下回溯，样本详情：
# （1）2024.9-2024.11 的出库订单 --评估稳定性、区分度
# （2）2024.12.1-2024.12.17 的进件 --评估单条规则命中率、独立命中率
# afz_v1_score ：租赁通用分，取值 -1 表示库无记录
path_my = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分/1.20241218/'
data_my_out = pd.read_excel(path_my + 'my_sample_20241218.xlsx')
data_my_in_1 = pd.read_csv(path_my + '租赁通用分V1_迪瓜_241220.csv')

data_my_in_1 = data_my_in_1[['certno_hash','mob_hash','apply_date','afz_v1_score']]
data_my_in_1['afz_v1_score'] = np.where(data_my_in_1.afz_v1_score == -1, -999, data_my_in_1.afz_v1_score)
data_my_in_1["apply_date"] = data_my_in_1["apply_date"].astype('str')
data_my_in_1[data_my_in_1.afz_v1_score == -999].shape
data_my_in_1 = data_my_in_1.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','afz_v1_score'])

data_my_out = data_my_out[['order_number','order_id', '下单日期', '归属渠道','idcard_sha256','mobile_sha256']]
data_my_out = data_my_out.rename(columns = {'idcard_sha256': 'certno_hash', 'mobile_sha256': 'mob_hash'})
data_my_out["apply_date"] = data_my_out["下单日期"].astype('str')
data_my_out["apply_date"] = data_my_out["apply_date"].str.replace('-','')
data_my_out['search_time'] = pd.to_datetime(data_my_out["下单日期"]).dt.strftime('%Y-%m')

data_my_out.shape
data_my_in_1.shape

data_my = data_my_out.merge(data_my_in_1, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
# data_my.drop(columns=['apply_date','certno_hash','mob_hash'],inplace = True)
data_my = data_my[['order_number','afz_v1_score']]
data_my.shape
data_my[:2]

del data_my_in_1
del data_my_out

# 2-线下回溯 2024.12.16-2025.1.31 有租后表现的 + 2025.2.1-2025.2.19 免押进件（2025.2.19 上线）
path_my = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分/2.20250320/'
data_my_out = pd.read_excel(path_my + 'my_sample_20250320.xlsx')
data_my_in_1 = pd.read_csv(path_my + '租赁通用分V1_20250401_DGXJ_ee2_zl_250326.csv')

data_my_in_1 = data_my_in_1[['certno_hash','mob_hash','apply_date','afz_v1_score']]
data_my_in_1['afz_v1_score'] = np.where(data_my_in_1.afz_v1_score == -1, -999, data_my_in_1.afz_v1_score)
data_my_in_1["apply_date"] = data_my_in_1["apply_date"].astype('str')
data_my_in_1[data_my_in_1.afz_v1_score == -999].shape
data_my_in_1 = data_my_in_1.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','afz_v1_score'])

data_my_out = data_my_out[['order_number','order_id', '下单日期', '归属渠道','idcard_sha256','mobile_sha256']]
data_my_out = data_my_out.rename(columns = {'idcard_sha256': 'certno_hash', 'mobile_sha256': 'mob_hash'})
data_my_out["apply_date"] = data_my_out["下单日期"].astype('str')
data_my_out["apply_date"] = data_my_out["apply_date"].str.replace('-','')
data_my_out['search_time'] = pd.to_datetime(data_my_out["下单日期"]).dt.strftime('%Y-%m')

data_my_out.shape
data_my_in_1.shape

data_my_1 = data_my_out.merge(data_my_in_1, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
# data_my.drop(columns=['apply_date','certno_hash','mob_hash'],inplace = True)
data_my_1 = data_my_1[['order_number','afz_v1_score']]
data_my_1.shape

data_my = pd.concat([data_my,data_my_1])
data_my.shape
data_my = data_my.drop_duplicates(subset = 'order_number')
data_my.shape

del data_my_in_1
del data_my_out

# 3-关联到主表
# 17763
# tmp = df_out_data[['order_number']].merge(data_my[['order_number']], on = 'order_number', how = 'inner')
df_out_data = df_out_data.merge(data_my, on = 'order_number', how = 'left')



# 4-线上调 2025-02-19
sql1='''-- 风控-蚂蚁链租赁风险分
-- 122905	122905	122905
SELECT trace_id, score as my_zl_score2
from db_credit.risk_antchain_ato
;
'''

my_zl_df_2 = query(sql1)
my_zl_df_2['my_zl_score2'] = my_zl_df_2['my_zl_score2'].astype(float)
my_zl_df_2.shape

df_out_data = df_out_data.merge(my_zl_df_2, on = 'trace_id', how = 'left')
# df_out_data['my_zl_score2'] = np.where(pd.isnull(df_out_data.afz_v1_score), df_out_data.my_zl_score2, df_out_data.afz_v1_score)
df_out_data['my_zl_score2'] = np.where(pd.isnull(df_out_data.my_zl_score2), df_out_data.afz_v1_score, df_out_data.my_zl_score2)
df_out_data = df_out_data.drop(columns = ['afz_v1_score'])
df_out_data.shape
df_out_data[:1] 

del my_zl_df_2 
del data_my

# df_out_data[df_out_data.my_zl_score2 > 0][['order_number','my_zl_score2']][:5]
# order_number	my_zl_score2
# 1657388	A20250212142919191	77.3086
# 1658578	A20250212174156265	68.2489
# 1659580	A2025021300115318	52.5108
# 1660252	A2025021310090343	51.0409
# 1660308	A20250213102333106	56.4121

(16, 4)

(17764, 8)

(17757, 4)

(17764, 2)

,order_number,afz_v1_score
0,A20240901005306132,47.5799
1,A20240901005509136,57.9553


...

(30069, 2)

(30069, 2)

(91238, 2)

(2269442, 12)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN


## （5-3-5）百融-风险勘测

In [97]:
# （5-3-5）百融-风险勘测

## （5-3-6）百融-云商分

In [98]:
# （5-3-6）百融-云商分
sql1='''-- 风控-百融贷前-云商分
-- 101331	101331	101331
SELECT *
from db_credit.risk_bairong_scoreysstd
;
'''

br_ys_df =query(sql1)
br_ys_df.shape

br_ys_df['br_ys_score'] = br_ys_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))  
br_ys_df['br_ys_score'] = br_ys_df['br_ys_score'].astype(int)
br_ys_df = br_ys_df[['trace_id', 'br_ys_score']]

# 3263 条关联不到 	order_id	order_number	
# df_out_data = df_out_data.merge(br_ys_df, on = 'trace_id', how = 'outer')
df_out_data = df_out_data.merge(br_ys_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

# 在 br_ys_df 有，但在 trace_df 没有，3000多条，2024.12 有511条
# bc74db481731377076457110184c188f	
# e3f5baf3d90ea794fdb6fb3a28dd8dc5

# df_out_data[df_out_data.trace_id == 'bc74db481731377076457110184c188f']
# br_ys_df[br_ys_df.trace_id == 'bc74db481731377076457110184c188f']

# df_out_data[df_out_data.trace_id == 'e3f5baf3d90ea794fdb6fb3a28dd8dc5']
# br_ys_df[br_ys_df.trace_id == 'e3f5baf3d90ea794fdb6fb3a28dd8dc5']

del br_ys_df

(306761, 6)

(2269442, 13)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN


## （5-3-7）百融-融安分

In [99]:
# （5-3-7）百融-融安分
# 仅取融安分-大表
sql1='''-- 风控-百融 
SELECT rb.trace_id
    ,replace(case when JSON_VALID(rb.query_rs) THEN JSON_EXTRACT(rb.query_rs, '$.Score.scoreprime') end,'"','') as br_ra_score
from db_credit.risk_bairong rb 
-- LIMIT 100 
;
'''

br_ra_df =query(sql1)
br_ra_df['br_ra_score'] = br_ra_df['br_ra_score'].astype(float)
br_ra_df.shape

# 百融-融安分A-主流-标准款-小表
# db_credit.risk_bairong_scoreprime
sql1='''-- 风控-百融 
SELECT trace_id, query_rs, query_time
from db_credit.risk_bairong_scoreprime
;
'''
br_ra_df_1 = query(sql1)
# br_ra_df_1['search_time'] = br_ra_df_1['query_time'].dt.strftime('%Y-%m')

br_ra_df_1['br_ra_score'] = br_ra_df_1['query_rs'].apply(lambda x: json.loads(x).get('RiskStrategy', {}).get('Score', {}).get('scoreprime', None))  
br_ra_df_1['br_ra_score'] = pd.to_numeric(br_ra_df_1['br_ra_score'].replace('"null"', np.nan), errors='coerce')
br_ra_df_1 = br_ra_df_1[['trace_id','br_ra_score']]

# 合并
br_ra_df = pd.concat([br_ra_df, br_ra_df_1])
br_ra_df.shape
br_ra_df = br_ra_df.drop_duplicates(subset = 'trace_id')

# 关联
# df_out_data = df_out_data.drop(columns = 'br_ra_score')
df_out_data = df_out_data.merge(br_ra_df,on = 'trace_id',how='left')
df_out_data.shape
df_out_data[:1]

del br_ra_df
del br_ra_df_1

# 在 br_ra_df 有，但在 trace_df 没有，25000条左右，2024.12 有511条
# sql1='''-- 风控-百融 
# SELECT rb.trace_id
#     ,replace(case when JSON_VALID(rb.query_rs) THEN JSON_EXTRACT(rb.query_rs, '$.Score.scoreprime') end,'"','') as br_ra_score
#     ,query_time
# from db_credit.risk_bairong rb 
# # LIMIT 100 
# ;
# '''

# br_ra_df =query(sql1)
# br_ra_df["下单日期"] = br_ra_df['query_time'].dt.date
# br_ra_df["下单日期"] = pd.to_datetime(br_ra_df["下单日期"],errors="coerce")
# br_ra_df['search_time'] = pd.to_datetime(br_ra_df["下单日期"]).dt.strftime('%Y-%m')
# br_ra_df.shape
# br_ra_df.drop_duplicates(subset = 'trace_id').shape

# tmp1 = df_out_data.merge(br_ra_df,on = 'trace_id',how='right')
# tmp1.shape
# tmp1[tmp1.br_ra_score_x.isnull()].shape
# tmp1[tmp1.br_ra_score_x.isnull()].drop_duplicates(subset = 'trace_id').shape
# tmp1[tmp1.br_ra_score_x.isnull()].drop_duplicates(subset = 'trace_id').search_time.value_counts()

# tmp1[(tmp1.br_ra_score_x.isnull())&(tmp1.search_time == '2025-01')][:3]
# 1979578101591333f0a05da8bcfacf16
# a1ca07b662b4ce93755f69421cc86ba3
# 1784690257903de4551080402ee0a996
# df_out_data[df_out_data.trace_id == '1979578101591333f0a05da8bcfacf16']
# br_ra_df[br_ra_df.trace_id == '1979578101591333f0a05da8bcfacf16']

# df_out_data[df_out_data.trace_id == 'a1ca07b662b4ce93755f69421cc86ba3']
# br_ra_df[br_ra_df.trace_id == 'a1ca07b662b4ce93755f69421cc86ba3']

(406760, 2)

(418877, 2)

(2269442, 14)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0


## （5-3-8-1）自有模型-ascore2311

In [100]:
# （5-3-8-1）自有模型-ascore2311 
sql1='''-- 风控-自有模型-ascore2311 
SELECT * 
from db_credit.risk_private_model
;
'''

mx_2311_df =query(sql1)
mx_2311_df.shape

mx_2311_df['model_score_2311'] = pd.to_numeric(mx_2311_df['score'], errors='coerce')

mx_2311_df = mx_2311_df[['trace_id', 'model_score_2311']]
mx_2311_df[:1]

(279971, 7)

,trace_id,model_score_2311
0,12f8bdc71e41d304140df3a262cc4951,19.514414


## （5-3-8-2）自有模型-2401

In [101]:
# （5-3-8-2）自有模型-2401
sql1='''-- 风控-自有模型-2401
SELECT * 
from db_credit.risk_private_model_2401
;
'''

mx_2401_df =query(sql1)
mx_2401_df.shape

mx_2401_df['model_score_2401'] = pd.to_numeric(mx_2401_df['score'], errors='coerce')
mx_2401_df = mx_2401_df[['trace_id', 'model_score_2401']]
mx_2401_df[:1]

(257371, 7)

,trace_id,model_score_2401
0,ba170e63115f61a35da1ea8e662eda36,16.362452


## （5-3-8-3）自有模型-yl

In [102]:
# （5-3-8-3）自有模型-yl 
sql1='''-- 风控-自有模型-yl
SELECT * 
from db_credit.risk_private_model_yl
;
'''

mx_yl_df =query(sql1)
mx_yl_df.shape

mx_yl_df['model_score_yl'] = pd.to_numeric(mx_yl_df['score'], errors='coerce')
mx_yl_df = mx_yl_df[['trace_id', 'model_score_yl']]
mx_yl_df[:1]

(279886, 7)

,trace_id,model_score_yl
0,12f8bdc71e41d304140df3a262cc4951,16.430973


## （5-3-8-4）自有模型-2501

In [103]:
# （5-3-8-4）自有模型-2501
sql1='''-- 风控-自有模型-2501
SELECT * 
from db_credit.risk_private_model_2501_base
;
'''

mx_2501_df =query(sql1)
mx_2501_df.shape

mx_2501_df['model_score_2501'] = pd.to_numeric(mx_2501_df['score'], errors='coerce')
mx_2501_df = mx_2501_df[['trace_id', 'model_score_2501']]
mx_2501_df[:1]

# 线下调用
# 自有模型-2501：250121上线，上线后2个星期试运行，20250210增量运行，20250218增量运行改为并行，随机切流30%
# 20250121-20250205左右的数据不要用，入模有个变量类型有误，导致模型评分有差异
# 监控自2025.2.18起并行的样本

(105498, 7)

,trace_id,model_score_2501
0,b2358986620820d3e7b53201311b7eae,24.780494


## （5-3-8-5）模型关联

In [104]:
# （5-3-8-5）模型关联
df_model = mx_2311_df.merge(mx_2401_df, on = 'trace_id', how='outer')
df_model.shape

df_model = df_model.merge(mx_yl_df, on = 'trace_id', how='outer')
df_model.shape

df_model = df_model.merge(mx_2501_df, on = 'trace_id', how='outer')
df_model.shape

# df_model.tail()
df_model = df_model.merge(risk_df_model, on='trace_id', how='left')
df_model.shape

# A202407222000361 1277221 ee3e7156005b1e040f7ca4a85455a796，在 df_model，不在 trace_df，往前回溯的时候少量数据侧漏
df_out_data = df_out_data.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='left')
# df_out_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='left')
# df_out_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='outer')
# 185431
# df_out_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='inner')
df_out_data.shape

df_out_data.drop('trace_id_y', axis=1, inplace=True)
df_out_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_out_data.shape
df_out_data[:2]

# risk_trace_id 可能有空， 但 order_trace_id 较全面
# df_out_data[df_out_data.order_trace_id == df_out_data.risk_trace_id].shape
# df_out_data[(df_out_data.order_trace_id != df_out_data.risk_trace_id)].shape
# df_out_data[(df_out_data.order_trace_id != df_out_data.risk_trace_id) & (df_out_data.risk_trace_id.notnull())].shape
# df_out_data[(df_out_data.order_trace_id != df_out_data.risk_trace_id) & (df_out_data.order_trace_id.notnull())].shape

# 下面是另外一种关联模式，结果一样的，只是结果表的trace_id为 trace_df 的 order_trace_id
# df_model = mx_2311_df[['trace_id', 'model_score_2311']].merge(mx_2401_df[['trace_id', 'model_score_2401']], on = 'trace_id', how='outer')
# df_model.shape

# df_model = df_model.merge(mx_yl_df[['trace_id', 'model_score_yl']], on = 'trace_id', how='outer')
# df_model.shape

# # df_model = df_model.merge(risk_df, on='', how='inner')
# df_model = df_model.merge(risk_df_model, on='trace_id', how='left')
# df_model.shape
# df_model[:2]

# temp = df_out_data.merge(df_model, left_on = 'order_trace_id', right_on = 'trace_id', how = 'inner')
# temp = df_out_data.merge(df_model, on = 'trace_id', how = 'inner') -- 0
# temp.shape
# temp[:5]
# temp[temp.model_score_2311_x !=temp.model_score_2311_y].shape
# temp[(temp.model_score_yl_x !=temp.model_score_yl_y)&(temp.model_score_yl_x.notnull())].shape
# temp[(temp.model_score_2401_x !=temp.model_score_2401_y)&(temp.model_score_2401_x.notnull())].shape

# trace_df[trace_df.order_trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']
# trace_df[trace_df.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']
# risk_df_model[risk_df_model.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']

# df_out_data[df_out_data.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']
# df_out_data[df_out_data.order_trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']

# df_model[df_model.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']

del mx_2311_df
del mx_2401_df
del mx_yl_df
del df_model

(279971, 3)

(279971, 4)

(279971, 5)

(279971, 6)

(2269442, 20)

(2269442, 19)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN
1,88931,A202301090006044,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,627.0,NaN,NaN,NaN,NaN,23.133,NaN,NaN,752.0,NaN,NaN,NaN,NaN,NaN


## （5-3-9）冰鉴-青云分

In [105]:
# （5-3-9）冰鉴-青云分
sql1='''-- 风控-冰鉴-青云分
SELECT * 
from db_credit.risk_icekredit_qy
;
'''

bj_qy_df =query(sql1)
bj_qy_df.shape

bj_qy_df['bj_qy_score'] = bj_qy_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('qing_yun22', {}).get('bj_score', None))  
bj_qy_df['bj_qy_score'] = bj_qy_df['bj_qy_score'].astype(int)
bj_qy_df = bj_qy_df[['trace_id', 'bj_qy_score']]


df_out_data = df_out_data.merge(bj_qy_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del bj_qy_df 

(345916, 5)

(2269442, 20)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0


## （5-3-10）评分等级

In [106]:
# （5-3-10）评分等级
sql1='''
SELECT * 
from db_credit.risk
;
'''

risk_df =query(sql1)
risk_df.shape

df_out_data = df_out_data.merge(risk_df[['trace_id','grade']], left_on='order_trace_id', right_on='trace_id', how='left')
df_out_data.drop('trace_id_y', axis=1, inplace=True)
df_out_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_out_data.shape

df_out_data.shape
df_out_data[:1]

del risk_df

(358169, 11)

(2269442, 21)

(2269442, 21)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A


## （5-3-11）全景雷达-新颜

In [107]:
# （5-3-11）全景雷达-新颜
# 76个变量
# （1）取数
sql1='''-- 风控-新颜-全景雷达
SELECT * 
from db_credit.risk_xinyan_qjld
;
'''

xy_qj_df =query(sql1)
xy_qj_df.shape
xy_qj_df[:1]

# xy_qj_df.time.max()
# Timestamp('2024-09-21 11:53:09')

# （2）新颜数据处理
def expand_json_columns(df, json_col,prefix='xy_qj_',sep='.'):
    # 只对非空的 JSON 列进行解析
    df['result_parsed'] = df[json_col].apply(lambda x:json.loads(x) if pd.notna(x) else None )

    # 对解析出来的 JSON 字段进行展开，并添加前缀
    df_json_expanded = pd.json_normalize(df['result_parsed'].dropna(),sep=sep)

    # 保留最内层字段名（去除层级前缀）
    df_json_expanded.columns = [col.split(sep)[-1] for col in df_json_expanded.columns]

    # 为所有字段名添加前缀
    df_json_expanded = df_json_expanded.add_prefix(prefix)

    # 将展开的字段拼接到原始 DataFrame 中
    df_final = pd.concat([df,df_json_expanded], axis=1)

     # 删除中间生成的列
    df_final = df_final.drop(columns='result_parsed')
    return  df_final

xy_qj_df = expand_json_columns(xy_qj_df,'rs','xy_qj_')

# xy_obj_cols = ['xy_qj_A22160007','xy_qj_B22170007','xy_qj_B22170008','xy_qj_B22170009','xy_qj_B22170010','xy_qj_B22170011',
# 'xy_qj_B22170031','xy_qj_B22170032','xy_qj_B22170033','xy_qj_B22170034',  #--百分比
# 'xy_qj_B22170040','xy_qj_B22170041','xy_qj_B22170042','xy_qj_B22170043','xy_qj_B22170044','xy_qj_B22170050','xy_qj_B22170054']

# xy_qj_B22170034  去除百分比后  转为float  
xy_qj_df['xy_qj_B22170034'] = pd.to_numeric(xy_qj_df['xy_qj_B22170034'].str.replace('%', ''), errors='coerce') / 100
# xy_qj_df['xy_qj_B22170034'].value_counts()

cols = ['xy_qj_A22160001','xy_qj_A22160002', 'xy_qj_A22160003', 'xy_qj_A22160004',
       'xy_qj_A22160005', 'xy_qj_A22160006', 
       'xy_qj_A22160008', 'xy_qj_A22160009', 'xy_qj_A22160010','xy_qj_C22180001', 'xy_qj_C22180002', 'xy_qj_C22180003',
       'xy_qj_C22180004', 'xy_qj_C22180005', 'xy_qj_C22180006','xy_qj_C22180007', 'xy_qj_C22180008', 'xy_qj_C22180009',
       'xy_qj_C22180010', 'xy_qj_C22180011', 'xy_qj_C22180012','xy_qj_B22170001', 'xy_qj_B22170002', 'xy_qj_B22170003',
       'xy_qj_B22170004', 'xy_qj_B22170005', 'xy_qj_B22170006',
        'xy_qj_B22170012',
       'xy_qj_B22170013', 'xy_qj_B22170014', 'xy_qj_B22170015','xy_qj_B22170016', 'xy_qj_B22170017', 'xy_qj_B22170018',
       'xy_qj_B22170019', 'xy_qj_B22170020', 'xy_qj_B22170021','xy_qj_B22170022', 'xy_qj_B22170023', 'xy_qj_B22170024',
       'xy_qj_B22170025', 'xy_qj_B22170026', 'xy_qj_B22170027','xy_qj_B22170028', 'xy_qj_B22170029', 'xy_qj_B22170030',
        'xy_qj_B22170035', 'xy_qj_B22170036',
       'xy_qj_B22170037', 'xy_qj_B22170038', 'xy_qj_B22170039',
        'xy_qj_B22170045',
       'xy_qj_B22170046', 'xy_qj_B22170047', 'xy_qj_B22170048',
       'xy_qj_B22170049',  'xy_qj_B22170051',
       'xy_qj_B22170052', 'xy_qj_B22170053']

for col in cols:
    xy_qj_df[col] = pd.to_numeric(xy_qj_df[col], errors='coerce')

xy_qj_df  = xy_qj_df.drop(columns=['id', 'params', 'rs', 'time', 'xy_qj_code', 'xy_qj_message', 'xy_qj_requestId',
       'xy_qj_requestTime', 'xy_qj_behavior_report_detail','xy_qj_current_report_detail'])   # ,inplace=True

xy_qj_df.shape
xy_qj_df[:1]

# # （3）关联
# df_out_data = df_out_data.merge(xy_qj_df, on='trace_id', how='left')
# df_out_data.shape
# df_out_data[:1]

(65535, 5)

,id,trace_id,params,rs,time
0,4,1d321ebae2a913a47c3e3c75ab300500,"{""idNo"": ""330501198903132044"", ""name"": ""王玲"", ""...","{""code"": ""success"", ""data"": {""apply_report_det...",2023-11-03 13:43:53


(65535, 77)

,trace_id,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054
0,1d321ebae2a913a47c3e3c75ab300500,542.0,73.0,118.0,88.0,8.0,257.0,2023-11,22.0,60.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,91.0,57600.0,17075.0,25500.0,84.0,524.0,4.0,13.0,34.0,70.0,131.0,"[50000,+)","[50000,+)","[50000,+)","[50000,+)","[50000,+)",9.0,21.0,20.0,20.0,4.0,10.0,23.0,36.0,48.0,36.0,48.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,"[500,1000)","[500,1000)","[500,1000)",0.97,2.0,9.0,18.0,29.0,54.0,"[20000,30000)","[50000,+)","[50000,+)","[50000,+)","[50000,+)",12.0,38.0,94.0,245.0,416.0,"(0,7]",84.0,127.0,719.0,2023-11


## （5-3-12）全景雷达-微聚未来

In [108]:
# （5-3-12）全景雷达-微聚未来
# 76个变量
# （1）
sql1='''-- 风控-微聚未来-全景雷达
SELECT * 
from db_credit.risk_weisuper_qjld
;
'''

xy_risk_weisuper_df =query(sql1)
xy_risk_weisuper_df.shape
xy_risk_weisuper_df[:1]
# xy_risk_weisuper_df.time.min()
# Timestamp('2024-09-21 12:11:08')

# # （2）微聚未来数据处理
def expand_json_columns(df, json_col,prefix='xy_qj_',sep='.'):
    # 只对非空的 JSON 列进行解析
    df['result_parsed'] = df[json_col].apply(lambda x:json.loads(x) if pd.notna(x) else None )

    # 对解析出来的 JSON 字段进行展开，并添加前缀
    df_json_expanded = pd.json_normalize(df['result_parsed'].dropna(),sep=sep)

    # 保留最内层字段名（去除层级前缀）
    df_json_expanded.columns = [col.split(sep)[-1] for col in df_json_expanded.columns]

    # 为所有字段名添加前缀
    df_json_expanded = df_json_expanded.add_prefix(prefix)

    # 将展开的字段拼接到原始 DataFrame 中
    df_final = pd.concat([df,df_json_expanded], axis=1)

     # 删除中间生成的列
    df_final = df_final.drop(columns='result_parsed')
    return  df_final

xy_risk_weisuper_df = expand_json_columns(xy_risk_weisuper_df,'rs','xy_qj_')

# # xy_obj_cols = ['xy_qj_A22160007','xy_qj_B22170007','xy_qj_B22170008','xy_qj_B22170009','xy_qj_B22170010','xy_qj_B22170011',
# # 'xy_qj_B22170031','xy_qj_B22170032','xy_qj_B22170033','xy_qj_B22170034',  #--百分比
# # 'xy_qj_B22170040','xy_qj_B22170041','xy_qj_B22170042','xy_qj_B22170043','xy_qj_B22170044','xy_qj_B22170050','xy_qj_B22170054']

# # xy_qj_B22170034  去除百分比后  转为float  
xy_risk_weisuper_df['xy_qj_B22170034'] = pd.to_numeric(xy_risk_weisuper_df['xy_qj_B22170034'].str.replace('%', ''), errors='coerce') / 100
# # xy_risk_weisuper_df['xy_qj_B22170034'].value_counts()

cols = ['xy_qj_A22160001','xy_qj_A22160002', 'xy_qj_A22160003', 'xy_qj_A22160004',
       'xy_qj_A22160005', 'xy_qj_A22160006', 
       'xy_qj_A22160008', 'xy_qj_A22160009', 'xy_qj_A22160010','xy_qj_C22180001', 'xy_qj_C22180002', 'xy_qj_C22180003',
       'xy_qj_C22180004', 'xy_qj_C22180005', 'xy_qj_C22180006','xy_qj_C22180007', 'xy_qj_C22180008', 'xy_qj_C22180009',
       'xy_qj_C22180010', 'xy_qj_C22180011', 'xy_qj_C22180012','xy_qj_B22170001', 'xy_qj_B22170002', 'xy_qj_B22170003',
       'xy_qj_B22170004', 'xy_qj_B22170005', 'xy_qj_B22170006',
        'xy_qj_B22170012',
       'xy_qj_B22170013', 'xy_qj_B22170014', 'xy_qj_B22170015','xy_qj_B22170016', 'xy_qj_B22170017', 'xy_qj_B22170018',
       'xy_qj_B22170019', 'xy_qj_B22170020', 'xy_qj_B22170021','xy_qj_B22170022', 'xy_qj_B22170023', 'xy_qj_B22170024',
       'xy_qj_B22170025', 'xy_qj_B22170026', 'xy_qj_B22170027','xy_qj_B22170028', 'xy_qj_B22170029', 'xy_qj_B22170030',
        'xy_qj_B22170035', 'xy_qj_B22170036',
       'xy_qj_B22170037', 'xy_qj_B22170038', 'xy_qj_B22170039',
        'xy_qj_B22170045',
       'xy_qj_B22170046', 'xy_qj_B22170047', 'xy_qj_B22170048',
       'xy_qj_B22170049',  'xy_qj_B22170051',
       'xy_qj_B22170052', 'xy_qj_B22170053']

for col in cols:
    xy_risk_weisuper_df[col] = pd.to_numeric(xy_risk_weisuper_df[col], errors='coerce')

# xy_risk_weisuper_df  = xy_risk_weisuper_df.drop(columns=['id', 'params', 'rs', 'time', 'xy_qj_code', 'xy_qj_message', 'xy_qj_requestId',
#        'xy_qj_requestTime', 'xy_qj_behavior_report_detail','xy_qj_current_report_detail'])   # ,inplace=True

xy_risk_weisuper_df  = xy_risk_weisuper_df.drop(columns=['id', 'params', 'rs', 'time', 'xy_qj_returnMsg', 'xy_qj_behavior_report_detail','xy_qj_current_report_detail'
                                                         , 'xy_qj_returnCode', 'xy_qj_apply_report_detail'])   # ,inplace=True
xy_risk_weisuper_df.shape
xy_risk_weisuper_df[:1]

# # （3）关联
xy_risk_qjld_df = pd.concat([xy_qj_df, xy_risk_weisuper_df])
xy_risk_qjld_df.shape
xy_risk_qjld_df = xy_risk_qjld_df.drop_duplicates(subset = ['trace_id'])
xy_risk_qjld_df.shape

df_out_data = df_out_data.merge(xy_risk_qjld_df, on='trace_id', how='left')
df_out_data.shape
df_out_data[:1]

# del xy_risk_qjld_df
del xy_qj_df
del xy_risk_weisuper_df

(145040, 5)

,id,trace_id,params,rs,time
0,1,17d29544198be5e82a0ca291a4a6203c,"{""idNo"": ""864a128343f3f0998c647a89e2af8393"", ""...","{""data"": {""apply_report_detail"": {""A22160001"":...",2024-09-21 12:11:08


(145040, 77)

,trace_id,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054
0,17d29544198be5e82a0ca291a4a6203c,535.0,76.0,58.0,31.0,9.0,155.0,2024-09,10.0,47.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,14.0,11400.0,3514.0,2500.0,85.0,547.0,1.0,5.0,8.0,10.0,13.0,"(0,500)","[3000,5000)","[5000,10000)","[5000,10000)","[10000,20000)",7.0,3.0,0.0,0.0,1.0,5.0,7.0,9.0,10.0,9.0,10.0,0.0,0.0,2.0,3.0,3.0,0.0,0.0,0.0,"(0,500)","[2000,3000)","[2000,3000)",0.77,1.0,4.0,16.0,22.0,22.0,"[1000,2000)","[3000,5000)","[3000,5000)","[5000,10000)","[5000,10000)",11.0,24.0,35.0,63.0,68.0,"(0,7]",84.0,12.0,665.0,2024-09


(210575, 77)

(210482, 77)

(2269442, 97)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-3-13）融360

In [109]:
# （5-3-13）融360
# 线下测试+线上调取

# 1-本次测试有欺诈和信用产品
# （1）2023.7-2024.9 有贷后表现的订单
path_360 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/2_20241218/1.样本发送/'
data_360_sample = pd.read_excel(path_360 + 'ceshi_sample_20241218.xlsx')
data_360_sample = data_360_sample[['order_number','idcard_md5','mobile_md5','下单日期']]
data_360_sample.shape

path_360 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/2_20241218/2.数据返回/维客壹佰/'
data_360_1 = pd.read_csv(path_360 + '维客壹佰-201048占信分臻藏版-支付风险V3.0-100%.csv')
# data_360_2 = pd.read_csv(path_360 + '维客壹佰-201053综合行为评估报告V2.4-100%.csv')
# data_360_3 = pd.read_csv(path_360 + '维客壹佰-201054综合行为评估报告V2.2-100%.csv', encoding = 'gbk')
# data_360_4 = pd.read_csv(path_360 + '维客壹佰-201055综合行为评估报告V2.3-100%.csv', encoding = 'gbk')
data_360_5 = pd.read_csv(path_360 + '维客壹佰-202098占信分臻藏版-行为风险V3.0-99.83%.csv', encoding = 'gbk')
data_360_6 = pd.read_csv(path_360 + '维客壹佰-202100占信分臻藏版-支付风险V3.0-100%.csv', encoding = 'gbk')
# data_360_7 = pd.read_csv(path_360 + '维客壹佰-202123占信分标准版-消费风险V3.0-99.97%.csv')


data_360_1 = data_360_1.rename(columns = {'score':'score_201048'})
data_360_1 = data_360_1[['idcard_md5','mobile_md5','下单日期','score_201048']]

data_360_5 = data_360_5.rename(columns = {'score':'score_202098'})
data_360_5 = data_360_5[['idcard_md5','mobile_md5','下单日期','score_202098']]

data_360_6 = data_360_6.rename(columns = {'score':'score_202100'})
data_360_6 = data_360_6[['idcard_md5','mobile_md5','下单日期','score_202100']]

# data_360_1.shape
# data_360_5.shape
# data_360_6.shape

data_360_1.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_5.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_6.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)

data_360_score =  data_360_1.merge(data_360_5, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')
data_360_score =  data_360_score.merge(data_360_6, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')
# data_360_score.shape

data_360_score =  data_360_sample.merge(data_360_score, on = ['idcard_md5','mobile_md5','下单日期'], how = 'left')
data_360_score = data_360_score.fillna(-999)
data_360_score = data_360_score.drop(columns= ['idcard_md5', 'mobile_md5','下单日期'])
data_360_score.shape
# data_360_score[:2]

# 2-线下回溯：# 本次回溯：2024.10-2025.1 有贷后表现的订单，2025.2.1-2025.2.25 进件免押（2025.2.25上线）
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/3_20250320/1.数据发送/'
data_360_sample_1 = pd.read_excel(path + 'ceshi_sample_20250320.xlsx')
data_360_sample_1 = data_360_sample_1[['order_number','idcard_md5','mobile_md5','下单日期']]
data_360_sample_1.shape
# data_360_sample_1[:1]

path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/3_20250320/2.数据返回/维客壹佰/'
data_360_1 = pd.read_excel(path + '维客壹佰-201048占控分-支付风险评估-100%.xlsx')
data_360_5 = pd.read_excel(path + '维客壹佰-202098占信分臻藏版-行为风险V3.0-99.99%.xlsx')
data_360_6 = pd.read_excel(path + '维客壹佰-202100占信分臻藏版-支付风险V3.0-100%.xlsx')

data_360_1 = data_360_1.rename(columns = {'score':'score_201048'})
data_360_1 = data_360_1[['idcard_md5','mobile_md5','下单日期','score_201048']]

data_360_5 = data_360_5.rename(columns = {'score':'score_202098'})
data_360_5 = data_360_5[['idcard_md5','mobile_md5','下单日期','score_202098']]

data_360_6 = data_360_6.rename(columns = {'score':'score_202100'})
data_360_6 = data_360_6[['idcard_md5','mobile_md5','下单日期','score_202100']]

data_360_1.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_5.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_6.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)

data_360_score_1 =  data_360_1.merge(data_360_5, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')
data_360_score_1 =  data_360_score_1.merge(data_360_6, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')

data_360_score_1 =  data_360_sample_1.merge(data_360_score_1, on = ['idcard_md5','mobile_md5','下单日期'], how = 'left')
data_360_score_1 = data_360_score_1.fillna(-999)
data_360_score_1 = data_360_score_1.drop(columns= ['idcard_md5', 'mobile_md5','下单日期'])
data_360_score_1.shape
data_360_score_1[:1]

# 线下调用合并
data_360_score = pd.concat([data_360_score, data_360_score_1])
data_360_score.drop_duplicates(subset = ['order_number'], inplace=True)

data_360_score.shape

# 3-线上调用-2025.2.25上线
sql1='''
SELECT trace_id, score as score_201048
from db_credit.risk_zhanrong_201048
;
'''
df_201048 = query(sql1)
df_201048.shape

sql1='''
SELECT trace_id, score as score_202098
from db_credit.risk_zhanrong_202098
;
'''
df_202098 = query(sql1)
df_202098.shape

sql1='''
SELECT trace_id, score as score_202100
from db_credit.risk_zhanrong_202100
;
'''
df_202100 = query(sql1)
df_202100.shape

df_online_360 = df_201048.merge(df_202098, on = 'trace_id', how = 'outer')
df_online_360 = df_online_360.merge(df_202100, on = 'trace_id', how = 'outer')
df_online_360.shape
df_online_360[:1]

# 4-合并
# 部分order_number没有trace_id

df_out_data = df_out_data.merge(data_360_score, on='order_number', how='left')
df_out_data.shape

df_out_data = df_out_data.merge(df_online_360, on='trace_id', how='left')
df_out_data.shape

df_out_data['score_201048'] = np.where(pd.notnull(df_out_data.score_201048_y), df_out_data.score_201048_y, df_out_data.score_201048_x)
df_out_data['score_202098'] = np.where(pd.notnull(df_out_data.score_202098_y), df_out_data.score_202098_y, df_out_data.score_202098_x)
df_out_data['score_202100'] = np.where(pd.notnull(df_out_data.score_202100_y), df_out_data.score_202100_y, df_out_data.score_202100_x)

df_out_data = df_out_data.drop(columns = ['score_201048_x','score_201048_y','score_202098_x','score_202098_y','score_202100_x','score_202100_y'])
df_out_data.shape
df_out_data[:1]

del df_online_360
del df_202100
del df_202098
del df_201048
del data_360_score_1
del data_360_score
del data_360_sample_1
del data_360_sample
del data_360_1
del data_360_5
del data_360_6

(19669, 4)

(19669, 4)

(22444, 4)

(22444, 4)

,order_number,score_201048,score_202098,score_202100
0,A2024100100112314,595,687.0,651


...

,trace_id,score_201048,score_202098,score_202100
0,00001bdaba337a4f0174b5eeee2dcfad,516,436,486


(2269442, 100)

(2269442, 103)

(2269442, 100)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-3-14）极信-银联

In [110]:
# （5-3-14）极信-银联
# 30 个变量
sql1='''-- 风控-极信-银联
SELECT * 
from db_credit.risk_jixin
;
'''

jx_yl_df =query(sql1)
jx_yl_df.shape

jx_yl_df['jx_yl_UPPC011'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC011'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None) 
jx_yl_df['jx_yl_UPPC036'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC036'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC098'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC098'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC134'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC134'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC173'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC173'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC196'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC196'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD002'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD002'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD004'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD004'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD005'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD005'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD007'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD007'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD009'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD009'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD028'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD028'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD029'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD029'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD068'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD068'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD069'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD069'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD075'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD075'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD091'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD091'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD097'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD097'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD098'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD098'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD100'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD100'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE061'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE061'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE074'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE074'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE077'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE077'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE079'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE079'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE082'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE082'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE086'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE086'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE090'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE090'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPG046'] = jx_yl_df['rs_decode'].apply(lambda 
                                                        x: json.loads(x)['data']['resultData'][0]['UPPG046'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPG050'] = jx_yl_df['rs_decode'].apply(lambda 
                                                        x: json.loads(x)['data']['resultData'][0]['UPPG050'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPG054'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPG054'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  

jx_yl_df.drop(columns=['id',  'params', 'rs', 'rs_decode', 'time'],inplace = True)
cols = ['jx_yl_UPPC011', 'jx_yl_UPPC036', 'jx_yl_UPPC098',
       'jx_yl_UPPC134', 'jx_yl_UPPC173', 'jx_yl_UPPC196', 'jx_yl_UPPD002',
       'jx_yl_UPPD004', 'jx_yl_UPPD005', 'jx_yl_UPPD007', 'jx_yl_UPPD009',
       'jx_yl_UPPD028', 'jx_yl_UPPD029', 'jx_yl_UPPD068', 'jx_yl_UPPD069',
       'jx_yl_UPPD075', 'jx_yl_UPPD091', 'jx_yl_UPPD097', 'jx_yl_UPPD098',
       'jx_yl_UPPD100', 'jx_yl_UPPE061', 'jx_yl_UPPE074', 'jx_yl_UPPE077',
       'jx_yl_UPPE079', 'jx_yl_UPPE082', 'jx_yl_UPPE086', 'jx_yl_UPPE090',
       'jx_yl_UPPG046', 'jx_yl_UPPG050', 'jx_yl_UPPG054']
for col in cols:
    jx_yl_df[col] = pd.to_numeric(jx_yl_df[col].replace('"null"', np.nan), errors='coerce')


df_out_data = df_out_data.merge(jx_yl_df, on = 'trace_id', how = 'left')
df_out_data.shape
# df_out_data[:1]

# del jx_yl_df

(248167, 6)

(2269442, 130)

## （5-3-15）京享租
<!-- 已在大表取数 -->
<!-- jd_user_level_risk -->

## （5-4-1）芝麻数控-交互安全风控

In [111]:
# （5-4-1）芝麻数控-交互安全风控
# 目前使用到 risk_t0, risk_t1、risk_t3、risk_t7 没有使用
# zm_jh_t0_level 蚂蚁数控风险等级 
sql1='''-- 芝麻数控-交互安全风控
select
order_id, 
replace(case when JSON_VALID(risk_t0) THEN JSON_EXTRACT(risk_t0, '$.level') end,'"','') as zm_jh_t0_level 
from db_credit.risk_alipay_interactive_prod
;
'''

zm_jh_df=query(sql1)
zm_jh_df.shape

# 508 条无租后表现
# df_out_data = df_out_data.merge(zm_jh_df, on = 'order_id', how = 'outer')
df_out_data = df_out_data.merge(zm_jh_df, on = 'order_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del zm_jh_df

(104086, 2)

(2269442, 131)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-4-2）极信-分数sfico 

In [112]:
# （5-4-2）极信-分数sfico 
sql1='''-- 风控-极信-分数sc32007   fico  
SELECT * 
from db_credit.risk_jixin_score_sc32007
;
'''

jx_fico_df =query(sql1)
jx_fico_df.shape

jx_fico_df['jx_fico_score'] = jx_fico_df['rs_decode'].apply(lambda x: json.loads(x).get('data', {}).get('resultData', {}).get('score', None))  
jx_fico_df['jx_fico_score'] = pd.to_numeric(jx_fico_df['jx_fico_score'].replace('"null"', np.nan), errors='coerce')
jx_fico_df = jx_fico_df[['trace_id', 'jx_fico_score']]


df_out_data = df_out_data.merge(jx_fico_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del jx_fico_df

(112964, 6)

(2269442, 132)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-4-3）极信-TD212

In [113]:
# （5-4-3）极信-TD212
sql1='''
SELECT *
# trace_id, 
# replace(case when JSON_VALID(rs_decode) THEN JSON_EXTRACT(rs_decode, '$.data.resultData.score') end,'"','') as td212_score 
from db_credit.risk_jixin_score_jx1000212
;
'''

jx_td_df = query(sql1)
jx_td_df['td212_score'] = jx_td_df['rs_decode'].apply(lambda x: json.loads(x).get('data', {}).get('resultData', {}).get('score', None)) 
jx_td_df = jx_td_df[['trace_id', 'td212_score']] 
jx_td_df['td212_score'] = jx_td_df['td212_score'].astype(float)
jx_td_df.shape
# jx_td_df[:5]

df_out_data = df_out_data.merge(jx_td_df,on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del jx_td_df

(76004, 2)

(2269442, 133)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-4-4）蚂蚁链-履约保护

In [114]:
# （5-4-4）蚂蚁链-履约保护
sql1='''
-- 122905	122905	122905
SELECT trace_id, nsf_rental_v1_3_hundred	
from db_credit.risk_antchain_risk_general
;
'''

my_zl_general_df=query(sql1)
my_zl_general_df.shape
# my_zl_general_df[:1]
# my_zl_general_df.time.max()
# Timestamp('2025-01-15 18:28:21')
# my_zl_general_df.time.min()
# Timestamp('2024-08-19 20:55:36')

df_out_data = df_out_data.merge(my_zl_general_df,on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del my_zl_general_df  

(70676, 2)

(2269442, 134)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 6.宽表

In [115]:
# （6-1）进件+自变量
# df_out_data：申请/调用
# df_out_data 中 order_id、order_number 没有重复也没有空
# df_out_data.shape
# df_out_data.drop_duplicates(subset = 'order_id').shape
# df_out_data.drop_duplicates(subset = 'order_number').shape
# df_out_data[df_out_data.order_id.isnull()].shape
# df_out_data[df_out_data.order_number.isnull()].shape

# 285992
# df_jj：进件
# df_jj.shape
# df_jj.drop_duplicates(subset = 'order_id').shape
# df_jj.drop_duplicates(subset = 'order_number').shape


df_info_ori = df_jj.merge(df_out_data, on = 'order_number', how = 'left')
# Timestamp('2024-10-16 00:00:00')  ，690 条出库 在 df_out_data 中无记录，中间表往前回溯的时候少量数据侧漏，如 A202407222000361 1277221 ee3e7156005b1e040f7ca4a85455a796
# df_info_ori[(df_analy.order_id.isnull()) & (df_info_ori.下单日期 <= '2025-03-10') & (df_info_ori.审核状态 == '出库')].下单日期.max()
df_info_ori = df_info_ori.drop(columns = ['risk_trace_id'])
df_info_ori.shape
df_info_ori.tail(1) 
# (291100, 142)

(373843, 148)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred
373842,A20250602094830188,2025-06-02,2025-06,芝麻租物,0,出库,[,0,0,0,0,1,[命中自有模型强拒],13660885012,51138119970509565X,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# （6-2）关联租后表现
# df_info_ori ：进件订单的各方面数据
# FPD7+、f3PD7+、Mob9PD30+、
# 历史最大逾期天数：为租期历史最大逾期
# var_1 = ['order_number', '历史最大逾期天数', 'FPD', 'SPD', 'TPD', '第二期逾期天数', '第三期逾期天数','第四期逾期天数', '首期应还日期', 'overdue_type', 'f2pd7', 'f3pd7', 'y_fraud_fpd7','y_fraud_f2pd7', 'y_fraud_f3pd7', 'y_credit_mob3pd30', 'y_credit']
# var_1 = ['order_number', 'y_fraud_fpd7', 'y_fraud_mob3dpd7', 'y_credit_mob3dpd30', 'y_credit']
var_1 = ['order_number', 'y_fraud_fpd7', 'y_fraud_mob3dpd7', 'y_credit_mob3dpd30', 'y_credit_mob6dpd30', 'y_credit']
df_info_ori = df_info_ori.merge(df_analy_data[var_1], on = 'order_number', how = 'left')
# (33986, 31)
# df_info_ori = df_info_ori.merge(df_analy_data, on = 'order_number', how = 'inner')
df_info_ori.shape
df_info_ori[:1]
# (291100, 147)

(373856, 153)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred,y_fraud_fpd7,y_fraud_mob3dpd7,y_credit_mob3dpd30,y_credit_mob6dpd30,y_credit
0,A202206131732031,2022-06-13,2022-06,未知渠道,0,人审拒绝,None,0,0,0,0,NaN,NaN,18301579950,342923199007100712,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# （6-3）规则命中处理--统计命中率的时候再加工
# 参考回捞分析

In [118]:
# （6-4）数据检查
# 1-
# df_info_ori['是否出库'] = np.where(df_info_ori.y_credit.isin([0,1]), 1, 0)
# df_info_ori.groupby(['归属渠道']).agg({'order_number': 'count'})
# df_info_ori[(df_info_ori.归属渠道 == '98租超')&(df_info_ori.search_time == '2025-03')].groupby(['下单日期']).agg({'order_number':'count', '是否出库':'sum'})
# df_info_ori[(df_info_ori.归属渠道 == '八派信息')&(df_info_ori.search_time == '2025-03')].groupby(['下单日期']).agg({'order_number':'count', '是否出库':'sum'})
# df_info_ori[(df_info_ori.归属渠道 == '九州信息')&(df_info_ori.search_time == '2025-03')].groupby(['下单日期']).agg({'order_number':'count', '是否出库':'sum'})

# 2-细化综合风险等级 免押
# tmp = df_info_ori[df_info_ori.下单日期 >= '2025-01-01'][['order_number','下单日期','拒绝理由','rentOnlineDetailLevelOverall']]

# pd.pivot_table(df_info_ori[df_info_ori.下单日期 >= '2025-03-15'],index='下单日期',columns='rentOnlineDetailLevelOverall',values='order_number',aggfunc='count')
# df_info_ori[(df_info_ori.下单日期 == '2025-03-30')&(df_info_ori.rentOnlineDetailLevelOverall.isin(['5','6']))][['order_number','下单日期','拒绝理由','rentOnlineDetailLevelOverall']]

# 3-火眸分和融安分的相关性
# df_info_ori[df_info_ori.下单日期 >= '2024-01-01'][['br_ra_score','bj_hm_score']].corr()
# df_analy_ks[df_analy_ks.下单日期 >= '2024-01-01'][['br_ra_score','bj_hm_score']].corr()

# （6-5）融360 2025.5.21
# score_201048_Bin score_202098_Bin score_202100_Bin
# df_info_ori[df_info_ori.search_time == '2025-02'].score_201048_Bin.value_counts()
# df_info_ori[df_info_ori.search_time == '2025-02'].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[df_info_ori.search_time == '2025-03'].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[df_info_ori.search_time == '2025-04'].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[df_info_ori.search_time == '2025-05'].groupby(['score_202100_Bin']).agg({'order_number':'count'})

# df_info_ori[(df_info_ori.search_time == '2024-12') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-01') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-02') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-03') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-04') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-05') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})

# （6-6）td212
conditions_1 = (df_info_ori.search_time == '2025-05')
conditions_2 = (df_info_ori.归属渠道 == '芝麻租物')
conditions_3 = (df_info_ori.td212_score > 98)
conditions_4 = (df_info_ori.下单日期 <= '2025-05-10')

df_info_ori[conditions_1 & conditions_2 & conditions_3 & conditions_4].shape

(76, 153)

# 7.分箱-业务角度

## 7_1.前置

In [119]:
# （7-1）前置
# 数据驱动+业务实际应用 
# 冰鉴火眸分<500且搜索渠道、二手非抖音豁免
df_info_ori['火眸分_Bin'] = df_info_ori.apply(lambda x: '1.(0,449]' if x.bj_hm_score >= 0 and x.bj_hm_score<= 449
                                           else '2.(449,499]' if x.bj_hm_score > 449 and x.bj_hm_score <= 499
                                           else '3.(499,530]' if x.bj_hm_score > 499 and x.bj_hm_score <= 530
                                           else '4.(530,579]' if x.bj_hm_score > 530 and x.bj_hm_score <= 579
                                           else '5.(579,657]' if x.bj_hm_score > 579 and x.bj_hm_score <= 657
                                           else '6.(657,inf)' if x.bj_hm_score > 657
                                           else '7.-999'
                                           , axis = 1)

## 7_2_1.机审

In [120]:
# （7-2）机审
# 细化综合风险等级 rentOnlineDetailLevelOverall
# 综合风险等级 rentOnlineRiskLevelOverall
# 共租行为等级 rentOnlineRiskLevelMultiRent
# 履约历史等级 rentOnlineRiskLevelFulfill
df_info_ori['rentOnlineRiskLevelFulfill_Bin'] = df_info_ori.apply(lambda x:'1.1' if x.rentOnlineRiskLevelFulfill == '1'
                                                                else '2.0' if x.rentOnlineRiskLevelFulfill == '0'
                                                                else '3.-999', axis = 1)
df_info_ori['rentOnlineRiskLevelOverall_Bin'] = df_info_ori.apply(lambda x: '1.3' if x.rentOnlineRiskLevelOverall == '3'
                                                                   else '2.2' if x.rentOnlineRiskLevelOverall == '2'
                                                                   else '3.1' if x.rentOnlineRiskLevelOverall == '1'
                                                                   else '4.0' if x.rentOnlineRiskLevelOverall == '0'
                                                                   else '5.-999', axis = 1)
df_info_ori['rentOnlineDetailLevelOverall_Bin'] = df_info_ori.apply(lambda x: '1.6' if x.rentOnlineDetailLevelOverall == '6'
                                                                   else '2.5' if x.rentOnlineDetailLevelOverall == '5'
                                                                   else '3.4' if x.rentOnlineDetailLevelOverall == '4'
                                                                   else '4.3' if x.rentOnlineDetailLevelOverall == '3'
                                                                   else '5.2' if x.rentOnlineDetailLevelOverall == '2'
                                                                   else '6.1' if x.rentOnlineDetailLevelOverall == '1'
                                                                   else '7.0' if x.rentOnlineDetailLevelOverall == '0'
                                                                   else '8.-999', axis = 1)

# 规则17		蚁盾分>=80
# 规则18		蚁盾分>50且为搜索渠道
# 规则25		命中融安分低于680
# 规则26		命中融安分低于700且为搜索渠道

df_info_ori['my_zl_score'] = df_info_ori['my_zl_score'].astype(float)
df_info_ori['br_ra_score'] = df_info_ori['br_ra_score'].astype(float)
df_info_ori['蚁盾分_Bin'] = df_info_ori.apply(lambda x: '7.[0,25]' if x.my_zl_score >=0 and x.my_zl_score <= 25
                                           else '6.(25,50]' if x.my_zl_score > 25 and x.my_zl_score  <= 50
                                           else '5.(50,60]' if x.my_zl_score > 50 and x.my_zl_score <= 60
                                           else '4.(60,70]' if x.my_zl_score > 60 and x.my_zl_score  <= 70
                                           else '3.(70,80)' if x.my_zl_score > 70 and x.my_zl_score  < 80
                                            else '2.[80,90)' if x.my_zl_score >= 80 and x.my_zl_score  < 90
                                            else '1.[90,inf)' if x.my_zl_score >= 90
                                            else '8.-999' ,axis = 1)
df_info_ori['融安分_Bin'] = df_info_ori.apply(lambda x: '1.[0,600]' if x.br_ra_score >= 0 and x.br_ra_score <= 600
                                           else '2.(600,650]' if x.br_ra_score > 600 and x.br_ra_score <= 650
                                           else '3.(650,680]' if x.br_ra_score > 650 and x.br_ra_score <= 680
                                           else '4.(680,700]' if x.br_ra_score > 680 and x.br_ra_score <= 700
                                           else '5.(700,750]' if x.br_ra_score > 700 and x.br_ra_score <= 750
                                           else '6.(750,800]' if x.br_ra_score > 750 and x.br_ra_score <= 800
                                           else '7.(800,inf)' if x.br_ra_score > 800
                                           else '8.-999', axis = 1)
# 蚁盾分2.0
df_info_ori['my_zl_score2_Bin'] = df_info_ori.apply(lambda x: '7.[0,25]' if x.my_zl_score2 >=0 and x.my_zl_score2 <= 25
                                                else '6.(25,45]' if x.my_zl_score2 > 25 and x.my_zl_score2  <= 45
                                                else '5.(45,50]' if x.my_zl_score2 > 45 and x.my_zl_score2  <= 50
                                                else '4.(50,65)' if x.my_zl_score2 > 50 and x.my_zl_score2  < 65
                                                else '3.[65,70)' if x.my_zl_score2 >= 65 and x.my_zl_score2  < 70
                                                else '2.[70,75)' if x.my_zl_score2 >= 70 and x.my_zl_score2  < 75
                                                else '1.[75,inf)' if x.my_zl_score2 >= 75
                                                else '8.-999' 
                                                ,axis = 1)

# 云商分
df_info_ori['br_ys_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,450.0]'  if x.br_ys_score >= 0 and x.br_ys_score <= 450 
                                                  else '2.(450,475.0]' if x.br_ys_score  > 450 and x.br_ys_score  <= 475
                                                  else '3.(475,495.0]' if x.br_ys_score  > 475 and x.br_ys_score  <= 495
                                                  else '4.(495,514.0]' if x.br_ys_score  > 495 and x.br_ys_score  <= 514
                                                  else '5.(514,542.0]' if x.br_ys_score  > 514 and x.br_ys_score  <= 542
                                                  else '6.(542,555.0]' if x.br_ys_score  > 542 and x.br_ys_score  <= 555
                                                  else '7.(555,615.0]' if x.br_ys_score  > 555 and x.br_ys_score  <= 615
                                                  else '8.(615,660.0]' if x.br_ys_score  > 615 and x.br_ys_score  <= 660
                                                  else '9.(660,705.0]' if x.br_ys_score  > 660 and x.br_ys_score  <= 705
                                                  else '91.(705,742.0]' if x.br_ys_score  > 705 and x.br_ys_score  <= 742
                                                  else '92.(742,inf)' if x.br_ys_score  > 742
                                                  else '93.-999', axis = 1)
# 青云分
df_info_ori['青云分_Bin'] = df_info_ori.apply(lambda x: '1.[0,449]' if x.bj_qy_score >= 0 and x.bj_qy_score <= 449
                                            else '2.(449,499]' if x.bj_qy_score > 449 and x.bj_qy_score <= 499
                                            else '3.(499,533.0]' if x.bj_qy_score > 499 and x.bj_qy_score <= 533
                                            else '4.(533.0,575.0]' if x.bj_qy_score > 533 and x.bj_qy_score <= 575
                                            else '5.(575.0,607.0]' if x.bj_qy_score > 575 and x.bj_qy_score <= 607
                                            else '6.(607.0,652.0]' if x.bj_qy_score > 607 and x.bj_qy_score <= 652
                                            else '7.(652, inf)' if x.bj_qy_score > 652
                                            else '8.-999', axis = 1)
# 按照之前分析的结果分组
# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
conditions_1 = (df_info_ori.is_model == '1')
conditions_2 = (df_info_ori.is_model == '2')
conditions_3 = (df_info_ori.is_model == '3')
conditions_4 = (df_info_ori.is_model == '4')

# df_info_ori['model_score_2311_Bin'] = df_info_ori.apply(lambda x: '1.[0,12.9620]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 >= 0  and x.model_score_2311 <= 12.9620
#                                                 else '2.(12.9620,15.2211]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 12.9620  and x.model_score_2311 <= 15.2211
#                                                 else '3.(15.2211,18.321]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 15.2211  and x.model_score_2311 <= 18.321
#                                                 else '4.(18.321, 23.297]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
#                                                 else '5.(23.297, inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 23.297 

#                                                 else '6.[0,18.321]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
#                                                 else '7.(18.321, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2311 > 18.321

#                                                 else '8.[0,16.6]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
#                                                 else '9.(16.6, 21.435]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2311 > 16.6 and x.model_score_2311 <= 21.435
#                                                 else '91.(21.435,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2311 > 21.435
#                                                 else 'other'
#                                                 , axis = 1)


# df_info_ori['model_score_yl_Bin'] = df_info_ori.apply(lambda x: '1.[0,13.5137]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl >= 0  and x.model_score_yl <= 13.5137
#                                                 else '2.(13.5137,14.9787]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 13.5137  and x.model_score_yl <= 14.9787
#                                                 else '3.(14.9787,17.55]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 14.9787  and x.model_score_yl <= 17.55
#                                                 else '4.(17.55,23.382]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
#                                                 else '5.(23.382, inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 23.382

#                                                 else '6.[0,17.55]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
#                                                 else '7.(17.55, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_yl > 17.55

#                                                 else '8.[0,10]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl >= 0  and x.model_score_yl <= 10
#                                                 else '9.(10,15.059]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl > 10  and x.model_score_yl <= 15.059
#                                                 else '91.(15.059, 21.661]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl > 15.059 and x.model_score_yl <= 21.661
#                                                 else '92.(21.661,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl > 21.661
#                                                 else 'other'
#                                                 , axis = 1)

# df_info_ori['model_score_2401_Bin'] = df_info_ori.apply(lambda x: '1.[0,6.1907]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 >= 0  and x.model_score_2401 <= 6.1907
#                                                 else '2.(6.1907,8.1386]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 6.1907  and x.model_score_2401 <= 8.1386
#                                                 else '3.(8.1386,11.04]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 8.1386  and x.model_score_2401 <= 11.04
#                                                 else '4.(11.04, 16]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16 
#                                                 else '5.(16,inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 16

#                                                 else '6.[0,11.04]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2401 >= 0  and x.model_score_2401 <= 11.04
#                                                 else '7.(11.04,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2401 > 11.04 

#                                                 else '8.[0,9.35]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
#                                                 else '9.(9.35, 14.93]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2401 > 9.35 and x.model_score_2401 <= 14.93
#                                                 else '91.(14.93,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2401 > 14.93
#                                                 else 'other'
#                                                 , axis = 1)

# df_info_ori['model_score_2501_Bin'] = df_info_ori.apply(lambda x: '1.[0,10.047]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 >= 0 and x.model_score_2501 <= 10.047
#                                                 else '2.(10.047,11.635]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 10.047 and x.model_score_2501 <= 11.635
#                                                 else '3.(11.635,14.104]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 11.635 and x.model_score_2501 <= 14.104
#                                                 else '4.(14.104, 18.132]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 14.104 and x.model_score_2501 <= 18.132
#                                                 else '5.(18.132,inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 18.132

#                                                 else '6.[0,10.047]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
#                                                 else '7.(10.047, 13.79]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
#                                                 else '8.(13.79,18.098]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
#                                                 else '9.(18.098,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 > 18.098

#                                                 else '91.[0,10.047]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
#                                                 else '92.(10.047, 13.79]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
#                                                 else '93.(13.79,18.098]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
#                                                 else '94.(18.098,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 > 18.098
#                                                 else 'other'
#                                                 , axis = 1)




df_info_ori['model_score_2311_Bin'] = df_info_ori.apply(lambda x: '1.[0,12.9620]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 12.9620
                                                else '2.(12.9620,15.2211]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 12.9620  and x.model_score_2311 <= 15.2211
                                                else '3.(15.2211,18.321]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 15.2211  and x.model_score_2311 <= 18.321
                                                else '4.(18.321, 23.297]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
                                                else '5.(23.297, inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 23.297 

                                                else '6.[0,18.321]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
                                                else '7.(18.321, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 18.321

                                                else '8.[0,16.6]' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
                                                else '9.(16.6, 21.435]' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 16.6 and x.model_score_2311 <= 21.435
                                                else '91.(21.435,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 21.435
                                                else 'other'
                                                , axis = 1)


df_info_ori['model_score_yl_Bin'] = df_info_ori.apply(lambda x: '1.[0,13.5137]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl >= 0  and x.model_score_yl <= 13.5137
                                                else '2.(13.5137,14.9787]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 13.5137  and x.model_score_yl <= 14.9787
                                                else '3.(14.9787,17.55]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 14.9787  and x.model_score_yl <= 17.55
                                                else '4.(17.55,23.382]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
                                                else '5.(23.382, inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 23.382

                                                else '6.[0,17.55]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
                                                else '7.(17.55, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 17.55

                                                else '8.[0,10]' if x.归属渠道 == '搜索渠道'  and x.model_score_yl >= 0  and x.model_score_yl <= 10
                                                else '9.(10,15.059]' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 10  and x.model_score_yl <= 15.059
                                                else '91.(15.059, 21.661]' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 15.059 and x.model_score_yl <= 21.661
                                                else '92.(21.661,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 21.661
                                                else 'other'
                                                , axis = 1)

df_info_ori['model_score_2401_Bin'] = df_info_ori.apply(lambda x: '1.[0,6.1907]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 6.1907
                                                else '2.(6.1907,8.1386]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 6.1907  and x.model_score_2401 <= 8.1386
                                                else '3.(8.1386,11.04]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 8.1386  and x.model_score_2401 <= 11.04
                                                else '4.(11.04, 16]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16 
                                                else '5.(16,inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 16

                                                else '6.[0,11.04]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2401 >= 0  and x.model_score_2401 <= 11.04
                                                else '7.(11.04,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2401 > 11.04 

                                                else '8.[0,9.35]' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
                                                else '9.(9.35, 14.93]' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 9.35 and x.model_score_2401 <= 14.93
                                                else '91.(14.93,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 14.93
                                                else 'other'
                                                , axis = 1)

df_info_ori['model_score_2501_Bin'] = df_info_ori.apply(lambda x: '1.[0,10.047]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 >= 0 and x.model_score_2501 <= 10.047
                                                else '2.(10.047,11.635]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 10.047 and x.model_score_2501 <= 11.635
                                                else '3.(11.635,14.104]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 11.635 and x.model_score_2501 <= 14.104
                                                else '4.(14.104, 18.132]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 14.104 and x.model_score_2501 <= 18.132
                                                else '5.(18.132,inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 18.132

                                                else '6.[0,10.047]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
                                                else '7.(10.047, 13.79]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
                                                else '8.(13.79,18.098]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
                                                else '9.(18.098,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 > 18.098

                                                else '91.[0,10.047]' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
                                                else '92.(10.047, 13.79]' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
                                                else '93.(13.79,18.098]' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
                                                else '94.(18.098,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 > 18.098
                                                else 'other'
                                                , axis = 1)

# 评分等级
df_info_ori['grade_Bin'] = df_info_ori.apply(lambda x: '1.F' if x.grade == 'F'
                                             else '2.E' if x.grade == 'E'
                                             else '3.D' if x.grade == 'D'
                                             else '4.C' if x.grade == 'C'
                                             else '5.B' if x.grade == 'B'
                                             else '6.A' if x.grade == 'A'
                                             else '7.S' if x.grade == 'S'
                                             else '8.-999'
                                             , axis = 1)
# 策略240703_4
df_info_ori['strategy_240703_4_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 > 1.5 and x.jx_yl_UPPC134 <= 3.5 and x.jx_yl_UPPD007 > 47
                                                         else '2.客群2' if x.model_score_2401 > 14.27667
                                                         else '3.客群3' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 <= 1.5
                                                         else '4.客群4' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 > 1.5 and x.jx_yl_UPPC134 <= 3.5 and x.jx_yl_UPPD007 <= 47
                                                         else '5.客群5' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 > 3.5
                                                         else '6.-999', axis = 1)
# 策略240703_10
df_info_ori['strategy_240703_10_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 <= 623.5 and x.xy_qj_B22170036 > 8.5
                                                          else '2.客群2' if x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 <= 623.5 and x.xy_qj_B22170036 <= 8.5
                                                          else '3.客群3' if x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 > 623.5
                                                          else '4.客群4' if x.model_score_2401 >= 0 and x.model_score_2401 <= 14.27667
                                                          else '5.-999', axis = 1)

# 策略240703_10
df_info_ori['strategy_240703_14_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 > 1.345 and x.my_zl_score > 18.324
                                                          else '2.客群2' if x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 > 1.345 and x.my_zl_score <= 18.324
                                                          else '3.客群3' if x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 <= 1.345
                                                          else '4.客群4' if x.model_score_2401 >= 0 and x.model_score_2401 <= 14.27667
                                                          else '5.-999', axis = 1)

# 融360 
df_info_ori['score_201048'] = df_info_ori['score_201048'].astype(float)
df_info_ori['score_202098'] = df_info_ori['score_202098'].astype(float)
df_info_ori['score_202100'] = df_info_ori['score_202100'].astype(float)

df_info_ori['score_201048_Bin'] = df_info_ori.apply(lambda x: '1.[0,409]' if x.score_201048 >= 0 and x.score_201048 <= 409
                                                    else '2.(409,424]' if x.score_201048 > 409 and x.score_201048 <= 424
                                                    else '3.(424,517]' if x.score_201048 > 424 and x.score_201048 <= 517
                                                    else '4.(517,543]' if x.score_201048 > 517 and x.score_201048 <= 543
                                                    else '5.(543,588]' if x.score_201048 > 543 and x.score_201048 <= 588
                                                    else '6.(588,inf)' if x.score_201048 > 588
                                                    else '7.[-999,-999]', axis = 1)
df_info_ori['score_202098_Bin'] = df_info_ori.apply(lambda x: '1.[0,417]' if x.score_202098 >= 0 and x.score_202098 <= 417
                                                    else '2.(417,470]' if x.score_202098 > 417 and x.score_202098 <= 470
                                                    else '3.(470,511]' if x.score_202098 > 470 and x.score_202098 <= 511
                                                    else '4.(511,546]' if x.score_202098 > 511 and x.score_202098 <= 546
                                                    else '5.(546,602)' if x.score_202098 > 546 and x.score_202098 <= 602
                                                    else '6.(602,inf)' if x.score_202098 > 602
                                                    else '7.[-999,-999]', axis = 1)
df_info_ori['score_202100_Bin'] = df_info_ori.apply(lambda x: '1.[0,387]' if x.score_202100 >= 0 and x.score_202100 <= 387
                                                    else '2.(387,405]' if x.score_202100 > 387 and x.score_202100 <= 405
                                                    else '3.(405,440]' if x.score_202100 > 405 and x.score_202100 <= 440
                                                    else '4.(440,477]' if x.score_202100 > 440 and x.score_202100 <= 477
                                                    else '5.(477,524]' if x.score_202100 > 477 and x.score_202100 <= 524
                                                    else '6.(524,inf)' if x.score_202100 > 524
                                                    else '7.[-999,-999]', axis = 1)

## 7_2_2.机审2

In [121]:
# 策略240801
# reject
df_info_ori['strategy_240801_1_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.model_score_2311 > 17.684
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.model_score_2311 <= 17.684
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_2_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.model_score_yl > 21.6
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.model_score_yl <= 21.6
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_3_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.model_score_2401 > 14.326
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.model_score_2401 <= 14.326
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_4_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.my_zl_score > 20.885
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.my_zl_score <= 20.885
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_5_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.xy_qj_C22180008 <= 10.5 and x.xy_qj_B22170035 > 4.5
                                                          else '2.客群2' if x.xy_qj_C22180008 <= 10.5 and x.xy_qj_B22170035 <= 4.5
                                                          else '3.客群3' if x.xy_qj_C22180008 > 10.5
                                                          else '4.-999', axis = 1)

# pass
df_info_ori['strategy_240801_7_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 > 0.655
                                                          else '2.客群2' if x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 <= 0.655
                                                          else '1.客群1' if x.jx_fico_score <= 604.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_8_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 604.5 and x.model_score_2401 <= 8.093
                                                          else '2.客群2' if x.jx_fico_score > 604.5 and x.model_score_2401 > 8.093
                                                          else '1.客群1' if x.jx_fico_score <= 604.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_9_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 561.5 and x.model_score_2401 <= 9.849
                                                          else '2.客群2' if x.jx_fico_score > 561.5 and x.model_score_2401 > 9.849
                                                          else '1.客群1' if x.jx_fico_score <= 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_10_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 561.5 and x.bj_qy_score > 433.5
                                                          else '2.客群2' if x.jx_fico_score > 561.5 and x.bj_qy_score <= 433.5
                                                          else '1.客群1' if x.jx_fico_score <= 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_11_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.model_score_2401 <= 14.326 and x.xy_qj_C22180008 > 10.5
                                                          else '2.客群2' if x.model_score_2401 <= 14.326 and x.xy_qj_C22180008 <= 10.5
                                                          else '1.客群1' if x.model_score_2401 > 14.326
                                                          else '4.-999', axis = 1)

## 7_3.出库前强拒

In [122]:
# （7-3）出库前强拒
# 规则37		蚂蚁数控风险等级=998强拒
# 规则38		蚂蚁数控风险等级=996强拒且非芝麻租物
# 规则39		蚂蚁数控风险等级=979强拒且非芝麻租物非搜索渠道
# 规则40		fico_S50c067信用分<537
# 规则41		非单人话且非扫一扫渠道：命中fico_S50c067信用分 > 489.5 且 fico_S50c067信用分 <= 537 且青云分 > 450 随机40% 豁免
# 规则42		银联模型ascore_yl2311分>17.55且fico_S50c067信用分 >537 且 fico_S50c067信用分 <=559，命中强拒  但青云分 > 450的话 随机40% 豁免该规则
# 规则43	   strategy_240927	"芝麻租物渠道： 1. 腾云212>98拒绝；
                                        # 2. 履约保护>26拒绝；（关系为“或”，命中其中一个拒绝）

                                # 搜索渠道： 1. 腾云212>98拒绝；
                                       # 2. 履约保护>26拒绝；（关系为“或”，命中其中一个拒绝）

                                # 其他渠道的： 1. 腾云212>95拒绝;
                                # 2. 履约保护>25拒绝; （关系为“或”，命中其中一个拒绝）"

# 0 为青云分豁免或回捞的单（有青云分<450的，但集中在2024.4及之前）,1表示强拒，2 表示命中但40%随机豁免，3 表示通过，4 表示通过且 jx_fico_score > 559，5 为jx_fico_score缺失
df_info_ori['规则42'] = df_info_ori.apply(lambda x: 0 if x.jx_fico_score >= 0 and x.jx_fico_score <= 537
                                        else 1 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl > 17.55 and  x.bj_qy_score <= 450
                                        else 2 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl > 17.55 and x.bj_qy_score > 450
                                        else 3 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl <= 17.55
                                        else 4 if x.jx_fico_score > 559
                                        else 5, axis = 1)
# 根据应用建议分组， nsf_rental_v1_3_hundred_Bin 为根据数据表现分组
df_info_ori['履约保护_Bin'] = df_info_ori.apply(lambda x: '91.[0,1]' if x.nsf_rental_v1_3_hundred >= 0 and x.nsf_rental_v1_3_hundred <= 1
                                            else '9.2' if x.nsf_rental_v1_3_hundred == 2
                                            else '8.3' if x.nsf_rental_v1_3_hundred == 3
                                            else '7.[4,7]' if x.nsf_rental_v1_3_hundred >= 4 and x.nsf_rental_v1_3_hundred <= 7
                                            else '6.[8,9]' if x.nsf_rental_v1_3_hundred >= 8 and x.nsf_rental_v1_3_hundred <= 9
                                          #   租物&搜索
                                            # else '5.[10,26]' if x.nsf_rental_v1_3_hundred >= 10 and x.nsf_rental_v1_3_hundred <= 26
                                            # else '4.[27,30]' if x.nsf_rental_v1_3_hundred >= 27 and x.nsf_rental_v1_3_hundred <= 30
                                          #   其他
                                            else '5.[10,25]' if x.nsf_rental_v1_3_hundred >= 10 and x.nsf_rental_v1_3_hundred <= 25
                                            else '4.[26,30]' if x.nsf_rental_v1_3_hundred >= 26 and x.nsf_rental_v1_3_hundred <= 30

                                            else '3.[31,95]' if x.nsf_rental_v1_3_hundred >= 31 and x.nsf_rental_v1_3_hundred <= 95
                                            else '2.[96,98]' if x.nsf_rental_v1_3_hundred >= 96 and x.nsf_rental_v1_3_hundred <= 98
                                            else '1.(98,inf)' if x.nsf_rental_v1_3_hundred > 98
                                            else '92.-999', axis = 1)

df_info_ori['td212_score'] = df_info_ori['td212_score'].astype(float)
df_info_ori['td212_Bin'] = df_info_ori.apply(lambda x: '1.(98,inf)' if x.td212_score > 98
                                             else '2.(95,98]' if x.td212_score > 95 and x.td212_score <= 98
                                             else '3.(90,95]' if x.td212_score > 90 and x.td212_score <= 95
                                             else '4.(50,90]' if x.td212_score > 50 and x.td212_score <= 90
                                             else '5.[0,50]' if x.td212_score >= 0 and x.td212_score <= 50
                                             else '6.-999', axis = 1)

df_info_ori['zm_jh_t0_level_Bin'] = df_info_ori.apply(lambda x: '1.998' if x.zm_jh_t0_level == '998'
                                                      else '2.996' if x.zm_jh_t0_level == '996'
                                                      else '3.979' if x.zm_jh_t0_level == '979'
                                                      else '4.975' if x.zm_jh_t0_level == '975'
                                                      else '5.969' if x.zm_jh_t0_level == '969'
                                                      else '6.989' if x.zm_jh_t0_level == '989'
                                                      else '7.-999'
                                                      , axis = 1)

df_info_ori['FICO_Bin'] = df_info_ori.apply(lambda x: '1.[0,489]' if x.jx_fico_score >= 0  and x.jx_fico_score <= 489
                                            else '2.(489,536]' if x.jx_fico_score > 489 and x.jx_fico_score <= 536
                                            else '3.(536,563]' if x.jx_fico_score > 536 and x.jx_fico_score <= 563
                                            else '4.(563,587]' if x.jx_fico_score > 563 and x.jx_fico_score <= 587
                                            else '5.(587,617]' if x.jx_fico_score > 587 and x.jx_fico_score <= 617
                                            else '6.(617,632]' if x.jx_fico_score > 617 and x.jx_fico_score <= 632
                                            else '7.(632,672]' if x.jx_fico_score > 632 and x.jx_fico_score <= 672
                                            else '8.(672,inf)' if x.jx_fico_score > 672
                                            else '9.-999', axis=1)

# 8.分箱-数据驱动

In [123]:
# （8-1-1）宽表-区分度
df_analy_ks = df_info_ori[df_info_ori.y_credit.isin([0,1])]
df_analy_ks.shape

# 字符型
# xy_qj_A22160007 ： 最近⼀次查询时间
# xy_qj_B22170054 ： 最近⼀次贷款放款时间
# xy_qj_B22170007	近1个⽉贷款总⾦额
# xy_qj_B22170008	近3个⽉贷款总⾦额
# xy_qj_B22170009	近6个⽉贷款总⾦额
# xy_qj_B22170010	近12个⽉贷款总⾦额
# xy_qj_B22170011	近24个⽉贷款总⾦额
# xy_qj_B22170031	近6个⽉累计逾期⾦额参考⾦额区间
# xy_qj_B22170032	近12个⽉累计逾期⾦额参考⾦额区间
# xy_qj_B22170033	近24个⽉累计逾期⾦额参考⾦额区间
# xy_qj_B22170040	近1个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170041	近3个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170042	近6个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170043	近12个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170044	近24个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170050	最近⼀次履约距今天数参考天数区间


# var_info = ['order_number','下单日期','search_time','归属渠道','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
var_info = ['order_number','下单日期','search_time','归属渠道','y_fraud_fpd7', 'y_fraud_mob3dpd7', 'y_credit_mob3dpd30', 'y_credit_mob6dpd30', 'y_credit']
var_1 = ['bj_hm_score','my_zl_score','my_zl_score2','br_ys_score','br_ra_score','model_score_2311','model_score_2401','model_score_yl','model_score_2501'
         ,'bj_qy_score','jx_fico_score','td212_score','nsf_rental_v1_3_hundred']
var_2 = list(jx_yl_df.columns)
var_2.remove('trace_id')
var_3 = list(xy_risk_qjld_df.columns)
var_3.remove('trace_id')


# df_analy_ks = df_analy_ks[var_info + var_1 + var_2 + var_3]
var_3.remove('xy_qj_A22160007')
var_3.remove('xy_qj_B22170054')
var_3.remove('xy_qj_B22170007')
var_3.remove('xy_qj_B22170008')
var_3.remove('xy_qj_B22170009')
var_3.remove('xy_qj_B22170010')
var_3.remove('xy_qj_B22170011')
var_3.remove('xy_qj_B22170031')
var_3.remove('xy_qj_B22170032')
var_3.remove('xy_qj_B22170033')
var_3.remove('xy_qj_B22170040')
var_3.remove('xy_qj_B22170041')
var_3.remove('xy_qj_B22170042')
var_3.remove('xy_qj_B22170043')
var_3.remove('xy_qj_B22170044')
var_3.remove('xy_qj_B22170050')

var_all = var_1 + var_2 + var_3
len(var_all)

for var in var_all:
    # print (var)
    df_analy_ks[var] = df_analy_ks[var].astype(float)
    df_analy_ks[var] = np.where(df_analy_ks[var] < 0, -999, df_analy_ks[var])
    df_analy_ks[var] = df_analy_ks[var].fillna(-999)
    
df_analy_ks.shape
df_analy_ks.tail(1)

(45074, 188)

103

(45074, 188)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred,y_fraud_fpd7,y_fraud_mob3dpd7,y_credit_mob3dpd30,y_credit_mob6dpd30,y_credit,火眸分_Bin,rentOnlineRiskLevelFulfill_Bin,rentOnlineRiskLevelOverall_Bin,rentOnlineDetailLevelOverall_Bin,蚁盾分_Bin,融安分_Bin,my_zl_score2_Bin,br_ys_score_Bin,青云分_Bin,model_score_2311_Bin,model_score_yl_Bin,model_score_2401_Bin,model_score_2501_Bin,grade_Bin,strategy_240703_4_Bin,strategy_240703_10_Bin,strategy_240703_14_Bin,score_201048_Bin,score_202098_Bin,score_202100_Bin,strategy_240801_1_Bin,strategy_240801_2_Bin,strategy_240801_3_Bin,strategy_240801_4_Bin,strategy_240801_5_Bin,strategy_240801_7_Bin,strategy_240801_8_Bin,strategy_240801_9_Bin,strategy_240801_10_Bin,strategy_240801_11_Bin,规则42,履约保护_Bin,td212_Bin,zm_jh_t0_level_Bin,FICO_Bin
372795,A20250601104622319,2025-06-01,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,13102536943,130321199412013317,None,2783786.0,2559f9139c712c63e61b8512198911b2,192870482d012404f214521bf5ba70c4,456.0,0,0,0,4,15.6255,61.0307,483.0,751.0,20.249994,13.485264,18.272508,12.600109,1,519.0,C,642.0,74.0,36.0,12.0,6.0,66.0,2025-05,9.0,17.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,14.0,4800.0,2637.0,2900.0,84.0,547.0,0.0,3.0,8.0,10.0,13.0,0,"[1000,2000)","[5000,10000)","[10000,20000)","[10000,20000)",7.0,1.0,2.0,0.0,0.0,3.0,8.0,9.0,11.0,9.0,11.0,0.0,0.0,3.0,3.0,3.0,1.0,1.0,1.0,"[500,1000)","[500,1000)","[500,1000)",0.61,1.0,6.0,7.0,7.0,7.0,"[500,1000)","[2000,3000)","[3000,5000)","[5000,10000)","[5000,10000)",4.0,11.0,18.0,26.0,30.0,"(7,15]",81.0,16.0,720.0,2025-04,468.0,571.0,526.0,252.0,457.0,252.0,30.0,138.0,29.0,18000.0,42000.0,32.0,74.0,21.0,21800.0,31.0,21.0,28.0,15.0,68.0,42000.0,7.0,21.0,29.0,0.85,0.55,0.3,0.61,1.0,2.0,21600.0,10.0,9.0,975,619.0,36.37,13.0,0.0,0.0,0.0,0.0,0.0,"2.(449,499]",2.0,4.0,3.4,"7.[0,25]","6.(750,800]","4.(50,65)","3.(475,495.0]","3.(499,533.0]","4.(18.321, 23.297]","4.(17.55

In [133]:
# （8-1-2）其他
# （1）S量策略开发的宽表
# var_5 = list(df_out_data.columns)
# var_5.remove('order_number')
# var_5.remove('order_id')
# var_5.remove('order_trace_id')
# var_5.remove('trace_id')
# var_5.remove('risk_trace_id')

# 租后表现
# var_6 = ['order_number','order_id','下单日期','search_time','归属渠道','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
# df_analy_ks[var_6 + var_5].to_excel('data_s_20250417.xlsx', index = False)

# # 申请
# tmp1 = df_risk_new[['order_number','下单日期','归属渠道','id_card_num']].merge(df_out_data[['order_number','bj_hm_score']], on = 'order_number', how = 'left')
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# 租机宝、富士康租机、兔丫租赁、邦客惠乘车码  -- 不算S量
# conditions_2 = ((tmp1.下单日期 >= '2025-04-01')&(tmp1.下单日期 <= '2025-04-15'))
# conditions_3 = ((tmp1.bj_hm_score >= 0)&(tmp1.bj_hm_score <= 499))
# # 申请数
# # (5652, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].shape
# (1036, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & conditions_3].shape
# 申请-去重订单数-身份证+申请日期维度去重
# (5435, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期']).shape
# tmp2 = tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期'])
# tmp2.groupby(['下单日期']).agg({'order_number': 'count'})
# (858, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & conditions_3].drop_duplicates(subset=['id_card_num','下单日期']).shape
# tmp3 = tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & conditions_3].drop_duplicates(subset=['id_card_num','下单日期'])
# tmp3.groupby(['下单日期']).agg({'order_number': 'count'})


# 进件-非免押
# tmp1 = df_jj[['order_number','拒绝理由','归属渠道','下单日期','id_card_num','审核状态']]
# tmp1['tmp'] = np.where(tmp1['拒绝理由'].str.contains('非免押'), 1, 0)

# # 890
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期']).shape
# # 49
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & (tmp1.tmp == 1)].drop_duplicates(subset=['id_card_num','下单日期']).shape
# tmp1[(tmp1.tmp == 1)&(tmp1.下单日期>='2025-04-01')].拒绝理由.value_counts()
# 审核状态为出库的为小蚂蚁出库 A202504161000356
# tmp1[(tmp1.tmp == 1)&(tmp1.下单日期>='2025-04-01')].审核状态.value_counts()
# [命中非免押用户拒绝]
# 命中非免押用户拒绝

# # 进件
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# conditions_2 = ((df_info_ori.下单日期 >= '2025-04-01')&(df_info_ori.下单日期 <= '2025-04-25'))
# df_info_ori[df_info_ori.归属渠道.isin(conditions_1) & conditions_2].shape
# df_info_ori[df_info_ori.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期']).shape
# df_info_ori[df_info_ori.归属渠道.isin(conditions_1) & conditions_2].to_excel('data_s_jj_20250426.xlsx', index = False)


# # （2）取消订单分析
# # 2025.3进件-取消的样本
# path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/11_一元发货/'
# df_qx_1 = pd.read_excel(path + '取消原因订单_20250403.xlsx')
# df_qx_1.shape
# df_qx_1 = df_qx_1.merge(df_info_ori, on = 'order_number', how = 'left')
# df_qx_1.shape
# # df_qx_1.to_excel('tmp.xlsx', index = False)
# # 细化综合风险等级 rentOnlineDetailLevelOverall
# # 蚁盾分2.0 my_zl_score2_Bin
# # 评分等级 grade
# # 自有模型 model_score_2311_Bin model_score_yl_Bin model_score_2401_Bin
# # df_qx_1.groupby(['rentOnlineDetailLevelOverall_Bin']).agg({'order_number':'count'})
# # df_qx_1.groupby(['my_zl_score2_Bin']).agg({'order_number':'count'})
# # df_qx_1.groupby(['grade_Bin']).agg({'order_number':'count'})
# df_qx_1[(df_qx_1.归属渠道 == '芝麻租物')].shape
# df_qx_1[(df_qx_1.归属渠道 == '芝麻租物')&(df_qx_1.rentOnlineDetailLevelOverall == '0')|(df_qx_1.grade == 'S')].shape


# # 历史的一元发货的订单-2023.3-2023.7
# # df_qx_2 = pd.read_excel(path + '1元发货订单.xlsx')
# # df_qx_2 = df_qx_2[['订单号','型号']]
# # df_qx_2 = df_qx_2.rename(columns = {'订单号':'order_number'})
# # df_qx_2.shape
# # df_qx_2['是否二手'] = np.where(df_qx_2['型号'].str.contains('95新')|df_qx_2['型号'].str.contains('99新')|df_qx_2['型号'].str.contains('准新'),"二手","全新")
# # df_qx_2['是否带锁']  = np.where(df_qx_2['型号'].str.contains(r'带锁|有锁'), '带锁', '未带锁')
# # df_qx_2['是否二手'].value_counts()
# # df_qx_2['是否带锁'].value_counts()

# # df_qx_2 = df_qx_2.merge(df_analy_data[['order_number','search_time','归属渠道','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit','历史最大逾期天数']], on = 'order_number', how = 'left')
# # df_qx_2['归属渠道_new'] = np.where((~df_qx_2.归属渠道.isin(['芝麻租物','搜索渠道'])), '其他', df_qx_2.归属渠道)
# # df_qx_2['归属渠道_new'].value_counts()
# # df_qx_2.shape
# # df_qx_2[:2]

# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_credit',values='order_number',aggfunc='count')

# （3）新旧渠道对比
# df_analy_ks.shape
# df_analy_ks[['order_number','归属渠道']].to_excel('tmp.xlsx', index = False)

# （4）抖音
# temp = df_info_ori[(df_info_ori.归属渠道 == '抖音渠道') & (df_info_ori.下单日期 >= '2025-03-01')]
# pd.pivot_table(temp,index='下单日期',columns='grade',values='order_number',aggfunc='count')
# pd.pivot_table(temp[temp.审核状态 == '出库'],index='下单日期',columns='grade',values='order_number',aggfunc='count')

# df_analy_ks[df_analy_ks.归属渠道 == '抖音渠道'][['order_number','search_time','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']].to_excel('抖音_20250506.xlsx', index = False)

# （5）免押用户归属渠道
# df_info_ori[(~df_info_ori.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))&(df_info_ori.下单日期 >= '2023-01-01')][['order_number','归属渠道','下单日期']].to_excel('免押用户归属渠道_20250423.xlsx', index = False)

# 联合运营租后表现
# df_analy_ks[df_analy_ks.是否联合运营 == '1'][['order_number','下单日期','y_fraud_fpd7','y_credit']]
# df_analy_ks[(df_analy_ks.是否联合运营 == '1')&(df_analy_ks.下单日期 >= '2025-03-25')&(df_analy_ks.下单日期 <= '2025-04-02')][['order_number','下单日期','y_fraud_fpd7','y_credit']]

# # 联合运营进件客群-R0、R1
# df_info_ori.是否联合运营.value_counts()
# df_info_ori[df_info_ori.是否联合运营 == '1'].rentOnlineDetailLevelOverall.value_counts()
# df_info_ori[(df_info_ori.是否联合运营 == '1')&(df_info_ori.rentOnlineDetailLevelOverall.isin(['1','2','3','4','5','6']))][['order_number','下单日期','rentOnlineDetailLevelOverall']]


# （6）周报-统计cpd、hpd时用最新的渠道，月更新即可
# df_analy_ks[['order_number','归属渠道','是否拒量']].to_excel('df_cpd_hpd_20250427.xlsx', index = False)

# （7）拒量-联合项目
# 进件-非免押
# conditions_1 = ((df_jj.归属渠道.isin(['芝麻租物','搜索渠道'])) & (df_jj.下单日期 >= '2024-01-01') & (df_jj.下单日期 <= '2025-02-28'))
# tmp1 = df_jj[conditions_1][['order_number','id_card_num','user_mobile','下单日期','拒绝理由','审核状态','是否拒量']]
# tmp1['tmp'] = np.where(tmp1['拒绝理由'].str.contains('非免押'), 1, 0)
# tmp1 = tmp1[tmp1.tmp == 0]
# tmp1 = tmp1.drop(columns = ['tmp','拒绝理由'])
# tmp1.shape
# tmp1[:1]

# tmp1.to_excel('df_jl_no_label.xlsx', index = False)

# # （8）拒量调优宽表
# tmp = df_risk_new[df_risk_new['是否拒量'] == 1][['下单日期','order_number','tips','审核状态']]
# tmp['是否出库'] = np.where(tmp.审核状态 == '出库', 1, 0)
# tmp['策略命中等级'] = tmp['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# tmp = tmp[['order_number','策略命中等级','是否出库']]
# tmp.shape

# df_analy_jl = tmp.merge(df_info_ori, on = 'order_number', how = 'left')
# df_analy_jl.shape

# var_info = ['order_number','下单日期','search_time','归属渠道','策略命中等级','是否出库','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
# # 产品-数值
# var_1_1 = ['bj_hm_score','rentOnlineDetailLevelOverall','rentOnlineRiskLevelOverall','rentOnlineRiskLevelMultiRent','rentOnlineRiskLevelFulfill'
#          ,'my_zl_score','my_zl_score2','br_ys_score','br_ra_score','model_score_2311','model_score_2401','model_score_yl','model_score_2501'
#          ,'bj_qy_score','score_201048','score_202098','score_202100','jx_fico_score','td212_score','nsf_rental_v1_3_hundred']
# # 产品-字符
# var_1_2 = ['grade','zm_jh_t0_level']
# var_2 = list(jx_yl_df.columns)
# var_2.remove('trace_id')
# var_3 = list(xy_risk_qjld_df.columns)
# var_3.remove('trace_id')


# # 字符型-16
# # xy_qj_A22160007 ： 最近⼀次查询时间
# # xy_qj_B22170054 ： 最近⼀次贷款放款时间
# # xy_qj_B22170007	近1个⽉贷款总⾦额
# # xy_qj_B22170008	近3个⽉贷款总⾦额
# # xy_qj_B22170009	近6个⽉贷款总⾦额
# # xy_qj_B22170010	近12个⽉贷款总⾦额
# # xy_qj_B22170011	近24个⽉贷款总⾦额
# # xy_qj_B22170031	近6个⽉累计逾期⾦额参考⾦额区间
# # xy_qj_B22170032	近12个⽉累计逾期⾦额参考⾦额区间
# # xy_qj_B22170033	近24个⽉累计逾期⾦额参考⾦额区间
# # xy_qj_B22170040	近1个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170041	近3个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170042	近6个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170043	近12个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170044	近24个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170050	最近⼀次履约距今天数参考天数区间

# var_3.remove('xy_qj_A22160007')
# var_3.remove('xy_qj_B22170054')
# var_3.remove('xy_qj_B22170007')
# var_3.remove('xy_qj_B22170008')
# var_3.remove('xy_qj_B22170009')
# var_3.remove('xy_qj_B22170010')
# var_3.remove('xy_qj_B22170011')
# var_3.remove('xy_qj_B22170031')
# var_3.remove('xy_qj_B22170032')
# var_3.remove('xy_qj_B22170033')
# var_3.remove('xy_qj_B22170040')
# var_3.remove('xy_qj_B22170041')
# var_3.remove('xy_qj_B22170042')
# var_3.remove('xy_qj_B22170043')
# var_3.remove('xy_qj_B22170044')
# var_3.remove('xy_qj_B22170050')

# # 变量-字符
# var_3_2 = ['xy_qj_A22160007','xy_qj_B22170054','xy_qj_B22170007','xy_qj_B22170008','xy_qj_B22170009','xy_qj_B22170010','xy_qj_B22170011','xy_qj_B22170031','xy_qj_B22170032'
#            ,'xy_qj_B22170033','xy_qj_B22170040','xy_qj_B22170041','xy_qj_B22170042','xy_qj_B22170043','xy_qj_B22170044','xy_qj_B22170050']

# var_all = var_1_1 + var_2 + var_3
# len(var_all)

# for var in var_all:
#     df_analy_jl[var] = np.where(df_analy_jl[var] == 'DEFAULT', -999, df_analy_jl[var])
#     df_analy_jl[var] = df_analy_jl[var].astype(float)
#     df_analy_jl[var] = np.where(df_analy_jl[var] < 0, -999, df_analy_jl[var])
#     df_analy_jl[var] = df_analy_jl[var].fillna(-999)
    
# df_analy_jl.shape
# df_analy_jl = df_analy_jl[var_info + var_all + var_3_2 + var_1_2]
# df_analy_jl.shape
# df_analy_jl.tail(1)

# # df_analy_jl.to_excel('拒量数据_20250507.xlsx', index = False)

# # （9）京享租宽表
# # bj_hm_score 冰鉴-火眸分
# # my_zl_score2 蚁盾分2.0
# # bj_qy_score 冰鉴-青云分
# # grade 评分等级
# # br_ra_score 百融-融安分
# # score_201048 融360
# # score_202098 融360
# # score_202100 融360
# # td212_score 极信-td212
# # my_zl_score 蚁盾分1.0
# # jx_yl_UPPE082 近12个月成功扣款频度占比
# # jx_yl_UPPD029 近1个月资金不足交易频度
# # jx_fico_score  极信-fico
# # jx_yl_UPPC196 近1个月消费频度
# # xy_qj_A22160004 申请命中消⾦类机构数

# var_jxz = ['jd_user_level_risk','bj_hm_score','my_zl_score2','bj_qy_score','grade','br_ra_score','score_201048','score_202098','score_202100','td212_score','my_zl_score',
#            'jx_yl_UPPE082','jx_yl_UPPD029','jx_fico_score','jx_yl_UPPC196','xy_qj_A22160004']
# var_info = ['order_number','下单日期','归属渠道','审核状态','拒绝理由']
# conditions_1 = (df_info_ori.下单日期 == '2025-05-20')
# conditions_2 = (df_info_ori.归属渠道 == '京东渠道')
# # df_info_ori[conditions_1].归属渠道.value_counts()
# # df_info_ori[:1]
# df_jxz = df_info_ori[conditions_1 & conditions_2][var_info + var_jxz]
# df_jxz.to_excel('data_jxz.xlsx', index = False)

# # （10）租物进件测算-区分型号
# # df_info_ori.tail()
# conditions_1 = (df_risk_new.归属渠道 == '芝麻租物')
# conditions_2 = (df_risk_new.进件 == '进件')
# conditions_3 = ((df_risk_new.下单日期 >= '2025-04-01') & (df_risk_new.下单日期 <= '2025-05-21'))

# tmp = df_risk_new[conditions_1 & conditions_2 & conditions_3][['order_number','product_name','下单日期','search_time','审核状态','拒绝理由','id_card_num','user_mobile','create_time']]
# tmp = tmp.merge(df_info_ori[['order_number','grade','td212_score']], on = 'order_number', how = 'left')
# tmp['product_name'] = tmp.product_name.str.replace(' ', '')
# tmp['product_name'] = tmp.apply(lambda x: x.product_name.lower(), axis = 1)

# tmp['product_new'] = np.where(tmp.product_name.str.contains('iphone16promax'), 'iphone16promax', tmp.product_name)
# # tmp['product_new'] = np.where(tmp.product_new.str.contains('iphone16pro'), 'iphone16pro', tmp.product_new)  -- 手工处理
# tmp['product_new'] = np.where(tmp.product_new.str.contains('iphone16plus'), 'iphone16plus', tmp.product_new)
# tmp['product_new'] = np.where(tmp.product_new.str.contains('iphone16e'), 'iphone16e', tmp.product_new)
# # tmp['product_new'] = np.where(tmp.product_new.str.contains('iphone16'), 'iphone16', tmp.product_new)

# tmp['product_new'] = np.where(tmp.product_new.str.contains('华为mate70pro+'), '华为mate70pro+', tmp.product_new)
# # tmp['product_new'] = np.where(tmp.product_new.str.contains('华为mate70pro'), '华为mate70pro', tmp.product_new)
# # tmp['product_new'] = np.where(tmp.product_new.str.contains('华为mate70'), '华为mate70', tmp.product_new)
# tmp['product_new'] = np.where(tmp.product_new.str.contains('华为pura70pro+'), '华为pura70pro+', tmp.product_new)

# tmp['product_new'] = np.where(tmp.product_new.str.contains('小米15pro'), '小米15pro', tmp.product_new)
# tmp['product_new'] = np.where(tmp.product_new.str.contains('小米15ultra'), '小米15ultra', tmp.product_new)
# # tmp['product'] = np.where(tmp.product.str.contains('小米15'), '小米15', tmp.product)

# tmp['product_new'] = np.where(tmp.product_new.str.contains('redmik80pro'), 'redmik80pro', tmp.product_new)
# # tmp['product'] = np.where(tmp.product_name.str.contains('redmik80'), 'redmik80', tmp.product)

# tmp.shape
# tmp[:3]

# tmp = tmp.sort_values(by = 'create_time')
# tmp = tmp.drop_duplicates(subset = ['id_card_num','user_mobile','下单日期'])
# tmp.shape
# # tmp.to_excel('cesuan_tmp_1.xlsx', index = False)

# # （11）租物-策略流变更
# # conditions_1 = (df_risk_new.归属渠道 == '芝麻租物')
# # conditions_2 = (df_risk_new.下单日期 == '2025-05-28')
# # df_risk_new[conditions_1 & conditions_2][['order_number','td212_score']].shape
# tmp1 = df_risk_new[conditions_1 & conditions_2][['order_number','进件','拒绝理由','审核状态']].merge(df_out_data[['order_number','td212_score','grade','bj_hm_score']], on = 'order_number', how = 'left')
# tmp1.shape
# tmp1[:5]
# tmp1.to_excel('tmp1.xlsx', index = False)

(2633, 7)

,order_number,进件,拒绝理由,审核状态,td212_score,grade,bj_hm_score
0,A202505280000361,未进件,[命中强拒加入临时黑名单],前置拦截,NaN,NaN,NaN
1,A202505280000453,未进件,[命中强拒加入临时黑名单],前置拦截,NaN,NaN,NaN
2,A202505280001255,未进件,None,进件前取消,NaN,NaN,NaN
3,A202505280002327,未进件,[命中强拒加入临时黑名单],前置拦截,NaN,NaN,NaN
4,A202505280002378,未进件,[命中强拒加入临时黑名单],前置拦截,NaN,NaN,NaN


In [ ]:
# （8-3）样本切分-机审
# 融安分&蚁盾分1.0
# 融安分2023.1.1就开始使用，蚁盾分1.0 2023.2开始使用
# br_ra_score、 my_zl_score
numeric_desc_var_21 = ['br_ra_score','my_zl_score']
df_train_21 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30')]
df_test_21 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-07-01')]
df_train_21['y'] = df_train_21['y_credit_mob3pd30']
df_test_21['y'] = df_test_21['y_credit_mob3pd30']
df_train_21.shape
df_test_21.shape

In [ ]:
# （8-4-1）样本切分-出库前强拒
# 训练集和测试集
# 不同产品的训练集和测试集可能不一样
# 3表示出库前强拒
# jx_fico_score ：极信-2024.2

numeric_desc_var_31 = ['jx_fico_score']
df_train_31 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30')]
df_test_31 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-07-01')]
df_train_31['y'] = df_train_31['y_credit_mob3pd30']
df_test_31['y'] = df_test_31['y_credit_mob3pd30']
df_train_31.shape
df_test_31.shape

In [ ]:
# （8-4-2）样本切分-出库前强拒
# td212_score ：腾云-2024.9-反欺诈评分-对坏客户定义是逾期超过30天,2024.9.8开始调用
# nsf_rental_v1_3_hundred ：履约保护-2024.8-mob3&dpd30+,2024.8.14开始调用

numeric_desc_var_32 = ['td212_score','nsf_rental_v1_3_hundred']
df_train_32 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-09-01') & (df_analy_ks.下单日期 <= '2024-10-31')]
df_test_32 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-11-01')]
df_train_32['y'] = df_train_32['y_credit_mob3pd30']
df_test_32['y'] = df_test_32['y_credit_mob3pd30']
df_train_32.shape
df_test_32.shape

In [125]:
# （8-5）分箱
%run C:/Users/Administrator/Desktop/mine/4_专题分析/1_回捞分析/2.model_func.ipynb
continous_merged_dict = {}
var_bin_list = []

# trainData_kf_Bin = df_train_31.copy()
# numeric_desc_var = numeric_desc_var_31

# trainData_kf_Bin = df_train_32.copy()
# numeric_desc_var = numeric_desc_var_32

# trainData_kf_Bin = df_train_21.copy()
# numeric_desc_var = numeric_desc_var_21

trainData_kf_Bin = df_train_11.copy()
numeric_desc_var = numeric_desc_var_11

for col in numeric_desc_var:
    print ("{} is in processing".format(col))
    if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
        max_interval = 5 
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
            if max_interval == 2:
                break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    else:
        max_interval = 7 #正常取值最多分5箱
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
                                      minBinPcnt = 0)
        trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
        monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
            if max_interval == 3:
                break
#             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    continous_merged_dict[col] = cutOff

bj_hm_score is in processing


In [629]:
# （8-6）改变映射方式
# trainData_kf_Bin_1 = df_train_31.copy()
# trainData_kf_Bin_1 = df_train_32.copy()
# trainData_kf_Bin_1 = df_train_21.copy()
trainData_kf_Bin_1 = df_train_11.copy()

# start_time = time.time()
for col in numeric_desc_var:
    if -1111 not in set(trainData_kf_Bin_1[col]) and -999 not in set(trainData_kf_Bin_1[col]):
        trainData_kf_Bin_1[col+'_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col]))
        # del trainData_kf_Bin_1[col]
    else:
        trainData_kf_Bin_1[col + '_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col],[-1111, -999]))
        # del trainData_kf_Bin_1[col]
        

In [ ]:
# （8-7）映射到test
# test = df_test_31.copy()
# test = df_test_32.copy()
test = df_test_21.copy()
short_list_ori = numeric_desc_var

for var in short_list_ori:
    if -1111 not in set(test[var]) and -999 not in set(test[var]):
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var]))
        # del test[var]
    else:
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var],[-1111, -999]))
        # del test[var]

test.shape

# 9.效果分析

## 9_21.火眸分

In [176]:
# （9-21）统计-火眸分

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))


pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='火眸分_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='火眸分_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='火眸分_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='火眸分_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='火眸分_Bin',columns='y_credit',values='order_number',aggfunc='count')

# df_analy_ks[(df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.search_time == '2025-03') & (df_analy_ks.bj_hm_score < 499) & (df_analy_ks.bj_hm_score > 0)][['order_number','下单日期','是否拒量','y_fraud_fpd7']]

y_fraud_fpd7,0.0,1.0
火眸分_Bin,,
"3.(499,530]",17.0,NaN
"4.(530,579]",40.0,1.0
"5.(579,657]",37.0,3.0
"6.(657,inf)",7.0,1.0
7.-999,15.0,2.0


y_fraud_mob3dpd7,0.0,1.0
火眸分_Bin,,
"3.(499,530]",17.0,NaN
"4.(530,579]",40.0,1.0
"5.(579,657]",37.0,3.0
"6.(657,inf)",7.0,1.0
7.-999,15.0,2.0


y_credit_mob3dpd30,0.0,1.0
火眸分_Bin,,
"3.(499,530]",17.0,NaN
"4.(530,579]",41.0,NaN
"5.(579,657]",39.0,1.0
"6.(657,inf)",8.0,NaN
7.-999,16.0,1.0


y_credit_mob6dpd30,0.0,1.0
火眸分_Bin,,
"3.(499,530]",17.0,NaN
"4.(530,579]",41.0,NaN
"5.(579,657]",39.0,1.0
"6.(657,inf)",8.0,NaN
7.-999,16.0,1.0


y_credit,0.0,1.0
火眸分_Bin,,
"3.(499,530]",17.0,NaN
"4.(530,579]",41.0,NaN
"5.(579,657]",39.0,1.0
"6.(657,inf)",8.0,NaN
7.-999,16.0,1.0


## 9_31.履约历史等级

In [211]:
# （9-31）统计-履约历史等级
# 细化综合风险等级 rentOnlineDetailLevelOverall
# 综合风险等级 rentOnlineRiskLevelOverall
# 共租行为等级 rentOnlineRiskLevelMultiRent
# 履约历史等级 rentOnlineRiskLevelFulfill

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelFulfill_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelFulfill_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelFulfill_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelFulfill_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelFulfill_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
rentOnlineRiskLevelFulfill_Bin,,
1.1,8.0,3.0
2.0,107.0,4.0
3.-999,1.0,NaN


y_fraud_mob3dpd7,0.0,1.0
rentOnlineRiskLevelFulfill_Bin,,
1.1,8.0,3.0
2.0,107.0,4.0
3.-999,1.0,NaN


y_credit_mob3dpd30,0.0,1.0
rentOnlineRiskLevelFulfill_Bin,,
1.1,11.0,NaN
2.0,109.0,2.0
3.-999,1.0,NaN


y_credit_mob6dpd30,0.0,1.0
rentOnlineRiskLevelFulfill_Bin,,
1.1,11.0,NaN
2.0,109.0,2.0
3.-999,1.0,NaN


y_credit,0.0,1.0
rentOnlineRiskLevelFulfill_Bin,,
1.1,11.0,NaN
2.0,109.0,2.0
3.-999,1.0,NaN


## 9_32.综合风险等级

In [247]:
# （9-32）统计-综合风险等级
# 细化综合风险等级 rentOnlineDetailLevelOverall
# 综合风险等级 rentOnlineRiskLevelOverall
# 共租行为等级 rentOnlineRiskLevelMultiRent
# 履约历史等级 rentOnlineRiskLevelFulfill

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelOverall_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelOverall_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelOverall_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelOverall_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineRiskLevelOverall_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
rentOnlineRiskLevelOverall_Bin,,
2.2,19.0,3.0
3.1,38.0,3.0
4.0,58.0,1.0
5.-999,1.0,NaN


y_fraud_mob3dpd7,0.0,1.0
rentOnlineRiskLevelOverall_Bin,,
2.2,19.0,3.0
3.1,38.0,3.0
4.0,58.0,1.0
5.-999,1.0,NaN


y_credit_mob3dpd30,0.0,1.0
rentOnlineRiskLevelOverall_Bin,,
2.2,22.0,NaN
3.1,39.0,2.0
4.0,59.0,NaN
5.-999,1.0,NaN


y_credit_mob6dpd30,0.0,1.0
rentOnlineRiskLevelOverall_Bin,,
2.2,22.0,NaN
3.1,39.0,2.0
4.0,59.0,NaN
5.-999,1.0,NaN


y_credit,0.0,1.0
rentOnlineRiskLevelOverall_Bin,,
2.2,22.0,NaN
3.1,39.0,2.0
4.0,59.0,NaN
5.-999,1.0,NaN


## 9_33.细分综合风险等级

In [281]:
# （9-33）统计-细分综合风险等级
# 细化综合风险等级 rentOnlineDetailLevelOverall
# 综合风险等级 rentOnlineRiskLevelOverall
# 共租行为等级 rentOnlineRiskLevelMultiRent
# 履约历史等级 rentOnlineRiskLevelFulfill

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineDetailLevelOverall_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineDetailLevelOverall_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineDetailLevelOverall_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineDetailLevelOverall_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='rentOnlineDetailLevelOverall_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
rentOnlineDetailLevelOverall_Bin,,
3.4,15.0,1.0
4.3,13.0,3.0
5.2,18.0,NaN
6.1,32.0,1.0
7.0,37.0,2.0
8.-999,1.0,NaN


y_fraud_mob3dpd7,0.0,1.0
rentOnlineDetailLevelOverall_Bin,,
3.4,15.0,1.0
4.3,13.0,3.0
5.2,18.0,NaN
6.1,32.0,1.0
7.0,37.0,2.0
8.-999,1.0,NaN


y_credit_mob3dpd30,0.0,1.0
rentOnlineDetailLevelOverall_Bin,,
3.4,16.0,NaN
4.3,15.0,1.0
5.2,18.0,NaN
6.1,32.0,1.0
7.0,39.0,NaN
8.-999,1.0,NaN


y_credit_mob6dpd30,0.0,1.0
rentOnlineDetailLevelOverall_Bin,,
3.4,16.0,NaN
4.3,15.0,1.0
5.2,18.0,NaN
6.1,32.0,1.0
7.0,39.0,NaN
8.-999,1.0,NaN


y_credit,0.0,1.0
rentOnlineDetailLevelOverall_Bin,,
3.4,16.0,NaN
4.3,15.0,1.0
5.2,18.0,NaN
6.1,32.0,1.0
7.0,39.0,NaN
8.-999,1.0,NaN


## 9_34_1.蚁盾分1.0

In [319]:
# （9-34-1）统计-蚁盾分1.0
# '蚁盾分_Bin'
# 规则17		蚁盾分>=80
# 规则18		蚁盾分>50且为搜索渠道

conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

view_m = '2024-12'
# view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='蚁盾分_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='蚁盾分_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='蚁盾分_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='蚁盾分_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='蚁盾分_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
蚁盾分_Bin,,
"3.(70,80)",29.0,1.0
"4.(60,70]",45.0,NaN
"5.(50,60]",65.0,4.0
"6.(25,50]",386.0,12.0
"7.[0,25]",825.0,12.0
8.-999,7.0,NaN


y_fraud_mob3dpd7,0.0,1.0
蚁盾分_Bin,,
"3.(70,80)",28,2
"4.(60,70]",42,3
"5.(50,60]",59,10
"6.(25,50]",346,52
"7.[0,25]",770,67
8.-999,6,1


y_credit_mob3dpd30,0.0,1.0
蚁盾分_Bin,,
"3.(70,80)",29.0,1.0
"4.(60,70]",43.0,2.0
"5.(50,60]",62.0,7.0
"6.(25,50]",358.0,40.0
"7.[0,25]",788.0,49.0
8.-999,7.0,NaN


y_credit_mob6dpd30,0.0,1.0
蚁盾分_Bin,,
"3.(70,80)",29.0,1.0
"4.(60,70]",42.0,3.0
"5.(50,60]",60.0,9.0
"6.(25,50]",349.0,49.0
"7.[0,25]",778.0,59.0
8.-999,7.0,NaN


y_credit,0.0,1.0
蚁盾分_Bin,,
"3.(70,80)",29.0,1.0
"4.(60,70]",42.0,3.0
"5.(50,60]",60.0,9.0
"6.(25,50]",349.0,49.0
"7.[0,25]",778.0,59.0
8.-999,7.0,NaN


## 9_34_2.蚁盾分2.0

In [344]:
# （9-34-2）统计-蚁盾分2.0
# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='my_zl_score2_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='my_zl_score2_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='my_zl_score2_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='my_zl_score2_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='my_zl_score2_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
my_zl_score2_Bin,,
"2.[70,75)",1.0,NaN
"3.[65,70)",4.0,NaN
"4.(50,65)",94.0,3.0
"5.(45,50]",9.0,2.0
"6.(25,45]",3.0,1.0
8.-999,5.0,1.0


y_fraud_mob3dpd7,0.0,1.0
my_zl_score2_Bin,,
"2.[70,75)",1.0,NaN
"3.[65,70)",4.0,NaN
"4.(50,65)",94.0,3.0
"5.(45,50]",9.0,2.0
"6.(25,45]",3.0,1.0
8.-999,5.0,1.0


y_credit_mob3dpd30,0.0,1.0
my_zl_score2_Bin,,
"2.[70,75)",1.0,NaN
"3.[65,70)",4.0,NaN
"4.(50,65)",95.0,2.0
"5.(45,50]",11.0,NaN
"6.(25,45]",4.0,NaN
8.-999,6.0,NaN


y_credit_mob6dpd30,0.0,1.0
my_zl_score2_Bin,,
"2.[70,75)",1.0,NaN
"3.[65,70)",4.0,NaN
"4.(50,65)",95.0,2.0
"5.(45,50]",11.0,NaN
"6.(25,45]",4.0,NaN
8.-999,6.0,NaN


y_credit,0.0,1.0
my_zl_score2_Bin,,
"2.[70,75)",1.0,NaN
"3.[65,70)",4.0,NaN
"4.(50,65)",95.0,2.0
"5.(45,50]",11.0,NaN
"6.(25,45]",4.0,NaN
8.-999,6.0,NaN


## 9_35.风险勘测

In [ ]:
# （9-35）统计-风险勘测


## 9_36.云商分

In [380]:
# （9-36）统计-云商分

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='br_ys_score_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='br_ys_score_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='br_ys_score_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='br_ys_score_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='br_ys_score_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
br_ys_score_Bin,,
"4.(495,514.0]",2.0,NaN
"5.(514,542.0]",10.0,NaN
"6.(542,555.0]",13.0,NaN
"7.(555,615.0]",23.0,1.0
"8.(615,660.0]",23.0,1.0
"9.(660,705.0]",18.0,NaN
"91.(705,742.0]",14.0,NaN
"92.(742,inf)",13.0,5.0


y_fraud_mob3dpd7,0.0,1.0
br_ys_score_Bin,,
"4.(495,514.0]",2.0,NaN
"5.(514,542.0]",10.0,NaN
"6.(542,555.0]",13.0,NaN
"7.(555,615.0]",23.0,1.0
"8.(615,660.0]",23.0,1.0
"9.(660,705.0]",18.0,NaN
"91.(705,742.0]",14.0,NaN
"92.(742,inf)",13.0,5.0


y_credit_mob3dpd30,0.0,1.0
br_ys_score_Bin,,
"4.(495,514.0]",2.0,NaN
"5.(514,542.0]",10.0,NaN
"6.(542,555.0]",13.0,NaN
"7.(555,615.0]",24.0,NaN
"8.(615,660.0]",24.0,NaN
"9.(660,705.0]",18.0,NaN
"91.(705,742.0]",14.0,NaN
"92.(742,inf)",16.0,2.0


y_credit_mob6dpd30,0.0,1.0
br_ys_score_Bin,,
"4.(495,514.0]",2.0,NaN
"5.(514,542.0]",10.0,NaN
"6.(542,555.0]",13.0,NaN
"7.(555,615.0]",24.0,NaN
"8.(615,660.0]",24.0,NaN
"9.(660,705.0]",18.0,NaN
"91.(705,742.0]",14.0,NaN
"92.(742,inf)",16.0,2.0


y_credit,0.0,1.0
br_ys_score_Bin,,
"4.(495,514.0]",2.0,NaN
"5.(514,542.0]",10.0,NaN
"6.(542,555.0]",13.0,NaN
"7.(555,615.0]",24.0,NaN
"8.(615,660.0]",24.0,NaN
"9.(660,705.0]",18.0,NaN
"91.(705,742.0]",14.0,NaN
"92.(742,inf)",16.0,2.0


## 9_37.融安分

In [1046]:
# （9-37）统计-融安分

# '融安分_Bin'
# 规则25		命中融安分低于680
# 规则26		命中融安分低于700且为搜索渠道


# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-01'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='融安分_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='融安分_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='融安分_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='融安分_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='融安分_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
融安分_Bin,,
"3.(650,680]",1.0,NaN
"4.(680,700]",10.0,NaN
"5.(700,750]",31.0,1.0
"6.(750,800]",19.0,NaN
"7.(800,inf)",42.0,1.0
8.-999,1.0,NaN


y_fraud_mob3dpd7,0.0,1.0
融安分_Bin,,
"3.(650,680]",1.0,NaN
"4.(680,700]",8.0,2.0
"5.(700,750]",27.0,5.0
"6.(750,800]",17.0,2.0
"7.(800,inf)",40.0,3.0
8.-999,NaN,1.0


y_credit_mob3dpd30,0.0,1.0
融安分_Bin,,
"3.(650,680]",1.0,NaN
"4.(680,700]",8.0,2.0
"5.(700,750]",29.0,3.0
"6.(750,800]",18.0,1.0
"7.(800,inf)",41.0,2.0
8.-999,1.0,NaN


y_credit_mob6dpd30,0.0,1.0
融安分_Bin,,
"3.(650,680]",1.0,NaN
"4.(680,700]",8.0,2.0
"5.(700,750]",29.0,3.0
"6.(750,800]",18.0,1.0
"7.(800,inf)",41.0,2.0
8.-999,1.0,NaN


y_credit,0.0,1.0
融安分_Bin,,
"3.(650,680]",1.0,NaN
"4.(680,700]",8.0,2.0
"5.(700,750]",29.0,3.0
"6.(750,800]",18.0,1.0
"7.(800,inf)",41.0,2.0
8.-999,1.0,NaN


## 9_38.自有模型

In [ ]:
# （9-2-1）统计-模型
conditions_1 = (df_analy_ks.is_model == '1')
conditions_2 = (df_analy_ks.is_model == '2')
conditions_3 = (df_analy_ks.is_model == '3')
conditions_4 = (df_analy_ks.is_model == '4')
conditions_6 = ((~df_analy_ks.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

conditions_5 = (df_analy_ks.search_time == '2025-03')



# other 的为抖音、号卡或拒量--另外一种分组

pd.pivot_table(df_analy_ks[conditions_1 & conditions_5 & conditions_6],index='model_score_2311_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_5 & conditions_6],index='model_score_2311_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_5 & conditions_6],index='model_score_2311_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_5 & conditions_6],index='model_score_2311_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_5 & conditions_6],index='model_score_2311_Bin',columns='y_credit',values='order_number',aggfunc='count')

# pd.pivot_table(df_analy_ks[conditions_2 & conditions_5 & conditions_6],index='model_score_yl_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_2 & conditions_5 & conditions_6],index='model_score_yl_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_2 & conditions_5 & conditions_6],index='model_score_yl_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_2 & conditions_5 & conditions_6],index='model_score_yl_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_2 & conditions_5 & conditions_6],index='model_score_yl_Bin',columns='y_credit',values='order_number',aggfunc='count')

# pd.pivot_table(df_analy_ks[conditions_3 & conditions_5 & conditions_6],index='model_score_2401_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_3 & conditions_5 & conditions_6],index='model_score_2401_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_3 & conditions_5 & conditions_6],index='model_score_2401_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_3 & conditions_5 & conditions_6],index='model_score_2401_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_3 & conditions_5 & conditions_6],index='model_score_2401_Bin',columns='y_credit',values='order_number',aggfunc='count')

# pd.pivot_table(df_analy_ks[conditions_4 & conditions_5 & conditions_6 & (df_analy_ks.下单日期 >= '2025-02-19')],index='model_score_2501_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_4 & conditions_5 & conditions_6 & (df_analy_ks.下单日期 >= '2025-02-19')],index='model_score_2501_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_4 & conditions_5 & conditions_6 & (df_analy_ks.下单日期 >= '2025-02-19')],index='model_score_2501_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_4 & conditions_5 & conditions_6 & (df_analy_ks.下单日期 >= '2025-02-19')],index='model_score_2501_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_4 & conditions_5 & conditions_6 & (df_analy_ks.下单日期 >= '2025-02-19')],index='model_score_2501_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
model_score_2311_Bin,,
"1.[0,12.9620]",3.0,NaN
"2.(12.9620,15.2211]",20.0,NaN
"3.(15.2211,18.321]",62.0,4.0
"4.(18.321, 23.297]",66.0,1.0
"5.(23.297, inf)",1.0,NaN
"6.[0,18.321]",13.0,1.0
"7.(18.321, inf)",4.0,NaN
"8.[0,16.6]",35.0,2.0
"9.(16.6, 21.435]",46.0,4.0


In [1027]:
# 数据检查
conditions_6 = ((df_analy_ks.归属渠道 == '芝麻租物')&(df_analy_ks.model_score_2501 > 18.132))
conditions_7 = ((df_analy_ks.归属渠道 == '搜索渠道')&(df_analy_ks.model_score_2501 > 18.098))
# var = ['order_number','is_model','model_score_2501','model_score_2501_Bin','归属渠道','search_time','y_fraud_fpd7','y_fraud_mob3dpd7','下单日期','是否拒量']
# df_analy_ks[conditions_4 & (df_analy_ks.下单日期 >= '2025-02-19') & conditions_6][var].shape
# df_analy_ks[conditions_4 & (df_analy_ks.下单日期 >= '2025-02-19') & conditions_7][var].shape

# df_analy_ks[conditions_4 & (df_analy_ks.下单日期 >= '2025-02-19') & (df_analy_ks.下单日期 <= '2025-03-31') & conditions_6][var]
# df_analy_ks[conditions_4 & (df_analy_ks.下单日期 >= '2025-02-19') & (df_analy_ks.下单日期 <= '2025-03-31') & conditions_7][var]

# df_analy_ks[conditions_4 & (df_analy_ks.下单日期 >= '2025-02-19') & conditions_6].是否拒量.value_counts()
# df_analy_ks[conditions_4 & (df_analy_ks.下单日期 >= '2025-02-19') & conditions_7].是否拒量.value_counts()

conditions_5 = (df_analy_ks.search_time == '2024-12')
# df_analy_ks[conditions_5 & (df_analy_ks.是否拒量 == 0) & (df_analy_ks.归属渠道 == '芝麻租物')].shape
# df_analy_ks[conditions_5 & (df_analy_ks.是否拒量 == 0) & (df_analy_ks.归属渠道 == '搜索渠道')].shape

# df_analy_ks[conditions_5 & (df_analy_ks.是否拒量 == 0) & (df_analy_ks.归属渠道 == '芝麻租物')].is_model.value_counts()
# df_analy_ks[conditions_5 & (df_analy_ks.是否拒量 == 0) & (df_analy_ks.归属渠道 == '搜索渠道')].is_model.value_counts()

# df_analy_ks['tmp'] = df_analy_ks.apply(lambda x: '芝麻租物' if x.归属渠道 == '芝麻租物' 
#                                        else '搜索渠道' if x.归属渠道 == '搜索渠道'
#                                        else '其他' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1'
#                                        else 'other', axis = 1)
df_analy_ks[conditions_5 & (df_analy_ks.是否拒量 == 0) & (conditions_3)].tmp.value_counts()

df_analy_ks[conditions_5 & (df_analy_ks.是否拒量 == 0) & (df_analy_ks.tmp != 'other')].is_model.value_counts()

tmp
芝麻租物     284
搜索渠道     273
其他        58
other     11
Name: count, dtype: int64

is_model
2    1897
3     615
1     569
Name: count, dtype: int64

## 3_39.青云分

In [458]:
# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='青云分_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='青云分_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='青云分_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='青云分_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='青云分_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
青云分_Bin,,
"2.(449,499]",12.0,3.0
"3.(499,533.0]",20.0,1.0
"4.(533.0,575.0]",33.0,1.0
"5.(575.0,607.0]",22.0,1.0
"6.(607.0,652.0]",17.0,1.0
"7.(652, inf)",12.0,NaN


y_fraud_mob3dpd7,0.0,1.0
青云分_Bin,,
"2.(449,499]",12.0,3.0
"3.(499,533.0]",20.0,1.0
"4.(533.0,575.0]",33.0,1.0
"5.(575.0,607.0]",22.0,1.0
"6.(607.0,652.0]",17.0,1.0
"7.(652, inf)",12.0,NaN


y_credit_mob3dpd30,0.0,1.0
青云分_Bin,,
"2.(449,499]",14.0,1.0
"3.(499,533.0]",20.0,1.0
"4.(533.0,575.0]",34.0,NaN
"5.(575.0,607.0]",23.0,NaN
"6.(607.0,652.0]",18.0,NaN
"7.(652, inf)",12.0,NaN


y_credit_mob6dpd30,0.0,1.0
青云分_Bin,,
"2.(449,499]",14.0,1.0
"3.(499,533.0]",20.0,1.0
"4.(533.0,575.0]",34.0,NaN
"5.(575.0,607.0]",23.0,NaN
"6.(607.0,652.0]",18.0,NaN
"7.(652, inf)",12.0,NaN


y_credit,0.0,1.0
青云分_Bin,,
"2.(449,499]",14.0,1.0
"3.(499,533.0]",20.0,1.0
"4.(533.0,575.0]",34.0,NaN
"5.(575.0,607.0]",23.0,NaN
"6.(607.0,652.0]",18.0,NaN
"7.(652, inf)",12.0,NaN


## 9_310.评分等级

In [493]:
# （9-39）统计-评分等级

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='grade_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='grade_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='grade_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='grade_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='grade_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
grade_Bin,,
3.D,10.0,3.0
4.C,22.0,2.0
5.B,32.0,1.0
6.A,23.0,NaN
7.S,29.0,1.0


y_fraud_mob3dpd7,0.0,1.0
grade_Bin,,
3.D,10.0,3.0
4.C,22.0,2.0
5.B,32.0,1.0
6.A,23.0,NaN
7.S,29.0,1.0


y_credit_mob3dpd30,0.0,1.0
grade_Bin,,
3.D,12.0,1.0
4.C,23.0,1.0
5.B,33.0,NaN
6.A,23.0,NaN
7.S,30.0,NaN


y_credit_mob6dpd30,0.0,1.0
grade_Bin,,
3.D,12.0,1.0
4.C,23.0,1.0
5.B,33.0,NaN
6.A,23.0,NaN
7.S,30.0,NaN


y_credit,0.0,1.0
grade_Bin,,
3.D,12.0,1.0
4.C,23.0,1.0
5.B,33.0,NaN
6.A,23.0,NaN
7.S,30.0,NaN


## 9_311.多头

## 9_312.策略240703_4

In [524]:
#  模型分 ascore_2401<=14.27667 and 极信银联指标 UPPC134>1.5 and 极信银联指标 UPPC134<=3.5 and 极信银联指标 UPPD007>47
# jx_yl_UPPC134	近1个月出账类交易频度
# jx_yl_UPPD007	近6个月失败交易频度

# model_score_2311 model_score_2401 model_score_yl
# df_analy_ks[['model_score_2401','jx_yl_UPPC134','jx_yl_UPPD007']].tail()

# 客群1--目标强拒客群：model_score_2401 <= 14.27667 and jx_yl_UPPC134 > 1.5 and jx_yl_UPPC134 <= 3.5 and jx_yl_UPPD007 > 47
# 客群2：model_score_2401 > 14.27667
# 客群3：model_score_2401 <= 14.27667 and jx_yl_UPPC134 <= 1.5
# 客群4：model_score_2401 <= 14.27667 and jx_yl_UPPC134 > 1.5 and jx_yl_UPPC134 <= 3.5 and jx_yl_UPPD007 <= 47
# 客群5：model_score_2401 <= 14.27667 and jx_yl_UPPC134 > 3.5

# 客群6表示银联指标未覆盖到
# df_analy_ks[(df_analy_ks.strategy_240703_4_Bin == '6.-999')&(df_analy_ks.search_time == '2025-02')&conditions_1]
# A20250210105718271
# A2025022222072929
# df_out_data[df_out_data.order_number == 'A20250210105718271']


# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-02-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_4_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_4_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_4_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_4_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_4_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0,1.0
strategy_240703_4_Bin,,
2.客群2,14.0,NaN
3.客群3,11.0,3.0
4.客群4,5.0,1.0
5.客群5,78.0,2.0
6.-999,8.0,1.0


y_fraud_mob3dpd7,0.0,1.0
strategy_240703_4_Bin,,
2.客群2,14.0,NaN
3.客群3,11.0,3.0
4.客群4,5.0,1.0
5.客群5,78.0,2.0
6.-999,8.0,1.0


y_credit_mob3dpd30,0.0,1.0
strategy_240703_4_Bin,,
2.客群2,14.0,NaN
3.客群3,14.0,NaN
4.客群4,6.0,NaN
5.客群5,79.0,1.0
6.-999,8.0,1.0


y_credit_mob6dpd30,0.0,1.0
strategy_240703_4_Bin,,
2.客群2,14.0,NaN
3.客群3,14.0,NaN
4.客群4,6.0,NaN
5.客群5,79.0,1.0
6.-999,8.0,1.0


y_credit,0.0,1.0
strategy_240703_4_Bin,,
2.客群2,14.0,NaN
3.客群3,14.0,NaN
4.客群4,6.0,NaN
5.客群5,79.0,1.0
6.-999,8.0,1.0


## 9_313.策略240703_10

In [557]:
# # 模型分 ascore_2401>14.27667 and 新颜全景指标 A22160001<=623.5 and 新颜全景指标 B22170036>8.5
# # xy_qj_A22160001	申请准⼊分
# # xy_qj_B22170036	近3个⽉失败扣款笔数

# # 客群1-目标客群： x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 <= 623.5 and x.xy_qj_B22170036 > 8.5
# # 客群2： x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 <= 623.5 and x.xy_qj_B22170036 <= 8.5
# # 客群3：x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 > 623.5
# # 客群4：x.model_score_2401 >= 0 and x.model_score_2401 <= 14.27667


# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-02-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_10_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_10_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_10_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_10_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_10_Bin',columns='y_credit',values='order_number',aggfunc='count')

# df_analy_ks['type'] = df_analy_ks.apply(lambda x: 1 if x.归属渠道 == '芝麻租物' and x.是否拒量 == 0
#                                         else 2 if x.归属渠道 == '搜索渠道' and x.是否拒量 == 0
#                                         else 3 if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 == 0
#                                         else 4, axis = 1)
# # # df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'].groupby(['type','search_time','strategy_240703_10_Bin','y_fraud_fpd7']).agg({'order_number': 'count'})
# tmp1 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_10_Bin'],columns='y_fraud_fpd7',values='order_number',aggfunc='count').reset_index()
# tmp2 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_10_Bin'],columns='y_fraud_f3pd7',values='order_number',aggfunc='count').reset_index()
# tmp3 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_10_Bin'],columns='y_credit_mob3pd30',values='order_number',aggfunc='count').reset_index()
# tmp4 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_10_Bin'],columns='y_credit',values='order_number',aggfunc='count').reset_index()


# tmp1.to_excel('tmp1.xlsx', index = False)
# tmp2.to_excel('tmp2.xlsx', index = False)
# tmp3.to_excel('tmp3.xlsx', index = False)
# tmp4.to_excel('tmp4.xlsx', index = False)

# tmp1[:5]

y_fraud_fpd7,0.0,1.0
strategy_240703_10_Bin,,
2.客群2,4.0,NaN
3.客群3,2.0,NaN
4.客群4,102.0,7.0
5.-999,8.0,NaN


y_fraud_mob3dpd7,0.0,1.0
strategy_240703_10_Bin,,
2.客群2,4.0,NaN
3.客群3,2.0,NaN
4.客群4,102.0,7.0
5.-999,8.0,NaN


y_credit_mob3dpd30,0.0,1.0
strategy_240703_10_Bin,,
2.客群2,4.0,NaN
3.客群3,2.0,NaN
4.客群4,107.0,2.0
5.-999,8.0,NaN


y_credit_mob6dpd30,0.0,1.0
strategy_240703_10_Bin,,
2.客群2,4.0,NaN
3.客群3,2.0,NaN
4.客群4,107.0,2.0
5.-999,8.0,NaN


y_credit,0.0,1.0
strategy_240703_10_Bin,,
2.客群2,4.0,NaN
3.客群3,2.0,NaN
4.客群4,107.0,2.0
5.-999,8.0,NaN


## 9_314.策略240703_14

In [590]:
# # 模型分 ascore_2401>14.27667 and 极信银联指标 UPPE074>1.345 and 蚂蚁链租赁风险分（蚁盾分）AntChain>18.324
# # jx_yl_UPPE074	近12个月平均代扣失败天数

# # 客群1-目标客群： x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 > 1.345 and x.my_zl_score > 18.324
# # 客群2： x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 > 1.345 and x.my_zl_score <= 18.324
# # 客群3： x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 <= 1.345
# # 客群4： x.model_score_2401 >= 0 and x.model_score_2401 <= 14.27667

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-03'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-02-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_14_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_14_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_14_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_14_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240703_14_Bin',columns='y_credit',values='order_number',aggfunc='count')

# df_analy_ks['type'] = df_analy_ks.apply(lambda x: 1 if x.归属渠道 == '芝麻租物' and x.是否拒量 == 0
#                                         else 2 if x.归属渠道 == '搜索渠道' and x.是否拒量 == 0
#                                         else 3 if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 == 0
#                                         else 4, axis = 1)
# # # df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'].groupby(['type','search_time','strategy_240703_14_Bin','y_fraud_fpd7']).agg({'order_number': 'count'})
# tmp1 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_14_Bin'],columns='y_fraud_fpd7',values='order_number',aggfunc='count').reset_index()
# tmp2 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_14_Bin'],columns='y_fraud_f3pd7',values='order_number',aggfunc='count').reset_index()
# tmp3 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_14_Bin'],columns='y_credit_mob3pd30',values='order_number',aggfunc='count').reset_index()
# tmp4 = pd.pivot_table(df_analy_ks[df_analy_ks.下单日期 >= '2024-07-01'],index=['type','search_time','strategy_240703_14_Bin'],columns='y_credit',values='order_number',aggfunc='count').reset_index()


# tmp1.to_excel('tmp1.xlsx', index = False)
# tmp2.to_excel('tmp2.xlsx', index = False)
# tmp3.to_excel('tmp3.xlsx', index = False)
# tmp4.to_excel('tmp4.xlsx', index = False)

y_fraud_fpd7,0.0,1.0
strategy_240703_14_Bin,,
3.客群3,6.0,NaN
4.客群4,102.0,7.0
5.-999,8.0,NaN


y_fraud_mob3dpd7,0.0,1.0
strategy_240703_14_Bin,,
3.客群3,6.0,NaN
4.客群4,102.0,7.0
5.-999,8.0,NaN


y_credit_mob3dpd30,0.0,1.0
strategy_240703_14_Bin,,
3.客群3,6.0,NaN
4.客群4,107.0,2.0
5.-999,8.0,NaN


y_credit_mob6dpd30,0.0,1.0
strategy_240703_14_Bin,,
3.客群3,6.0,NaN
4.客群4,107.0,2.0
5.-999,8.0,NaN


y_credit,0.0,1.0
strategy_240703_14_Bin,,
3.客群3,6.0,NaN
4.客群4,107.0,2.0
5.-999,8.0,NaN


## 9_315.策略240801

In [959]:
# "def get_rule(df):
#     #拒绝策略
#     df['kill_rule1']=np.where((df['jx_fico_score']<=561.5)&(df['model_score_2311']>17.684),1,0) -1
#     df['kill_rule2']=np.where((df['jx_fico_score']<=561.5)&(df['model_score_yl']>21.6),1,0)     -2
#     df['kill_rule3']=np.where((df['jx_fico_score']<=561.5)&(df['model_score_2401']>14.326),1,0) -3
#     df['kill_rule4']=np.where((df['xy_qj_C22180008']<=10.5)&(df['xy_qj_B22170035']>4.5),1,0)    --5 业务解释跟实际相悖
#     df['kill_rule5']=np.where((df['bj_qy_score']<=433.5),1,0)
#     df['kill_rule6']=np.where((df['jx_fico_score']<=561.5)&(df['my_zl_score']>20.885),1,0)      -4
#     #通过策略
#     df['pass_rule1']=np.where((df['jx_fico_score']>604.5)&(df['jx_yl_UPPE082']>0.655),1,0)        -7
#     df['pass_rule2']=np.where((df['jx_fico_score']>604.5)&(df['model_score_2401']<=8.093),1,0)    -8
#     df['pass_rule3']=np.where((df['jx_fico_score']>561.5)&(df['model_score_2401']<=9.849),1,0)    -9
#     df['pass_rule4']=np.where((df['model_score_2401']<=14.326)&(df['xy_qj_C22180008']>10.5),1,0)  -11 业务解释跟实际相悖
#     df['pass_rule5']=np.where((df['jx_fico_score']>561.5)&(df['bj_qy_score']>433.5),1,0)          -10

#     #拒绝策略汇总
#     df['kill_rule_all'] = df[[ 'kill_rule1', 'kill_rule2', 'kill_rule3',
#                              'kill_rule4', 'kill_rule5', 'kill_rule6']].max(axis=1)
#     #通过策略汇总
#     df['pass_rule_all'] = df[['pass_rule1', 'pass_rule2', 'pass_rule3', 
#                               'pass_rule4', 'pass_rule5']].max(axis=1)
#     #实际通过策略，命中拒绝则不通过
#     df['pass_rule_fin'] = '待定'
#     df.loc[(df['pass_rule_all']==1),'pass_rule_fin'] = '通过'
#     df.loc[(df['kill_rule_all']==1),'pass_rule_fin'] = '拒绝'

# 命中拒绝策略直接拒绝，命中通过策略免信审，其他部分进一步信审
# 在机审最后，机审通过才会跑，先空跑获取结果，不做策略判断
# 免人审且命中多头，不豁免上锁，每天豁免20台
# 单人会话、扫一扫 渠道非免人审的强拒

# jx_fico_score：2024.4正式上线，2024.1-2024.3 少量调用
# 全景雷达：2023.11上线
# 蚁盾分1.0:2023.1上线
# 银联：2023.11上线

# 该策略2024.8.1上线
# my_zl_score model_score_2311	model_score_2401	model_score_yl	
# xy_qj_C22180008	消⾦贷款类产品数
# xy_qj_B22170035	近1个⽉失败扣款笔数
# jx_yl_UPPE082	近12个月成功扣款频度占比


# strategy_240801_1_Bin
# 1-5  7-11

# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

view_m = '2025-01'
# view_m = '2024-11'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-04-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

# strategy_240801_1_Bin
# strategy_240801_2_Bin
# strategy_240801_3_Bin
# strategy_240801_4_Bin
# strategy_240801_5_Bin
# strategy_240801_7_Bin
# strategy_240801_8_Bin
# strategy_240801_9_Bin
# strategy_240801_10_Bin
# strategy_240801_11_Bin

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240801_11_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240801_11_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240801_11_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240801_11_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='strategy_240801_11_Bin',columns='y_credit',values='order_number',aggfunc='count')


y_fraud_fpd7,0.0,1.0
strategy_240801_11_Bin,,
1.客群1,11.0,1.0
2.客群2,22.0,NaN
3.客群3,29.0,1.0
4.-999,42.0,NaN


y_fraud_mob3dpd7,0.0,1.0
strategy_240801_11_Bin,,
1.客群1,9,3
2.客群2,19,3
3.客群3,28,2
4.-999,37,5


y_credit_mob3dpd30,0.0,1.0
strategy_240801_11_Bin,,
1.客群1,10,2
2.客群2,21,1
3.客群3,28,2
4.-999,39,3


y_credit_mob6dpd30,0.0,1.0
strategy_240801_11_Bin,,
1.客群1,10,2
2.客群2,21,1
3.客群3,28,2
4.-999,39,3


y_credit,0.0,1.0
strategy_240801_11_Bin,,
1.客群1,10,2
2.客群2,21,1
3.客群3,28,2
4.-999,39,3


## 9_316.融360

In [283]:
# score_201048_Bin 
# score_202098_Bin 
# score_202100_Bin

conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

# view_m = '2024-12'
view_m = '2025-05'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2023-10-01') & (df_analy_ks.下单日期 <= '2023-12-31'))
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='score_201048_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='score_201048_Bin',columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='score_201048_Bin',columns='y_credit_mob3dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='score_201048_Bin',columns='y_credit_mob6dpd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='score_201048_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud_fpd7,0.0
score_201048_Bin,
"1.[0,409]",6
"2.(409,424]",6
"3.(424,517]",365
"4.(517,543]",261
"5.(543,588]",483
"6.(588,inf)",442
"7.[-999,-999]",12


y_fraud_mob3dpd7,0.0
score_201048_Bin,
"1.[0,409]",6
"2.(409,424]",6
"3.(424,517]",365
"4.(517,543]",261
"5.(543,588]",483
"6.(588,inf)",442
"7.[-999,-999]",12


y_credit_mob3dpd30,0.0
score_201048_Bin,
"1.[0,409]",6
"2.(409,424]",6
"3.(424,517]",365
"4.(517,543]",261
"5.(543,588]",483
"6.(588,inf)",442
"7.[-999,-999]",12


y_credit_mob6dpd30,0.0
score_201048_Bin,
"1.[0,409]",6
"2.(409,424]",6
"3.(424,517]",365
"4.(517,543]",261
"5.(543,588]",483
"6.(588,inf)",442
"7.[-999,-999]",12


y_credit,0.0
score_201048_Bin,
"1.[0,409]",6
"2.(409,424]",6
"3.(424,517]",365
"4.(517,543]",261
"5.(543,588]",483
"6.(588,inf)",442
"7.[-999,-999]",12


## 9_41.出库前

In [ ]:
# （9-3-1）统计-极信分
# 规则40，结合规则41一起分析
trainData_kf_Bin_1['jx_fico_score_Bin'] = np.where((trainData_kf_Bin_1.jx_fico_score >= 0)&(trainData_kf_Bin_1.jx_fico_score <= 489), '(-999,489]', trainData_kf_Bin_1.jx_fico_score_Bin)
trainData_kf_Bin_1['jx_fico_score_Bin'] = np.where((trainData_kf_Bin_1.jx_fico_score > 489)&(trainData_kf_Bin_1.jx_fico_score <= 536), '(489,536]', trainData_kf_Bin_1.jx_fico_score_Bin)
trainData_kf_Bin_1['jx_fico_score_Bin'] = np.where((trainData_kf_Bin_1.jx_fico_score > 536)&(trainData_kf_Bin_1.jx_fico_score <= 563), '(536.0,563.0]', trainData_kf_Bin_1.jx_fico_score_Bin)

test['jx_fico_score_Bin'] = np.where((test.jx_fico_score >= 0)&(test.jx_fico_score <= 489), '(-999,489]', test.jx_fico_score_Bin)
test['jx_fico_score_Bin'] = np.where((test.jx_fico_score > 489)&(test.jx_fico_score <= 536), '(489,536]', test.jx_fico_score_Bin)
test['jx_fico_score_Bin'] = np.where((test.jx_fico_score > 536)&(test.jx_fico_score <= 563), '(536.0,563.0]', test.jx_fico_score_Bin)

# conditions_1 = (trainData_kf_Bin_1.归属渠道 != '抖音渠道')
# pd.pivot_table(trainData_kf_Bin_1[conditions_1],index='jx_fico_score_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(trainData_kf_Bin_1[conditions_1],index='jx_fico_score_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(trainData_kf_Bin_1[conditions_1],index='jx_fico_score_Bin',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
# pd.pivot_table(trainData_kf_Bin_1[conditions_1],index='jx_fico_score_Bin',columns='y_credit',values='order_number',aggfunc='count')

view_m = '2025-01'
conditions_1 = (test.归属渠道 != '抖音渠道')
conditions_2 = (test.search_time == view_m)
pd.pivot_table(test[conditions_1 & conditions_2],index='jx_fico_score_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(test[conditions_1 & conditions_2],index='jx_fico_score_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
pd.pivot_table(test[conditions_1 & conditions_2],index='jx_fico_score_Bin',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
pd.pivot_table(test[conditions_1 & conditions_2],index='jx_fico_score_Bin',columns='y_credit',values='order_number',aggfunc='count')

# 分析-极信分
# （1）2024.7-2025.1 规则41 豁免了200 个订单
# 规则41为随机豁免的订单，200
# test[(test.jx_fico_score >= 0)&(test.jx_fico_score<=536)&(test.下单日期>='2024-07-01')&(test.归属渠道!='抖音渠道')&(test.下单日期<='2025-01-31')].shape
# test[(test.jx_fico_score >= 0)&(test.jx_fico_score<=536)&(test.下单日期>='2024-07-01')&(test.归属渠道!='抖音渠道')&(test.下单日期<='2025-01-31')].归属渠道.value_counts()
# 453
# test[(test.jx_fico_score >= 0)&(test.jx_fico_score<=536)&(test.下单日期>='2024-07-01')&(test.归属渠道!='抖音渠道')&(test.下单日期<='2025-01-31')].bj_qy_score.min()
# 巨量回捞
# test[(test.jx_fico_score >= 0)&(test.jx_fico_score<489.5)&(test.下单日期>='2024-07-01')&(test.归属渠道!='抖音渠道')&(test.下单日期<='2025-01-31')].shape

In [195]:
# （9-3-1）统计-极信FICO
# conditions_1 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
# conditions_1 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_1 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

view_m = '2024-07'
# view_m = '2025-01'
conditions_2 = (df_analy_ks.search_time == view_m)
# conditions_2 = ((df_analy_ks.下单日期 >= '2024-02-01') & (df_analy_ks.下单日期 <= '2024-06-30'))

pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='FICO_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='FICO_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='FICO_Bin',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='FICO_Bin',columns='y_credit',values='order_number',aggfunc='count')



y_fraud_fpd7,0.0,1.0
FICO_Bin,,
"2.(489,536]",5.0,1.0
"3.(536,563]",4.0,2.0
"4.(563,587]",10.0,NaN
"5.(587,617]",7.0,NaN
"6.(617,632]",5.0,NaN
"7.(632,672]",19.0,1.0
"8.(672,inf)",9.0,NaN
9.-999,6.0,NaN


y_fraud_f3pd7,0.0,1.0
FICO_Bin,,
"2.(489,536]",5.0,1.0
"3.(536,563]",4.0,2.0
"4.(563,587]",10.0,NaN
"5.(587,617]",6.0,1.0
"6.(617,632]",5.0,NaN
"7.(632,672]",19.0,1.0
"8.(672,inf)",9.0,NaN
9.-999,6.0,NaN


y_credit_mob3pd30,0.0,1.0
FICO_Bin,,
"2.(489,536]",5.0,1.0
"3.(536,563]",4.0,2.0
"4.(563,587]",10.0,NaN
"5.(587,617]",6.0,1.0
"6.(617,632]",5.0,NaN
"7.(632,672]",19.0,1.0
"8.(672,inf)",9.0,NaN
9.-999,6.0,NaN


y_credit,0.0,1.0
FICO_Bin,,
"2.(489,536]",4.0,2.0
"3.(536,563]",4.0,2.0
"4.(563,587]",10.0,NaN
"5.(587,617]",5.0,2.0
"6.(617,632]",5.0,NaN
"7.(632,672]",16.0,4.0
"8.(672,inf)",8.0,1.0
9.-999,5.0,1.0


In [ ]:
# （9-3-2）统计-规则42
# conditions_1 = (df_analy_ks.归属渠道 != '抖音渠道')
# 0 为青云分豁免或回捞的单（有青云分<450的，但集中在2024.4及之前，17单；有jx_fico_score<489.5，大部分为回捞的单）,1表示强拒，2 表示命中但40%随机豁免，3 表示通过，4 表示通过且 jx_fico_score > 559，5 为jx_fico_score缺失
# 2为206个，即豁免了206个订单；1的为14个，2024.7.12及之前（2025.2有1单拒量回捞）
# df_analy_ks[conditions_1].规则42.value_counts()
# df_analy_ks[conditions_1&(df_analy_ks.规则42 == 0)&(df_analy_ks.jx_fico_score<489.5)][['order_number','下单日期','归属渠道','拒绝理由','model_score_yl','bj_qy_score','jx_fico_score','规则42']]
# df_analy_ks[conditions_1 & (df_analy_ks.规则42 == 1)][['order_number','下单日期','归属渠道','拒绝理由','model_score_yl','bj_qy_score','jx_fico_score','规则42']]
# df_analy_ks[conditions_1 & (df_analy_ks.规则42 == 2)].bj_qy_score.min()
# df_analy_ks[conditions_1 & (df_analy_ks.规则42 == 2)].model_score_yl.min()
# df_analy_ks[(df_analy_ks.jx_fico_score > 537)&(df_analy_ks.jx_fico_score <= 559)&(df_analy_ks.规则42 == 3)].model_score_yl.max()
# df_analy_ks[conditions_1&(df_analy_ks.规则42 == 3)&(df_analy_ks.jx_fico_score != -999)&(df_analy_ks.jx_fico_score <= 559)][['order_number','下单日期','归属渠道','拒绝理由','model_score_yl','bj_qy_score','jx_fico_score','规则42']].tail()
# df_analy_ks[conditions_1&(df_analy_ks.规则42 == 3)&(df_analy_ks.bj_qy_score< 450)].shape
# df_analy_ks[conditions_1&(df_analy_ks.规则42 == 3)&(df_analy_ks.bj_qy_score>= 450)].shape
# df_analy_ks[conditions_1&(df_analy_ks.规则42 == 4)&(df_analy_ks.jx_fico_score>= 0)].jx_fico_score.min()

# 各个客群的风险表现
# conditions_1 = (df_train_31.归属渠道 != '抖音渠道')
# pd.pivot_table(df_train_31[conditions_1],index='规则42',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_train_31[conditions_1],index='规则42',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_train_31[conditions_1],index='规则42',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_train_31[conditions_1],index='规则42',columns='y_credit',values='order_number',aggfunc='count')

view_m = '2024-07'
conditions_1 = (df_test_31.归属渠道 != '抖音渠道')
conditions_2 = (df_test_31.search_time == view_m)
pd.pivot_table(df_test_31[conditions_1 & conditions_2],index='规则42',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_test_31[conditions_1 & conditions_2],index='规则42',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
pd.pivot_table(df_test_31[conditions_1 & conditions_2],index='规则42',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
pd.pivot_table(df_test_31[conditions_1 & conditions_2],index='规则42',columns='y_credit',values='order_number',aggfunc='count')

In [ ]:
# （9-3-3）统计-规则43
# "芝麻租物渠道： 1. 腾云212>98拒绝；
# 2. 履约保护>26拒绝；（关系为“或”，命中其中一个拒绝）

# 搜索渠道： 1. 腾云212>98拒绝；
# 2. 履约保护>26拒绝；（关系为“或”，命中其中一个拒绝）

# 其他渠道的： 1. 腾云212>95拒绝;
# 2. 履约保护>25拒绝; （关系为“或”，命中其中一个拒绝）"


# nsf_rental_v1_3_hundred_Bin
# td212_score_Bin
# 数据表现分组较集中
# continous_merged_dict
# {'td212_score': [-1111, -999, 35.12],
#  'nsf_rental_v1_3_hundred': [-1111, -999, 7.0, 12.0, 25.0]}

view_m = '2025-01'
conditions_1 = (df_analy_ks.归属渠道 == '芝麻租物')
# conditions_1 = (df_analy_ks.归属渠道 == '搜索渠道')
# conditions_1 = (~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道']))
conditions_2 = (df_analy_ks.search_time == view_m)
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='履约保护_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='履约保护_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='履约保护_Bin',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='履约保护_Bin',columns='y_credit',values='order_number',aggfunc='count')

# view_m = '2025-01'
# # conditions_1 = (df_analy_ks.归属渠道 == '芝麻租物')
# # conditions_1 = (df_analy_ks.归属渠道 == '搜索渠道')
# conditions_1 = (~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道']))
# conditions_2 = (df_analy_ks.search_time == view_m)
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='td212_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='td212_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='td212_Bin',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='td212_Bin',columns='y_credit',values='order_number',aggfunc='count')


In [ ]:
# （9-3-4）统计-规则37-39
# 数控风险等级 zm_jh_t0_level
# # conditions_1 = (df_analy_ks.归属渠道 == '芝麻租物')
# # conditions_1 = (df_analy_ks.归属渠道 == '搜索渠道')
# conditions_1 = (~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道']))
# # conditions_2 = (df_analy_ks.search_time >= '2023-10') & (df_analy_ks.search_time <= '2023-12')
# conditions_2 = (df_analy_ks.search_time >= '2024-01') & (df_analy_ks.search_time <= '2024-06')

# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
# pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_credit',values='order_number',aggfunc='count')

view_m = '2024-07'
# conditions_1 = (df_analy_ks.归属渠道 == '芝麻租物')
# conditions_1 = (df_analy_ks.归属渠道 == '搜索渠道')
conditions_1 = (~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道']))
conditions_2 = (df_analy_ks.search_time == view_m)
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_ks[conditions_1 & conditions_2],index='zm_jh_t0_level_Bin',columns='y_credit',values='order_number',aggfunc='count')

# 10.稳定性分析

## 10_1.前置拦截

In [272]:
# （1-1）
# 目前仅火谋分
# 观察2024.10及之后的
# 基于202410的申请分组
# 申请 & 出库
%run C:/Users/hxq/Desktop/mine/1_参考/1_code/湖南银行poc/2.model_func.ipynb
conditions_1 = (df_risk_new.下单日期 >= '2024-10-01')
var_bin_qz = ['bj_hm_score']
df_psi_qz = df_risk_new[conditions_1][['order_number','下单日期','search_time','归属渠道','审核状态','id_card_num','user_mobile']].merge(df_out_data[['order_number'] + var_bin_qz], on = 'order_number', how = 'left')
df_analy_ks['是否出库'] = 1
df_psi_qz = df_psi_qz.merge(df_analy_ks[['order_number','是否出库','是否拒量']], on = 'order_number', how = 'left')
df_psi_qz = df_psi_qz.drop_duplicates(subset = 'order_number')

df_psi_qz = df_psi_qz.sort_values(by = '是否出库', ascending = False)
df_psi_qz = df_psi_qz.drop_duplicates(subset = ['id_card_num','user_mobile','下单日期'],  keep='first')
df_psi_qz.shape
# df_psi_qz.是否出库.value_counts()

for col in var_bin_qz:
    df_psi_qz[col] = df_psi_qz[col].fillna(-999)
    df_psi_qz[col] = np.where(df_psi_qz[col] <= 0, -999, df_psi_qz[col])
df_psi_qz.shape


# （1-2）分组
df_bin_qz = df_psi_qz[(df_psi_qz.下单日期 >= '2024-10-01') & (df_psi_qz.下单日期 <= '2024-10-31')]
df_bin_qz.shape
train, test = train_test_split(df_bin_qz, test_size = 0.2, random_state = 0)
train['target'] = 1
test['target'] = 0
df_bin_qz = pd.concat([train, test])

df_bin_qz.shape
df_bin_qz[:1]


cutOff = df_qcut_me(df_bin_qz, 'bj_hm_score', 'target', max_interval=10, special_attribute = [-1111, -999])
# cutOff

# （1-3）根据业务应用做相应的调整
cutOff[2] = 449
cutOff[3] = 475
cutOff[4] = 499
# cutOff
# df_analy_ks[:1]
# [-1111, -999, 496.0, 515.0, 529.0, 543.0, 558.0, 574.0, 591.0, 610.0, 637.0] -- 基于进件 202501
# [-1111, -999, 439.0, 469.0, 492.0, 512.0, 531.0, 549.0, 570.0, 593.0, 622.0] -- 基于申请 202501
# [-1111, -999, 442.0, 473.0, 496.0, 516.0, 534.0, 553.0, 573.0, 595.0, 624.0] -- 基于申请 202410
# [-1111, -999, 446.0, 478.0, 502.0, 520.0, 538.0, 556.0, 575.0, 596.0, 625.0] -- 基于申请 202410，身份证+手机号+申请日期去重
# [-1111, -999, 439.0, 469.0, 491.0, 512.0, 531.0, 549.0, 570.0, 592.0, 622.0] -- 基于申请 202411

# （1-4）映射
var_bin_list_qz = []
df = df_psi_qz.copy()
for col in var_bin_qz:
    # print ("{} is in processing".format(col))
    if -1111 not in set(df[col]) and -999 not in set(df[col]): 
        # cutOff = df_qcut_me(df, col, 'target', max_interval=10, special_attribute=[])
        df[col+'_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[]))
        # del df[col]
        # var_bin_list_qz.append(col)
    else:
        # cutOff = df_qcut_me(df, col, 'target', max_interval = 12, special_attribute = [-1111, -999])
        df[col + '_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[-1111, -999])) 
        # del df[col]
        # var_bin_list_qz.append(col)
# 火谋分
df['bj_hm_score_Bin'] = df.apply(lambda x : '1.[0,449]'  if x.bj_hm_score >= 0 and x.bj_hm_score <= 449
                                                  else '2.(449,475.0]' if x.bj_hm_score  > 449 and x.bj_hm_score  <= 475
                                                  else '3.(475,499.0]' if x.bj_hm_score  > 475 and x.bj_hm_score  <= 499
                                                  else '4.(499,520.0]' if x.bj_hm_score  > 499 and x.bj_hm_score  <= 520
                                                  else '5.(520,538.0]' if x.bj_hm_score  > 520 and x.bj_hm_score  <= 538
                                                  else '6.(538,556.0]' if x.bj_hm_score  > 538 and x.bj_hm_score  <= 556
                                                  else '7.(556,575.0]' if x.bj_hm_score  > 556 and x.bj_hm_score  <= 575
                                                  else '8.(575,596.0]' if x.bj_hm_score  > 575 and x.bj_hm_score  <= 596
                                                  else '9.(596,625.0]' if x.bj_hm_score  > 596 and x.bj_hm_score  <= 625
                                                  else '91.(625,inf)' if x.bj_hm_score  > 625
                                                  else '92.-999', axis = 1)

# （1-5）统计
# df.审核状态 == '出库' 有可能为拒量商家的出库
# conditions_2 = (df.审核状态 == '出库') & (df.是否出库 == 1)  & (df.是否拒量 == 0)
conditions_2 = (df.是否出库 == 1)
conditions_3 = ((~df.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df.是否拒量 == 0))
pd.pivot_table(df, index='bj_hm_score_Bin', columns='search_time', values='order_number',aggfunc='count')
pd.pivot_table(df[conditions_2 & conditions_3], index='bj_hm_score_Bin', columns='search_time', values='order_number',aggfunc='count')

# 数据检查 
# df.bj_hm_score_Bin.value_counts()
# df_psi_qz[(df_psi_qz.bj_hm_score > 516) & (df_psi_qz.bj_hm_score <= 534)].shape
# df_psi_qz['是否出库'] = df_psi_qz['是否出库'].fillna(0)
# pd.pivot_table(df_psi_qz,index='审核状态',columns='是否出库',values='order_number',aggfunc='count')
# df_psi_qz[(df_psi_qz.是否出库 == 1) & (df_psi_qz.审核状态 == '人审拒绝')]
# df_psi_qz[(df_psi_qz.是否出库 == 1) & (df_psi_qz.审核状态 == '客户取消')]
# df_psi_qz[(df_psi_qz.是否出库 == 1) & (df_psi_qz.审核状态 == '已退款')]
# df_psi_qz[(df_psi_qz.是否出库 == 0) & (df_psi_qz.审核状态 == '出库')]
# df_psi_qz[(df_psi_qz.是否出库 == 1) & (df_psi_qz.审核状态.isin(['客户取消','已退款','人审拒绝']))].search_time.value_counts()

(797356, 10)

(797356, 10)

(82716, 10)

(82716, 11)

,order_number,下单日期,search_time,归属渠道,审核状态,id_card_num,user_mobile,bj_hm_score,是否出库,是否拒量,target
43278,A20241014155124351,2024-10-14,2024-10,九州信息,前置拦截,530421198711091919,18908774750,489.0,NaN,NaN,1


search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
bj_hm_score_Bin,,,,,,,,
"1.[0,449]",6756,7218,8506,5911,6180,8774,10797,13544
"2.(449,475.0]",5058,5419,6301,4296,4797,6564,8060,9730
"3.(475,499.0]",5979,6314,7722,5271,5688,7815,9625,11371
"4.(499,520.0]",6760,6965,8321,5842,6051,8423,10323,11966
"5.(520,538.0]",6421,6553,7976,5586,6067,8051,9730,11538
"6.(538,556.0]",6042,6213,7489,5301,5652,7395,9253,10777
"7.(556,575.0]",5918,6070,7293,5073,5343,7289,8685,10778
"8.(575,596.0]",6148,6169,7354,5133,5587,7673,9279,11257
"9.(596,625.0]",6196,6266,7312,5203,5541,7671,9666,11708


search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
bj_hm_score_Bin,,,,,,,,
"1.[0,449]",29,54,56,24,32,46,52,92
"2.(449,475.0]",42,72,66,31,45,55,202,235
"3.(475,499.0]",48,81,100,45,48,71,245,324
"4.(499,520.0]",177,272,339,189,171,237,317,357
"5.(520,538.0]",185,299,364,187,208,286,362,342
"6.(538,556.0]",205,295,387,206,200,270,310,372
"7.(556,575.0]",200,318,358,195,207,248,343,373
"8.(575,596.0]",194,287,369,208,198,253,341,351
"9.(596,625.0]",189,303,377,199,211,271,392,433


## 10_2.机审

### 10_21.机审1

In [337]:
# （1）宽表整合
# 观察2024.10及之后的
# 基于202410的免押进件分组
# 进件 & 出库

conditions_1 = (df_info_ori.下单日期 >= '2024-10-01')
conditions_2 = (~df_info_ori.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))
# var_bin_sj_str = []
df_psi_js = df_info_ori[conditions_1 & conditions_2]
df_analy_ks['是否出库'] = 1
df_psi_js = df_psi_js.merge(df_analy_ks[['order_number','是否出库']], on = 'order_number', how = 'left')
df_psi_js = df_psi_js.drop_duplicates(subset = 'order_number')

df_psi_js = df_psi_js.sort_values(by = '是否出库', ascending = False)
df_psi_js = df_psi_js.drop_duplicates(subset = ['id_card_num','user_mobile','下单日期'],  keep='first')
df_psi_js.shape

(159985, 189)

In [233]:
# （2）分类变量
conditions_3 = (df_psi_js.是否出库 == 1)
conditions_4 = ((~df_psi_js.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_psi_js.是否拒量 == 0))
pd.pivot_table(df_psi_js, index='grade_Bin', columns='search_time', values='order_number',aggfunc='count')
pd.pivot_table(df_psi_js[conditions_3 & conditions_4], index='grade_Bin', columns='search_time', values='order_number',aggfunc='count')

# 履约历史等级 rentOnlineRiskLevelFulfill_Bin
# 综合风险等级 rentOnlineRiskLevelOverall_Bin
# 细化综合风险等级 rentOnlineDetailLevelOverall_Bin
# 评分等级 grade_Bin

search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
grade_Bin,,,,,,,,
1.F,1401,1314,1421,1073,1278,1923,2851,2868
2.E,260,234,334,271,284,442,631,643
3.D,3138,2943,3407,2494,3241,4549,6828,7733
4.C,3310,3239,3975,3185,3659,4908,7299,8329
5.B,3069,3106,4190,2988,3474,4515,6349,7443
6.A,996,1190,1536,1081,1241,1470,2010,2269
7.S,1407,1496,2099,1403,1485,1778,2290,2467
8.-999,33,2001,3335,2575,420,45,62,697


search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
grade_Bin,,,,,,,,
3.D,114.0,156.0,156.0,84.0,106.0,168.0,266.0,319.0
4.C,321.0,523.0,546.0,312.0,297.0,463.0,790.0,870.0
5.B,486.0,732.0,936.0,521.0,499.0,750.0,1096.0,1137.0
6.A,234.0,419.0,542.0,271.0,305.0,385.0,547.0,557.0
7.S,451.0,651.0,901.0,484.0,456.0,544.0,777.0,783.0
8.-999,1.0,16.0,19.0,1.0,NaN,1.0,NaN,NaN


In [340]:
# （3）连续变量
# 用2024.10 的进件等频分组
# 进件 & 出库
# df_psi_js = df_psi_js[df_psi_js.下单日期 >= '2025-03-01']


df_psi_js['jx_fico_score'] = df_psi_js['jx_fico_score'].fillna(-999)
df_psi_js['jx_fico_score'] = np.where(df_psi_js.jx_fico_score <= 0, -999, df_psi_js.jx_fico_score)
df_bin_js = df_psi_js[(df_psi_js.下单日期 >= '2024-10-01') & (df_psi_js.下单日期 <= '2024-10-31')]

df_bin_js.shape
train, test = train_test_split(df_bin_js, test_size = 0.2, random_state = 0)
train['target'] = 1
test['target'] = 0
df_bin_js = pd.concat([train, test])

df_bin_js.shape
# df_bin_js[:1]


cutOff = df_qcut_me(df_bin_js, 'jx_fico_score', 'target', max_interval=10, special_attribute = [-1111, -999])
cutOff


# （1-3）根据业务应用做相应的调整
# 蚁盾分1.0
# cutOff[10] = 70
# cutOff
# cutOff[5] = 4.5
# cutOff[6] = 10.5
# cutOff[8] = 0.95

# # （1-4）映射
var_bin_list_js = []
df = df_psi_js.copy()
var_bin_js = ['jx_fico_score']
for col in var_bin_js:
    # print ("{} is in processing".format(col))
    if -1111 not in set(df[col]) and -999 not in set(df[col]): 
        # cutOff = df_qcut_me(df, col, 'target', max_interval=10, special_attribute=[])
        df[col+'_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[]))
        # del df[col]
        # var_bin_list_js.append(col)
    else:
        # cutOff = df_qcut_me(df, col, 'target', max_interval = 12, special_attribute = [-1111, -999])
        df[col + '_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[-1111, -999])) 
        # del df[col]
        # var_bin_list_js.append(col)
    
df['jx_fico_score_Bin'] = df.apply(lambda x : '1.[0,489.5]' if x.jx_fico_score >= 0 and x.jx_fico_score <= 489.5 
                                                  else '2.(489.5 ,537]' if x.jx_fico_score > 489.5  and x.jx_fico_score <= 537
                                                  else '3.(537,559]' if x.jx_fico_score > 537 and x.jx_fico_score <= 559
                                                  else '4.(559,588]' if x.jx_fico_score > 559 and x.jx_fico_score <= 588
                                                  else '5.(588,604.5]' if x.jx_fico_score > 588 and x.jx_fico_score <= 604.5
                                                  else '6.(604.5,622]' if x.jx_fico_score > 604.5 and x.jx_fico_score <= 622
                                                  else '7.(622,642]' if x.jx_fico_score > 622 and x.jx_fico_score <= 642
                                                  else '8.(642,664]' if x.jx_fico_score > 642 and x.jx_fico_score <= 664
                                                  else '9.(664,692]' if x.jx_fico_score > 664 and x.jx_fico_score <= 692
                                                  else '91.(692,inf)' if x.jx_fico_score > 692
                                                  else '92.-999', axis = 1)

# （1-5）统计
# df.审核状态 == '出库' 有可能为拒量商家的出库
# conditions_3 = (df.审核状态 == '出库') & (df.是否出库 == 1)  & (df.是否拒量 == 0)
conditions_3 = (df.是否出库 == 1)
conditions_4 = ((~df.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df.是否拒量 == 0))
pd.pivot_table(df, index='jx_fico_score_Bin', columns='search_time', values='order_number',aggfunc='count')
pd.pivot_table(df[conditions_3 & conditions_4], index='jx_fico_score_Bin', columns='search_time', values='order_number',aggfunc='count')


(13614, 189)

(13614, 190)

[-1111, -999, 516.0, 547.0, 570.0, 588.0, 605.0, 622.0, 642.0, 664.0, 692.0]

search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
jx_fico_score_Bin,,,,,,,,
"1.[0,489.5]",283,180,299,297,229,293,542,674
"2.(489.5 ,537]",658,618,783,735,781,955,1559,1562
"3.(537,559]",538,510,730,544,619,836,1318,1383
"4.(559,588]",856,936,1306,932,1094,1487,2258,2479
"5.(588,604.5]",521,590,758,549,674,902,1400,1772
"6.(604.5,622]",630,658,898,633,810,1140,1633,1982
"7.(622,642]",588,743,959,673,797,1132,1697,2185
"8.(642,664]",593,671,901,612,748,1061,1535,2068
"9.(664,692]",557,650,898,638,708,1006,1406,1810


search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
jx_fico_score_Bin,,,,,,,,
"1.[0,489.5]",NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
"2.(489.5 ,537]",17.0,19.0,26.0,27.0,14.0,30.0,48.0,21.0
"3.(537,559]",41.0,40.0,70.0,34.0,12.0,20.0,50.0,36.0
"4.(559,588]",268.0,395.0,533.0,280.0,275.0,387.0,563.0,493.0
"5.(588,604.5]",194.0,285.0,337.0,171.0,188.0,226.0,400.0,390.0
"6.(604.5,622]",229.0,330.0,416.0,229.0,243.0,347.0,483.0,510.0
"7.(622,642]",231.0,374.0,426.0,233.0,233.0,354.0,536.0,580.0
"8.(642,664]",216.0,351.0,441.0,232.0,250.0,336.0,501.0,546.0
"9.(664,692]",200.0,350.0,414.0,246.0,213.0,299.0,448.0,523.0


In [ ]:
# 蚁盾分1.0
df_info_ori['my_zl_score_Bin'] = df_info_ori.apply(lambda x: '91.[0,10]' if x.my_zl_score >=0 and x.my_zl_score <= 10
                                               else '9.(10,20]' if x.my_zl_score > 10 and x.my_zl_score  <= 20
                                               else '8.(20,30]' if x.my_zl_score > 20 and x.my_zl_score  <= 30
                                               else '7.(30,40]' if x.my_zl_score > 30 and x.my_zl_score  <= 40
                                               else '6.(40,50]' if x.my_zl_score > 40 and x.my_zl_score  <= 50
                                               else '5.(50,60]' if x.my_zl_score > 50 and x.my_zl_score  <= 60
                                               else '4.(60,70]' if x.my_zl_score > 60 and x.my_zl_score  <= 70
                                               else '3.(70,80]' if x.my_zl_score > 70 and x.my_zl_score  <= 80
                                               else '2.(80,90]' if x.my_zl_score > 80 and x.my_zl_score  <= 90
                                               else '1.(90,inf)' if x.my_zl_score > 90
                                               else '92.-999' ,axis = 1)

# 蚁盾分2.0
df_info_ori['my_zl_score2_Bin'] = df_info_ori.apply(lambda x: '91.[0,50]' if x.my_zl_score2 >=0 and x.my_zl_score2 <= 50
                                               else '9.(50,53]' if x.my_zl_score2 > 50 and x.my_zl_score2  <= 53
                                               else '8.(53,56]' if x.my_zl_score2 > 53 and x.my_zl_score2  <= 56
                                               else '7.(56,58]' if x.my_zl_score2 > 56 and x.my_zl_score2  <= 58
                                               else '6.(58,60]' if x.my_zl_score2 > 58 and x.my_zl_score2  <= 60
                                               else '5.(60,62.5]' if x.my_zl_score2 > 60 and x.my_zl_score2  <= 62.5
                                               else '4.(62.5,65]' if x.my_zl_score2 > 62.5 and x.my_zl_score2  <= 65
                                               else '3.(65,70]' if x.my_zl_score2 > 65 and x.my_zl_score2  <= 70
                                               else '2.(70,80]' if x.my_zl_score2 > 70 and x.my_zl_score2  <= 80
                                               else '1.(80,inf)' if x.my_zl_score2 > 80
                                               else '92.-999' ,axis = 1)

# 云商分
df_info_ori['br_ys_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,450.0]'  if x.br_ys_score >= 0 and x.br_ys_score <= 450 
                                                  else '2.(450,475.0]' if x.br_ys_score  > 450 and x.br_ys_score  <= 475
                                                  else '3.(475,495.0]' if x.br_ys_score  > 475 and x.br_ys_score  <= 495
                                                  else '4.(495,514.0]' if x.br_ys_score  > 495 and x.br_ys_score  <= 514
                                                  else '5.(514,540.0]' if x.br_ys_score  > 514 and x.br_ys_score  <= 540
                                                  else '6.(540,565.0]' if x.br_ys_score  > 540 and x.br_ys_score  <= 565
                                                  else '7.(565,620.0]' if x.br_ys_score  > 565 and x.br_ys_score  <= 620
                                                  else '8.(620,685.0]' if x.br_ys_score  > 620 and x.br_ys_score  <= 685
                                                  else '9.(685,742.0]' if x.br_ys_score  > 685 and x.br_ys_score  <= 742
                                                  else '91.(742,inf)' if x.br_ys_score  > 742
                                                  else '92.-999', axis = 1)

# 融安分
df_info_ori['br_ra_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,680.0]'  if x.br_ra_score >= 0 and x.br_ra_score <= 680 
                                                  else '2.(680,700.0]' if x.br_ra_score  > 680 and x.br_ra_score  <= 700
                                                  else '3.(700,713.0]' if x.br_ra_score  > 700 and x.br_ra_score  <= 713
                                                  else '4.(713,726.0]' if x.br_ra_score  > 713 and x.br_ra_score  <= 726
                                                  else '5.(726,740.0]' if x.br_ra_score  > 726 and x.br_ra_score  <= 740
                                                  else '6.(740,755.0]' if x.br_ra_score  > 740 and x.br_ra_score  <= 755
                                                  else '7.(755,771.0]' if x.br_ra_score  > 755 and x.br_ra_score  <= 771
                                                  else '8.(771,801.0]' if x.br_ra_score  > 771 and x.br_ra_score  <= 801
                                                  else '9.(801,854.0]' if x.br_ra_score  > 801 and x.br_ra_score  <= 854
                                                  else '91.(854,inf)' if x.br_ra_score  > 854
                                                  else '92.-999', axis = 1)

# 青云
df_info_ori['bj_qy_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,433.5]'  if x.bj_qy_score >= 0 and x.bj_qy_score <= 433.5 
                                                  else '2.(433.5,477.0]' if x.bj_qy_score  > 433.5 and x.bj_qy_score  <= 477
                                                  else '3.(477,499.0]' if x.bj_qy_score  > 477 and x.bj_qy_score  <= 499
                                                  else '4.(499,516.0]' if x.bj_qy_score  > 499 and x.bj_qy_score  <= 516
                                                  else '5.(516,533.0]' if x.bj_qy_score  > 516 and x.bj_qy_score  <= 533
                                                  else '6.(533,551.0]' if x.bj_qy_score  > 533 and x.bj_qy_score  <= 551
                                                  else '7.(551,570.0]' if x.bj_qy_score  > 551 and x.bj_qy_score  <= 570
                                                  else '8.(570,594.0]' if x.bj_qy_score  > 570 and x.bj_qy_score  <= 594
                                                  else '9.(594,627.0]' if x.bj_qy_score  > 594 and x.bj_qy_score  <= 627
                                                  else '91.(627,inf)' if x.bj_qy_score  > 627
                                                  else '92.-999', axis = 1)

# jx_yl_UPPC134   近1个月出账类交易频度
df_info_ori['jx_yl_UPPC134_Bin'] = df_info_ori.apply(lambda x : '1.[0,1.5]'  if x.jx_yl_UPPC134 >= 0 and x.jx_yl_UPPC134 <= 1.5
                                                  else '2.(1.5,3.5]' if x.jx_yl_UPPC134  > 1.5 and x.jx_yl_UPPC134  <= 3.5
                                                  else '3.(3.5,6.5]' if x.jx_yl_UPPC134  > 3.5 and x.jx_yl_UPPC134  <= 6.5
                                                  else '4.(6.5,10.0]' if x.jx_yl_UPPC134  > 6.5 and x.jx_yl_UPPC134  <= 10
                                                  else '5.(10,15.0]' if x.jx_yl_UPPC134  > 10 and x.jx_yl_UPPC134  <= 15
                                                  else '6.(15,21.0]' if x.jx_yl_UPPC134  > 15 and x.jx_yl_UPPC134  <= 21
                                                  else '7.(21,28.0]' if x.jx_yl_UPPC134  > 21 and x.jx_yl_UPPC134  <= 28
                                                  else '8.(28,38.0]' if x.jx_yl_UPPC134  > 28 and x.jx_yl_UPPC134  <= 38
                                                  else '9.(38,56.0]' if x.jx_yl_UPPC134  > 38 and x.jx_yl_UPPC134  <= 56
                                                  else '91.(56,inf)' if x.jx_yl_UPPC134  > 56
                                                  else '92.-999', axis = 1)
# jx_yl_UPPD007	近6个月失败交易频度
df_info_ori['jx_yl_UPPD007_Bin'] = df_info_ori.apply(lambda x : '91.[0,5]'  if x.jx_yl_UPPD007 >= 0 and x.jx_yl_UPPD007 <= 5
                                                  else '9.(5,11]' if x.jx_yl_UPPD007  > 5 and x.jx_yl_UPPD007  <= 11
                                                  else '8.(11,17]' if x.jx_yl_UPPD007  > 11 and x.jx_yl_UPPD007  <= 17
                                                  else '7.(17,26.0]' if x.jx_yl_UPPD007  > 17 and x.jx_yl_UPPD007  <= 26
                                                  else '6.(26,38.0]' if x.jx_yl_UPPD007  > 26 and x.jx_yl_UPPD007  <= 38
                                                  else '5.(38,47.0]' if x.jx_yl_UPPD007  > 38 and x.jx_yl_UPPD007  <= 47
                                                  else '4.(47,77.0]' if x.jx_yl_UPPD007  > 47 and x.jx_yl_UPPD007  <= 77
                                                  else '3.(77,125.0]' if x.jx_yl_UPPD007  > 77 and x.jx_yl_UPPD007  <= 125
                                                  else '2.(125,236.0]' if x.jx_yl_UPPD007  > 125 and x.jx_yl_UPPD007  <= 236
                                                  else '1.(236,inf)' if x.jx_yl_UPPD007  > 236
                                                  else '92.-999', axis = 1)

# jx_yl_UPPE074	近12个月平均代扣失败天数
df_info_ori['jx_yl_UPPE074_Bin'] = df_info_ori.apply(lambda x : '6.[0,0.82]'  if x.jx_yl_UPPE074 >= 0 and x.jx_yl_UPPE074 <= 0.82
                                                  else '5.(0.82,1)' if x.jx_yl_UPPE074 > 0.82 and x.jx_yl_UPPE074 < 1
                                                  else '4.1' if x.jx_yl_UPPE074 == 1
                                                  else '3.(1,1.345]' if x.jx_yl_UPPE074 > 1 and x.jx_yl_UPPE074 <= 1.345
                                                  else '2.(1.345,2.31]' if x.jx_yl_UPPE074 > 1.345 and x.jx_yl_UPPE074 <= 2.31
                                                  else '1.(2.31,inf)' if x.jx_yl_UPPE074 > 2.31
                                                  else '8.-999', axis = 1)

# jx_yl_UPPE082	近12个月成功扣款频度占比
df_info_ori['jx_yl_UPPE082_Bin'] = df_info_ori.apply(lambda x : '1.[0,0.15]'  if x.jx_yl_UPPE082 >= 0 and x.jx_yl_UPPE082 <= 0.15
                                                  else '2.(0.15,0.3]' if x.jx_yl_UPPE082  > 0.15 and x.jx_yl_UPPE082  <= 0.3
                                                  else '3.(0.3,0.45]' if x.jx_yl_UPPE082  > 0.3 and x.jx_yl_UPPE082  <= 0.45
                                                  else '4.(0.45,0.55]' if x.jx_yl_UPPE082  > 0.45 and x.jx_yl_UPPE082  <= 0.55
                                                  else '5.(0.55,0.655]' if x.jx_yl_UPPE082  > 0.55 and x.jx_yl_UPPE082  <= 0.655
                                                  else '6.(0.655,0.81]' if x.jx_yl_UPPE082  > 0.655 and x.jx_yl_UPPE082  <= 0.81
                                                  else '7.(0.81,0.95]' if x.jx_yl_UPPE082  > 0.81 and x.jx_yl_UPPE082  <= 0.95
                                                  else '8.(0.95,inf)' if x.jx_yl_UPPE082  > 0.95
                                                  else '9.-999', axis = 1)

# xy_qj_C22180008	消⾦贷款类产品数
df_info_ori['xy_qj_C22180008_Bin'] = df_info_ori.apply(lambda x : '7.[0,2]' if x.xy_qj_C22180008 >= 0 and x.xy_qj_C22180008 <= 2
                                                  else '6.(2,5]' if x.xy_qj_C22180008 > 2 and x.xy_qj_C22180008 <= 5
                                                  else '5.(5,10.5]' if x.xy_qj_C22180008 > 5 and x.xy_qj_C22180008 <= 10.5
                                                  else '4.(10.5,15]' if x.xy_qj_C22180008 > 10.5 and x.xy_qj_C22180008 <= 15
                                                  else '3.(15,23]' if x.xy_qj_C22180008 > 15 and x.xy_qj_C22180008 <= 23
                                                  else '2.(23,36]' if x.xy_qj_C22180008 > 23 and x.xy_qj_C22180008 <= 36
                                                  else '1.(36,inf)' if x.xy_qj_C22180008 > 36
                                                  else '8.-999', axis = 1)

# xy_qj_B22170035	近1个⽉失败扣款笔数
df_info_ori['xy_qj_B22170035_Bin'] = df_info_ori.apply(lambda x : '6.[0,1]' if x.xy_qj_B22170035 >= 0 and x.xy_qj_B22170035 <= 1
                                                  else '5.(1,2]' if x.xy_qj_B22170035 > 1 and x.xy_qj_B22170035 <= 2
                                                  else '4.(2,4.5]' if x.xy_qj_B22170035 > 2 and x.xy_qj_B22170035 <= 4.5
                                                  else '3.(4.5,7]' if x.xy_qj_B22170035 > 4.5 and x.xy_qj_B22170035 <= 7
                                                  else '2.(7,15]' if x.xy_qj_B22170035 > 7 and x.xy_qj_B22170035 <= 15
                                                  else '1.(15,inf)' if x.xy_qj_B22170035 > 15
                                                  else '7.-999', axis = 1)

# xy_qj_B22170036	近3个⽉失败扣款笔数
df_info_ori['xy_qj_B22170036_Bin'] = df_info_ori.apply(lambda x : '7.[0,1]' if x.xy_qj_B22170036 >= 0 and x.xy_qj_B22170036 <= 1
                                                  else '6.(1,2]' if x.xy_qj_B22170036 > 1 and x.xy_qj_B22170036 <= 2
                                                  else '5.(2,4]' if x.xy_qj_B22170036 > 2 and x.xy_qj_B22170036 <= 4
                                                  else '4.(4,8.5]' if x.xy_qj_B22170036 > 4 and x.xy_qj_B22170036 <= 8.5
                                                  else '3.(8.5,16]' if x.xy_qj_B22170036 > 8.5 and x.xy_qj_B22170036 <= 16
                                                  else '2.(16,31]' if x.xy_qj_B22170036 > 16 and x.xy_qj_B22170036 <= 31
                                                  else '1.(31,inf)' if x.xy_qj_B22170036 > 31
                                                  else '91.-999', axis = 1)
# xy_qj_A22160001	申请准⼊分
df_info_ori['xy_qj_A22160001_Bin'] = df_info_ori.apply(lambda x : '1.[0,477]' if x.xy_qj_A22160001 >= 0 and x.xy_qj_A22160001 <= 477
                                                  else '2.(477,501]' if x.xy_qj_A22160001 > 477 and x.xy_qj_A22160001 <= 501
                                                  else '3.(501,520]' if x.xy_qj_A22160001 > 501 and x.xy_qj_A22160001 <= 520
                                                  else '4.(520,534]' if x.xy_qj_A22160001 > 520 and x.xy_qj_A22160001 <= 534
                                                  else '5.(534,543]' if x.xy_qj_A22160001 > 534 and x.xy_qj_A22160001 <= 543
                                                  else '6.(543,580]' if x.xy_qj_A22160001 > 543 and x.xy_qj_A22160001 <= 580
                                                  else '7.(580,621]' if x.xy_qj_A22160001 > 580 and x.xy_qj_A22160001 <= 621
                                                  else '8.(621,622]' if x.xy_qj_A22160001 > 621 and x.xy_qj_A22160001 <= 622
                                                  else '9.(622,623.5]' if x.xy_qj_A22160001 > 622 and x.xy_qj_A22160001 <= 623.5
                                                  else '91.(623.5,inf)' if x.xy_qj_A22160001 > 623.5
                                                  else '92.-999', axis = 1)
# score_201048
df_info_ori['score_201048_Bin'] = df_info_ori.apply(lambda x : '1.[0,465]' if x.score_201048 >= 0 and x.score_201048 <= 465
                                                  else '2.(465,491]' if x.score_201048 > 465 and x.score_201048 <= 491
                                                  else '3.(491,510]' if x.score_201048 > 491 and x.score_201048 <= 510
                                                  else '4.(510,526]' if x.score_201048 > 510 and x.score_201048 <= 526
                                                  else '5.(526,540]' if x.score_201048 > 526 and x.score_201048 <= 540
                                                  else '6.(540,555]' if x.score_201048 > 540 and x.score_201048 <= 555
                                                  else '7.(555,571]' if x.score_201048 > 555 and x.score_201048 <= 571
                                                  else '8.(571,589]' if x.score_201048 > 571 and x.score_201048 <= 589
                                                  else '9.(589,613]' if x.score_201048 > 589 and x.score_201048 <= 613
                                                  else '91.(613,inf)' if x.score_201048 > 613
                                                  else '92.-999', axis = 1)

# score_202098
df_info_ori['score_202098_Bin'] = df_info_ori.apply(lambda x : '1.[0,431]' if x.score_202098 >= 0 and x.score_202098 <= 431
                                                  else '2.(431,456]' if x.score_202098 > 431 and x.score_202098 <= 456
                                                  else '3.(456,474]' if x.score_202098 > 456 and x.score_202098 <= 474
                                                  else '4.(474,490]' if x.score_202098 > 474 and x.score_202098 <= 490
                                                  else '5.(490,506]' if x.score_202098 > 490 and x.score_202098 <= 506
                                                  else '6.(506,523]' if x.score_202098 > 506 and x.score_202098 <= 523
                                                  else '7.(523,542]' if x.score_202098 > 523 and x.score_202098 <= 542
                                                  else '8.(542,564]' if x.score_202098 > 542 and x.score_202098 <= 564
                                                  else '9.(564,595]' if x.score_202098 > 564 and x.score_202098 <= 595
                                                  else '91.(595,inf)' if x.score_202098 > 595
                                                  else '92.-999', axis = 1)

# score_202100
df_info_ori['score_202100_Bin'] = df_info_ori.apply(lambda x : '1.[0,442]' if x.score_202100 >= 0 and x.score_202100 <= 442
                                                  else '2.(442,466]' if x.score_202100 > 442 and x.score_202100 <= 466
                                                  else '3.(466,484]' if x.score_202100 > 466 and x.score_202100 <= 484
                                                  else '4.(484,500]' if x.score_202100 > 484 and x.score_202100 <= 500
                                                  else '5.(500,515]' if x.score_202100 > 500 and x.score_202100 <= 515
                                                  else '6.(515,531]' if x.score_202100 > 515 and x.score_202100 <= 531
                                                  else '7.(531,548]' if x.score_202100 > 531 and x.score_202100 <= 548
                                                  else '8.(548,568]' if x.score_202100 > 548 and x.score_202100 <= 568
                                                  else '9.(568,597]' if x.score_202100 > 568 and x.score_202100 <= 597
                                                  else '91.(597,inf)' if x.score_202100 > 597
                                                  else '92.-999', axis = 1)

### 10_22.机审2

In [338]:
df_psi_js.shape
df_psi_js[:1]

# 样本应取进件免押的且未被机审强拒的（0801除外），但实际应用中，由于拒量商家、拒量回捞原因，机审原因被覆盖，
# 故 分析样本为进件免押的，即跟机审1的样本一致

(159985, 189)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred,y_fraud_fpd7,y_fraud_mob3dpd7,y_credit_mob3dpd30,y_credit_mob6dpd30,y_credit,火眸分_Bin,rentOnlineRiskLevelFulfill_Bin,rentOnlineRiskLevelOverall_Bin,rentOnlineDetailLevelOverall_Bin,蚁盾分_Bin,融安分_Bin,my_zl_score2_Bin,br_ys_score_Bin,青云分_Bin,model_score_2311_Bin,model_score_yl_Bin,model_score_2401_Bin,model_score_2501_Bin,grade_Bin,strategy_240703_4_Bin,strategy_240703_10_Bin,strategy_240703_14_Bin,score_201048_Bin,score_202098_Bin,score_202100_Bin,strategy_240801_1_Bin,strategy_240801_2_Bin,strategy_240801_3_Bin,strategy_240801_4_Bin,strategy_240801_5_Bin,strategy_240801_7_Bin,strategy_240801_8_Bin,strategy_240801_9_Bin,strategy_240801_10_Bin,strategy_240801_11_Bin,规则42,履约保护_Bin,td212_Bin,zm_jh_t0_level_Bin,FICO_Bin,是否出库
44,A2024100108384045,2024-10-01,2024-10,生活号,1,客户取消,[,0,0,0,0,None,None,19167617224,450325200308041217,None,1546538.0,81c1d1f2a5fcf827d199c2eded4c2056,86cbf23bbc1e6f3f32238430bad35faa,591.0,1,1,0,2,48.5025,50.4152,780.0,724.0,14.319542,7.924964,12.569588,NaN,2,536.0,C,580.0,73.0,6.0,3.0,0.0,13.0,2024-09,2.0,5.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,3600.0,3600.0,3600.0,85.0,547.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,"[500,1000)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,"[500,1000)",0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,,85.0,0.0,414.0,2023-08,586.0,575.0,565.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,5700.0,5.0,22.0,4.0,0.0,0.0,15.0,26.0,0.0,9.0,5700.0,2.0,15.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,800.0,7.0,6.0,989,677.0,32.25,20.0,0.0,0.0,0.0,0.0,0.0,"5.(579,657]",2.0,3.1,5.2,"6.(25,50]","5.(700,750]","4.(50,65)","91.(742,inf)","4.(533.0,575.0]","6.[0,18.321]","6.[0,17.55]","6.[0,11.04]",other,4.C,3.客群3,4.客群4,4.客群4,"5.(543,588]","5.(546,602)","6.(524,inf)",3.客群3,3.客群3,3.客群3,3.客群3,2.客群2,4.-999,3.客群3,3.客群3,3.客群3,2.客群2,4,"5.[10,25]","5.[0,50]",6.989,"8.(672

In [ ]:
df_info_ori['jx_fico_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,489.5]' if x.jx_fico_score >= 0 and x.jx_fico_score <= 489.5 
                                                  else '2.(489.5,537]' if x.jx_fico_score > 489.5  and x.jx_fico_score <= 537
                                                  else '3.(537,559]' if x.jx_fico_score > 537 and x.jx_fico_score <= 559
                                                  else '4.(559,588]' if x.jx_fico_score > 559 and x.jx_fico_score <= 588
                                                  else '5.(588,604.5]' if x.jx_fico_score > 588 and x.jx_fico_score <= 604.5
                                                  else '6.(604.5,622]' if x.jx_fico_score > 604.5 and x.jx_fico_score <= 622
                                                  else '7.(622,642]' if x.jx_fico_score > 622 and x.jx_fico_score <= 642
                                                  else '8.(642,664]' if x.jx_fico_score > 642 and x.jx_fico_score <= 664
                                                  else '9.(664,692]' if x.jx_fico_score > 664 and x.jx_fico_score <= 692
                                                  else '91.(692,inf)' if x.jx_fico_score > 692
                                                  else '92.-999', axis = 1)

##  10_3.出库前

In [351]:
# （1）宽表整合
# 观察2024.10及之后的
# 基于202410的免押进件且机审未被强拒的且不是拒量的
# 免押进件且机审未被强拒的且不是拒量的 & 出库
# df_info_ori[(df_info_ori.审核状态 == '机审强拒')&(df_info_ori.下单日期<='2025-05-27')&
#             (df_info_ori.是否拒量 == 0)&(df_info_ori.td212_score>0)].shape

conditions_1 = (df_info_ori.下单日期 >= '2024-10-01')
conditions_2 = (~df_info_ori.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))
conditions_3 = (df_info_ori.审核状态 != '机审强拒')
conditions_4 = (df_info_ori.是否拒量 == 0)

# var_bin_ckq_str = []
df_psi_ckq = df_info_ori[conditions_1 & conditions_2 & conditions_3 & conditions_4]
df_analy_ks['是否出库'] = 1
df_psi_ckq = df_psi_ckq.merge(df_analy_ks[['order_number','是否出库']], on = 'order_number', how = 'left')
df_psi_ckq = df_psi_ckq.drop_duplicates(subset = 'order_number')

df_psi_ckq = df_psi_ckq.sort_values(by = '是否出库', ascending = False)
df_psi_ckq = df_psi_ckq.drop_duplicates(subset = ['id_card_num','user_mobile','下单日期'],  keep='first')
df_psi_ckq.shape

(61451, 189)

In [353]:
# （2）分类变量
conditions_5 = (df_psi_ckq.是否出库 == 1)
conditions_6 = ((~df_psi_ckq.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_psi_ckq.是否拒量 == 0))
pd.pivot_table(df_psi_ckq, index='zm_jh_t0_level_Bin', columns='search_time', values='order_number',aggfunc='count')
pd.pivot_table(df_psi_ckq[conditions_5 & conditions_6], index='zm_jh_t0_level_Bin', columns='search_time', values='order_number',aggfunc='count')

search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
zm_jh_t0_level_Bin,,,,,,,,
1.998,28,23,30,17,11,28,58,57
2.996,303,320,427,281,258,424,792,941
3.979,780,956,1160,808,869,1239,2106,2523
4.975,1119,1392,1857,1321,1330,1810,2909,3526
5.969,324,427,573,415,467,645,903,1115
6.989,1176,1195,1535,1042,1075,1273,1565,1692
7.-999,788,2667,4232,3187,987,1206,1579,1680


search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
zm_jh_t0_level_Bin,,,,,,,,
2.996,106.0,154.0,191.0,71.0,52.0,139.0,307.0,343.0
3.979,331.0,536.0,638.0,283.0,311.0,468.0,767.0,896.0
4.975,505.0,824.0,1039.0,594.0,570.0,821.0,1231.0,1318.0
5.969,142.0,251.0,329.0,201.0,203.0,292.0,394.0,432.0
6.989,511.0,703.0,878.0,522.0,527.0,586.0,776.0,659.0
7.-999,12.0,29.0,25.0,2.0,NaN,5.0,1.0,18.0


In [359]:
# （3）连续变量
# 用2024.10 的进件等频分组
# 进件 & 出库
df_psi_ckq['nsf_rental_v1_3_hundred'] = df_psi_ckq['nsf_rental_v1_3_hundred'].fillna(-999)
df_psi_ckq['nsf_rental_v1_3_hundred'] = np.where(df_psi_ckq.nsf_rental_v1_3_hundred <= 0, -999, df_psi_ckq.nsf_rental_v1_3_hundred)
df_bin_ckq = df_psi_ckq[(df_psi_ckq.下单日期 >= '2024-10-01') & (df_psi_ckq.下单日期 <= '2024-10-31')]

df_bin_ckq.shape
train, test = train_test_split(df_bin_ckq, test_size = 0.2, random_state = 0)
train['target'] = 1
test['target'] = 0
df_bin_ckq = pd.concat([train, test])

df_bin_ckq.shape
# df_bin_ckq[:1]


cutOff = df_qcut_me(df_bin_ckq, 'nsf_rental_v1_3_hundred', 'target', max_interval=10, special_attribute = [-1111, -999])
cutOff


# （1-3）根据业务应用做相应的调整
# 蚁盾分1.0
# cutOff[10] = 70
# cutOff
# cutOff[5] = 4.5
# cutOff[6] = 10.5
# cutOff[8] = 0.95

# # （1-4）映射
var_bin_list_ckq = []
df = df_psi_ckq.copy()
var_bin_ckq = ['nsf_rental_v1_3_hundred']
for col in var_bin_ckq:
    # print ("{} is in processing".format(col))
    if -1111 not in set(df[col]) and -999 not in set(df[col]): 
        # cutOff = df_qcut_me(df, col, 'target', max_interval=10, special_attribute=[])
        df[col+'_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[]))
        # del df[col]
        # var_bin_list_ckq.append(col)
    else:
        # cutOff = df_qcut_me(df, col, 'target', max_interval = 12, special_attribute = [-1111, -999])
        df[col + '_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[-1111, -999])) 
        # del df[col]
        # var_bin_list_ckq.append(col)
        
df['nsf_rental_v1_3_hundred_Bin'] = df.apply(lambda x : '91.[0,7]' if x.nsf_rental_v1_3_hundred >= 0 and x.nsf_rental_v1_3_hundred <= 7
                                                  else '9.(7,8]' if x.nsf_rental_v1_3_hundred > 7  and x.nsf_rental_v1_3_hundred <= 8
                                                  else '8.(8,9]' if x.nsf_rental_v1_3_hundred > 8 and x.nsf_rental_v1_3_hundred <= 9
                                                  else '7.(9,10]' if x.nsf_rental_v1_3_hundred > 9 and x.nsf_rental_v1_3_hundred <= 10
                                                  else '6.(10,11]' if x.nsf_rental_v1_3_hundred > 10 and x.nsf_rental_v1_3_hundred <= 11
                                                  else '5.(11,12]' if x.nsf_rental_v1_3_hundred > 11 and x.nsf_rental_v1_3_hundred <= 12
                                                  else '4.(12,14]' if x.nsf_rental_v1_3_hundred > 12 and x.nsf_rental_v1_3_hundred <= 14
                                                  else '3.(14,16]' if x.nsf_rental_v1_3_hundred > 14 and x.nsf_rental_v1_3_hundred <= 16
                                                  else '2.(16,25]' if x.nsf_rental_v1_3_hundred > 16 and x.nsf_rental_v1_3_hundred <= 25
                                                  else '1.(25,inf)' if x.nsf_rental_v1_3_hundred > 25
                                                  else '92.-999', axis = 1)

# （1-5）统计
# df.审核状态 == '出库' 有可能为拒量商家的出库
# conditions_5 = (df.审核状态 == '出库') & (df.是否出库 == 1)  & (df.是否拒量 == 0)

conditions_5 = (df.是否出库 == 1)
conditions_6 = ((~df.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df.是否拒量 == 0))
pd.pivot_table(df, index='nsf_rental_v1_3_hundred_Bin', columns='search_time', values='order_number',aggfunc='count')
pd.pivot_table(df[conditions_5 & conditions_6], index='nsf_rental_v1_3_hundred_Bin', columns='search_time', values='order_number',aggfunc='count')




(4518, 189)

(4518, 190)

[-1111, -999, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 14.0, 16.0, 20.0]

search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
nsf_rental_v1_3_hundred_Bin,,,,,,,,
"1.(25,inf)",173,186,306,238,244,529,728,576
"2.(16,25]",597,608,793,618,590,915,1527,1598
"3.(14,16]",359,404,472,364,349,486,844,831
"4.(12,14]",512,609,787,533,575,817,1123,1372
"5.(11,12]",333,404,519,390,398,566,818,908
"6.(10,11]",409,471,587,442,447,585,936,1104
"7.(9,10]",447,536,677,455,514,681,984,1128
"8.(8,9]",488,556,722,458,499,682,930,1111
"9.(7,8]",449,471,622,395,401,562,821,833


search_time,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
nsf_rental_v1_3_hundred_Bin,,,,,,,,
"1.(25,inf)",10.0,8.0,12.0,4.0,9.0,14.0,29.0,37.0
"2.(16,25]",193.0,279.0,336.0,169.0,198.0,298.0,536.0,478.0
"3.(14,16]",114.0,189.0,218.0,136.0,105.0,187.0,295.0,268.0
"4.(12,14]",185.0,298.0,377.0,190.0,199.0,292.0,402.0,503.0
"5.(11,12]",127.0,212.0,279.0,166.0,155.0,200.0,312.0,334.0
"6.(10,11]",160.0,266.0,305.0,157.0,172.0,227.0,356.0,402.0
"7.(9,10]",161.0,307.0,365.0,186.0,209.0,280.0,387.0,411.0
"8.(8,9]",189.0,306.0,405.0,210.0,214.0,318.0,406.0,450.0
"9.(7,8]",206.0,277.0,354.0,197.0,184.0,250.0,354.0,303.0


In [ ]:
# td212_score
df_info_ori['td212_score_Bin'] = df_info_ori.apply(lambda x : '91.[0,14.24]' if x.td212_score >= 0 and x.td212_score <= 14.24
                                                  else '9.(14.24 ,21.34]' if x.td212_score > 14.24  and x.td212_score <= 21.34
                                                  else '8.(21.34,26.34]' if x.td212_score > 21.34 and x.td212_score <= 26.34
                                                  else '7.(26.34,30.65]' if x.td212_score > 26.34 and x.td212_score <= 30.65
                                                  else '6.(30.65,34.46]' if x.td212_score > 30.65 and x.td212_score <= 34.46
                                                  else '5.(34.46,38.19]' if x.td212_score > 34.46 and x.td212_score <= 38.19
                                                  else '4.(38.19,41.99]' if x.td212_score > 38.19 and x.td212_score <= 41.99
                                                  else '3.(41.99,48.83]' if x.td212_score > 41.99 and x.td212_score <= 48.83
                                                  else '2.(48.83,95]' if x.td212_score > 48.83 and x.td212_score <= 95
                                                  else '1.(95,inf)' if x.td212_score > 95
                                                  else '92.-999', axis = 1)

# 履约保护
df_info_ori['nsf_rental_v1_3_hundred_Bin'] = df_info_ori.apply(lambda x : '91.[0,7]' if x.nsf_rental_v1_3_hundred >= 0 and x.nsf_rental_v1_3_hundred <= 7
                                                  else '9.(7,8]' if x.nsf_rental_v1_3_hundred > 7  and x.nsf_rental_v1_3_hundred <= 8
                                                  else '8.(8,9]' if x.nsf_rental_v1_3_hundred > 8 and x.nsf_rental_v1_3_hundred <= 9
                                                  else '7.(9,10]' if x.nsf_rental_v1_3_hundred > 9 and x.nsf_rental_v1_3_hundred <= 10
                                                  else '6.(10,11]' if x.nsf_rental_v1_3_hundred > 10 and x.nsf_rental_v1_3_hundred <= 11
                                                  else '5.(11,12]' if x.nsf_rental_v1_3_hundred > 11 and x.nsf_rental_v1_3_hundred <= 12
                                                  else '4.(12,14]' if x.nsf_rental_v1_3_hundred > 12 and x.nsf_rental_v1_3_hundred <= 14
                                                  else '3.(14,16]' if x.nsf_rental_v1_3_hundred > 14 and x.nsf_rental_v1_3_hundred <= 16
                                                  else '2.(16,25]' if x.nsf_rental_v1_3_hundred > 16 and x.nsf_rental_v1_3_hundred <= 25
                                                  else '1.(25,inf)' if x.nsf_rental_v1_3_hundred > 25
                                                  else '92.-999', axis = 1)

# 11.模型监控

## 11_1.转化

In [146]:
# （1）宽表整合
# 观察2024.02及之后的
# 基于202410的免押进件分组
# 进件 & 出库
%run C:/Users/hxq/Desktop/mine/1_参考/1_code/湖南银行poc/2.model_func.ipynb

conditions_1 = (df_info_ori.下单日期 >= '2024-03-01')
conditions_2 = (~df_info_ori.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))
# var_bin_sj_str = []
df_psi_js = df_info_ori[conditions_1 & conditions_2]
df_analy_ks['是否出库'] = 1
df_psi_js = df_psi_js.merge(df_analy_ks[['order_number','是否出库']], on = 'order_number', how = 'left')
df_psi_js = df_psi_js.drop_duplicates(subset = 'order_number')

df_psi_js = df_psi_js.sort_values(by = '是否出库', ascending = False)
df_psi_js = df_psi_js.drop_duplicates(subset = ['id_card_num','user_mobile','下单日期'],  keep='first')
df_psi_js.shape

(243064, 189)

In [149]:
# （3-1）连续变量
# 用2024.10 的进件等频分组
# 进件 & 出库
# df_psi_js = df_psi_js[df_psi_js.下单日期 >= '2025-03-01']

# model_score_2311
df_psi_js['model_score_2311'] = df_psi_js['model_score_2311'].fillna(-999)
df_psi_js['model_score_2311'] = np.where(df_psi_js.model_score_2311 <= 0, -999, df_psi_js.model_score_2311)
df_bin_js = df_psi_js[(df_psi_js.下单日期 >= '2024-10-01') & (df_psi_js.下单日期 <= '2024-10-31')]

df_bin_js.shape
train, test = train_test_split(df_bin_js, test_size = 0.2, random_state = 0)
train['target'] = 1
test['target'] = 0
df_bin_js = pd.concat([train, test])

df_bin_js.shape
# df_bin_js[:1]


cutOff = df_qcut_me(df_bin_js, 'model_score_2311', 'target', max_interval=10, special_attribute = [-1111, -999])
cutOff


# （1-3）根据业务应用做相应的调整
# 蚁盾分1.0
# cutOff[10] = 70
# cutOff
# cutOff[5] = 4.5
# cutOff[6] = 10.5
# cutOff[8] = 0.95

# # （1-4）映射
var_bin_list_js = []
df = df_psi_js.copy()
var_bin_js = ['model_score_2311']
for col in var_bin_js:
    # print ("{} is in processing".format(col))
    if -1111 not in set(df[col]) and -999 not in set(df[col]): 
        # cutOff = df_qcut_me(df, col, 'target', max_interval=10, special_attribute=[])
        df[col+'_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[]))
        # del df[col]
        # var_bin_list_js.append(col)
    else:
        # cutOff = df_qcut_me(df, col, 'target', max_interval = 12, special_attribute = [-1111, -999])
        df[col + '_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[-1111, -999])) 
        # del df[col]
        # var_bin_list_js.append(col)

df['model_score_2311_Bin'] = df.apply(lambda x : '91.[0,14.374]' if x.model_score_2311 >= 0 and x.model_score_2311 <= 14.374 
                                                  else '9.(14.374,16.299]' if x.model_score_2311 > 14.374  and x.model_score_2311 <= 16.299
                                                  else '8.(16.299,17.669]' if x.model_score_2311 > 16.299 and x.model_score_2311 <= 17.669
                                                  else '7.(17.669,19.989]' if x.model_score_2311 > 17.669 and x.model_score_2311 <= 19.989
                                                  else '6.(19.989,20.787]' if x.model_score_2311 > 19.989 and x.model_score_2311 <= 20.787
                                                  else '5.(20.787,22.754]' if x.model_score_2311 > 20.787 and x.model_score_2311 <= 22.754
                                                  else '4.(22.754,24.404]' if x.model_score_2311 > 22.754 and x.model_score_2311 <= 24.404
                                                  else '3.(24.404,25.065]' if x.model_score_2311 > 24.404 and x.model_score_2311 <= 25.065
                                                  else '2.(25.065,27.295]' if x.model_score_2311 > 25.065 and x.model_score_2311 <= 27.295
                                                  else '1.(27.295,inf)' if x.model_score_2311 > 27.295
                                                  else '92.-999', axis = 1)

# （1-5）统计
# df.审核状态 == '出库' 有可能为拒量商家的出库
# conditions_3 = (df.审核状态 == '出库') & (df.是否出库 == 1)  & (df.是否拒量 == 0)
conditions_3 = (df.是否出库 == 1)
conditions_4 = ((~df.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df.是否拒量 == 0))
conditions_5 = (df.is_model == '1')
conditions_6 = ['单人聊天会话中的小程序消息卡片（分享）',
                '八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']

# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
# conditions_1 = (df_info_ori.is_model == '1')
# conditions_2 = (df_info_ori.is_model == '2')
# conditions_3 = (df_info_ori.is_model == '3')
# conditions_4 = (df_info_ori.is_model == '4')
   

df['model_score_2311_result']= df.apply(lambda x: '1.accept' if x.归属渠道 != '搜索渠道'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
                                                else '2.review' if x.归属渠道 != '搜索渠道'  and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
                                                else '3.reject' if x.归属渠道 != '搜索渠道'  and x.model_score_2311 > 23.297 

                                                else '1.accept' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
                                                else '2.review' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 16.6 and x.model_score_2311 <= 21.435
                                                else '3.reject' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 21.435
                                                else 'other'
                                                , axis = 1)
df['model_score_2311_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_2311_result == '2.review'), '3.reject', df.model_score_2311_result)
df['model_score_2311_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_2311_result == '1.accept'), '2.review', df.model_score_2311_result)
df['model_score_2311_result_f'] = np.where(df.model_score_2311_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')

# 进件
pd.pivot_table(df, index='model_score_2311_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的进件
pd.pivot_table(df[conditions_5], index='model_score_2311_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的通过
pd.pivot_table(df[conditions_5 & (df.model_score_2311_result_f == '模型通过')], index='model_score_2311_Bin', columns='search_time', values='order_number',aggfunc='count')
# # 使用该模型的出库
# pd.pivot_table(df[conditions_3 & conditions_4 & conditions_5], index='model_score_2311_Bin', columns='search_time', values='order_number',aggfunc='count')
# 总体出库
pd.pivot_table(df[conditions_3 & conditions_4], index='model_score_2311_Bin', columns='search_time', values='order_number',aggfunc='count')


(13614, 189)

(13614, 190)

[-1111,
 -999,
 14.37375657366382,
 16.29894164069047,
 17.66904988819089,
 19.989107427119524,
 20.786642297987072,
 22.75366749027868,
 24.403848118578782,
 25.064675333325532,
 27.29507224561608]

search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2311_Bin,,,,,,,,,,,,,,,,
"1.(27.295,inf)",430,514,535,422,548,859,1350,1359,1355,1878,1431,1970,2652,4120,5288,113
"2.(25.065,27.295]",425,556,562,636,821,1001,1445,1328,1321,1734,1415,1875,2586,3691,4367,81
"3.(24.404,25.065]",463,540,592,596,724,919,1207,1230,1368,1731,1313,1771,2321,3156,4042,91
"4.(22.754,24.404]",650,892,850,975,1189,1308,1708,1514,1602,1855,1481,1793,2453,3664,4564,114
"5.(20.787,22.754]",487,820,802,1091,1228,1241,1601,1209,1180,1299,1009,1186,1616,2556,3065,73
"6.(19.989,20.787]",581,797,688,927,988,1269,1585,1510,1555,2061,1412,1522,2039,2769,3565,78
"7.(17.669,19.989]",761,1157,1060,1316,1576,1565,1876,1357,1226,1495,1203,1247,1591,2447,3328,82
"8.(16.299,17.669]",558,1052,977,1230,1402,1531,1777,1343,1268,1669,1197,1297,1591,2238,2611,82
"9.(14.374,16.299]",623,1164,1155,1506,1594,1599,1831,1372,1564,2086,1393,1388,1870,2429,2708,51


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2311_Bin,,,,,,,,,,,,,,,,
"1.(27.295,inf)",92,107,94,84,114,153,255,263,256,381,267,352,374,601,663,6
"2.(25.065,27.295]",74,107,122,124,147,231,261,250,249,366,272,295,350,516,547,14
"3.(24.404,25.065]",88,109,118,112,137,174,218,216,234,333,257,280,306,422,520,10
"4.(22.754,24.404]",146,184,166,184,235,257,321,275,307,360,272,275,378,506,617,18
"5.(20.787,22.754]",99,171,170,237,233,255,298,221,251,279,194,225,198,348,446,13
"6.(19.989,20.787]",104,169,120,189,189,283,285,289,302,424,271,256,263,375,467,12
"7.(17.669,19.989]",172,251,208,259,308,283,381,227,236,292,228,203,226,342,448,9
"8.(16.299,17.669]",115,238,185,249,291,323,324,276,255,310,241,205,234,312,372,16
"9.(14.374,16.299]",123,258,225,330,315,343,366,247,309,386,292,220,260,354,380,6


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2311_Bin,,,,,,,,,,,,,,,,
"4.(22.754,24.404]",65,62,56,76,92,88,101,47,45,47,44,47,62,74,87,1
"5.(20.787,22.754]",84,134,142,218,200,206,224,134,153,156,120,160,129,215,293,8
"6.(19.989,20.787]",99,164,118,183,186,271,257,255,262,377,243,238,240,320,420,12
"7.(17.669,19.989]",169,248,207,254,307,272,355,210,213,266,205,187,199,295,394,7
"8.(16.299,17.669]",115,238,185,249,291,323,324,276,255,310,241,205,234,312,372,16
"9.(14.374,16.299]",123,258,225,330,315,343,366,247,309,386,292,220,260,354,380,6
"91.[0,14.374]",159,271,244,351,368,405,435,260,211,222,129,105,144,159,206,2


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2311_Bin,,,,,,,,,,,,,,,,
"1.(27.295,inf)",9.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0,5.0,1.0,3.0,15.0,39.0,73.0,114.0,1.0
"2.(25.065,27.295]",11.0,4.0,4.0,5.0,9.0,15.0,23.0,16.0,29.0,38.0,23.0,35.0,74.0,130.0,128.0,5.0
"3.(24.404,25.065]",22.0,15.0,13.0,12.0,15.0,31.0,37.0,43.0,105.0,136.0,81.0,102.0,196.0,265.0,333.0,8.0
"4.(22.754,24.404]",42.0,47.0,19.0,25.0,32.0,51.0,65.0,41.0,118.0,156.0,107.0,105.0,176.0,298.0,348.0,9.0
"5.(20.787,22.754]",48.0,70.0,50.0,74.0,75.0,103.0,158.0,52.0,102.0,135.0,67.0,73.0,110.0,243.0,282.0,9.0
"6.(19.989,20.787]",91.0,119.0,101.0,106.0,148.0,264.0,339.0,346.0,502.0,662.0,382.0,359.0,491.0,665.0,751.0,15.0
"7.(17.669,19.989]",105.0,195.0,122.0,172.0,187.0,204.0,257.0,124.0,213.0,273.0,161.0,177.0,188.0,354.0,467.0,7.0
"8.(16.299,17.669]",116.0,237.0,201.0,250.0,263.0,308.0,350.0,254.0,396.0,528.0,290.0,295.0,381.0,537.0,602.0,20.0
"9.(14.374,16.299]",150.0,339.0,328.0,360.0,421.0,496.0,495.0,383.0,621.0,811.0,415.0,385.0,481.0,673.0,767.0,14.0


In [150]:
# （3-2）model_score_yl
df_psi_js['model_score_yl'] = df_psi_js['model_score_yl'].fillna(-999)
df_psi_js['model_score_yl'] = np.where(df_psi_js.model_score_yl <= 0, -999, df_psi_js.model_score_yl)
df_bin_js = df_psi_js[(df_psi_js.下单日期 >= '2024-10-01') & (df_psi_js.下单日期 <= '2024-10-31')]

df_bin_js.shape
train, test = train_test_split(df_bin_js, test_size = 0.2, random_state = 0)
train['target'] = 1
test['target'] = 0
df_bin_js = pd.concat([train, test])

df_bin_js.shape
# df_bin_js[:1]


cutOff = df_qcut_me(df_bin_js, 'model_score_yl', 'target', max_interval=10, special_attribute = [-1111, -999])
cutOff


# （1-3）根据业务应用做相应的调整
# 蚁盾分1.0
# cutOff[10] = 70
# cutOff
# cutOff[5] = 4.5
# cutOff[6] = 10.5
# cutOff[8] = 0.95

# # （1-4）映射
var_bin_list_js = []
df = df_psi_js.copy()
var_bin_js = ['model_score_yl']
for col in var_bin_js:
    # print ("{} is in processing".format(col))
    if -1111 not in set(df[col]) and -999 not in set(df[col]): 
        # cutOff = df_qcut_me(df, col, 'target', max_interval=10, special_attribute=[])
        df[col+'_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[]))
        # del df[col]
        # var_bin_list_js.append(col)
    else:
        # cutOff = df_qcut_me(df, col, 'target', max_interval = 12, special_attribute = [-1111, -999])
        df[col + '_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[-1111, -999])) 
        # del df[col]
        # var_bin_list_js.append(col)


df['model_score_yl_Bin'] = df.apply(lambda x : '91.[0,12.142]' if x.model_score_yl >= 0 and x.model_score_yl <= 12.142 
                                                  else '9.(12.142,14.524]' if x.model_score_yl > 12.142  and x.model_score_yl <= 14.524
                                                  else '8.(14.524,16.515]' if x.model_score_yl > 14.524 and x.model_score_yl <= 16.515
                                                  else '7.(16.515,18.653]' if x.model_score_yl > 16.515 and x.model_score_yl <= 18.653
                                                  else '6.(18.653,20.508]' if x.model_score_yl > 18.653 and x.model_score_yl <= 20.508
                                                  else '5.(20.508,22.376]' if x.model_score_yl > 20.508 and x.model_score_yl <= 22.376
                                                  else '4.(22.376,24.748]' if x.model_score_yl > 22.376 and x.model_score_yl <= 24.748
                                                  else '3.(24.748,27.666]' if x.model_score_yl > 24.748 and x.model_score_yl <= 27.666
                                                  else '2.(27.666,30.907]' if x.model_score_yl > 27.666 and x.model_score_yl <= 30.907
                                                  else '1.(30.907,inf)' if x.model_score_yl > 30.907
                                                  else '92.-999', axis = 1)

# （1-5）统计
# df.审核状态 == '出库' 有可能为拒量商家的出库
# conditions_3 = (df.审核状态 == '出库') & (df.是否出库 == 1)  & (df.是否拒量 == 0)
conditions_3 = (df.是否出库 == 1)
conditions_4 = ((~df.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df.是否拒量 == 0))
conditions_5 = (df.is_model == '2')
conditions_6 = ['单人聊天会话中的小程序消息卡片（分享）',
                '八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']

# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
# conditions_1 = (df_info_ori.is_model == '1')
# conditions_2 = (df_info_ori.is_model == '2')
# conditions_3 = (df_info_ori.is_model == '3')
# conditions_4 = (df_info_ori.is_model == '4')
   

df['model_score_yl_result']= df.apply(lambda x: '1.accept' if x.归属渠道 != '搜索渠道'  and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
                                                else '2.review' if x.归属渠道 != '搜索渠道'  and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
                                                else '3.reject' if x.归属渠道 != '搜索渠道'  and x.model_score_yl > 23.382

                                                else '1.accept' if x.归属渠道 == '搜索渠道'  and x.model_score_yl >= 0  and x.model_score_yl <= 15.059
                                                else '2.review' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 15.059 and x.model_score_yl <= 21.661
                                                else '3.reject' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 21.661
                                                else 'other'
                                                , axis = 1)
df['model_score_yl_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_yl_result == '2.review'), '3.reject', df.model_score_yl_result)
df['model_score_yl_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_yl_result == '1.accept'), '2.review', df.model_score_yl_result)
# 2025.3.28 上线
# 芝麻租物，ascore_yl202311>23.382强拒；搜索渠道，ascore_yl202311>21.661强拒’调整为‘芝麻租物和搜索渠道，ascore_yl202311>24.5强拒
df['model_score_yl_result'] = np.where((df.下单日期 >= '2025-03-28') & (df.model_score_yl_result == '3.reject') & (df.model_score_yl <= 24.5) & (df.归属渠道.isin(['芝麻租物','搜索渠道'])), '2.review', df.model_score_yl_result)

df['model_score_yl_result_f'] = np.where(df.model_score_yl_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')
# 进件
pd.pivot_table(df, index='model_score_yl_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的进件
pd.pivot_table(df[conditions_5], index='model_score_yl_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的通过
pd.pivot_table(df[conditions_5 & (df.model_score_yl_result_f == '模型通过')], index='model_score_yl_Bin', columns='search_time', values='order_number',aggfunc='count')
# # 使用该模型的出库
# pd.pivot_table(df[conditions_3 & conditions_4 & conditions_5], index='model_score_yl_Bin', columns='search_time', values='order_number',aggfunc='count')
# 总体出库
pd.pivot_table(df[conditions_3 & conditions_4], index='model_score_yl_Bin', columns='search_time', values='order_number',aggfunc='count')


(13614, 189)

(13614, 190)

[-1111,
 -999,
 12.142192479713067,
 14.524291112026152,
 16.51520622811055,
 18.653814541773485,
 20.50829456258781,
 22.3759875000574,
 24.748492597107376,
 27.666273953574265,
 30.907265187932698]

search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_yl_Bin,,,,,,,,,,,,,,,,
"1.(30.907,inf)",547,660,638,678,935,1138,1634,1358,1325,1819,1436,2087,2735,4181,5321,112
"2.(27.666,30.907]",484,642,634,699,895,1132,1392,1358,1289,1674,1322,1758,2308,3447,4386,85
"3.(24.748,27.666]",535,702,686,797,1055,1146,1532,1358,1426,1678,1372,1776,2364,3525,4366,106
"4.(22.376,24.748]",537,817,810,892,1105,1196,1577,1357,1307,1696,1311,1627,2281,3225,3917,87
"5.(20.508,22.376]",576,850,780,964,1068,1189,1630,1359,1364,1787,1328,1505,2096,2913,3535,81
"6.(18.653,20.508]",558,900,845,1142,1214,1342,1614,1364,1398,1601,1248,1362,1820,2528,3301,93
"7.(16.515,18.653]",653,963,996,1141,1362,1406,1688,1352,1368,1694,1275,1415,1808,2558,3126,78
"8.(14.524,16.515]",560,1027,968,1170,1278,1359,1683,1374,1430,1807,1265,1309,1659,2328,2785,69
"9.(12.142,14.524]",605,1084,984,1344,1421,1461,1812,1346,1445,1885,1145,1103,1478,2042,2398,48


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_yl_Bin,,,,,,,,,,,,,,,,
"1.(30.907,inf)",109.0,247.0,391.0,413.0,557.0,672.0,981.0,823.0,799.0,1109.0,877.0,1069.0,1142.0,1739.0,2127.0,48.0
"2.(27.666,30.907]",84.0,228.0,372.0,428.0,540.0,706.0,841.0,831.0,766.0,1005.0,789.0,872.0,998.0,1407.0,1735.0,30.0
"3.(24.748,27.666]",102.0,247.0,410.0,496.0,657.0,701.0,912.0,833.0,857.0,977.0,803.0,930.0,1018.0,1505.0,1755.0,38.0
"4.(22.376,24.748]",100.0,296.0,491.0,547.0,666.0,717.0,978.0,818.0,775.0,1014.0,769.0,830.0,964.0,1355.0,1553.0,27.0
"5.(20.508,22.376]",114.0,303.0,463.0,575.0,635.0,726.0,1008.0,851.0,834.0,1087.0,845.0,758.0,877.0,1248.0,1464.0,33.0
"6.(18.653,20.508]",119.0,319.0,505.0,664.0,732.0,775.0,968.0,851.0,878.0,1001.0,772.0,688.0,776.0,1046.0,1329.0,42.0
"7.(16.515,18.653]",132.0,341.0,615.0,653.0,817.0,852.0,1010.0,811.0,861.0,1007.0,764.0,705.0,744.0,1106.0,1324.0,23.0
"8.(14.524,16.515]",112.0,361.0,579.0,693.0,762.0,825.0,1037.0,857.0,847.0,1090.0,761.0,640.0,710.0,973.0,1153.0,25.0
"9.(12.142,14.524]",119.0,416.0,582.0,805.0,844.0,858.0,1108.0,820.0,877.0,1202.0,693.0,547.0,632.0,836.0,971.0,18.0


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_yl_Bin,,,,,,,,,,,,,,,,
"4.(22.376,24.748]",29,117,170,199,225,212,291,178,192,254,217,218,317,1013,1226,21
"5.(20.508,22.376]",103,267,401,514,564,607,801,637,668,860,632,593,682,1021,1227,30
"6.(18.653,20.508]",117,315,500,646,719,728,854,742,771,897,692,620,685,890,1166,36
"7.(16.515,18.653]",131,338,608,641,813,836,961,767,803,950,725,669,702,1005,1218,22
"8.(14.524,16.515]",112,361,579,693,762,825,1037,857,847,1090,761,640,710,973,1153,25
"9.(12.142,14.524]",119,416,582,805,844,858,1108,820,877,1202,693,547,632,836,971,18
"91.[0,12.142]",156,454,610,849,996,1063,1264,826,720,781,489,370,428,623,748,19


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_yl_Bin,,,,,,,,,,,,,,,,
"1.(30.907,inf)",17.0,14.0,4.0,7.0,4.0,5.0,7.0,3.0,4.0,3.0,1.0,13.0,29.0,70.0,73.0,3.0
"2.(27.666,30.907]",18.0,22.0,1.0,10.0,10.0,19.0,15.0,11.0,19.0,26.0,23.0,35.0,61.0,93.0,143.0,1.0
"3.(24.748,27.666]",34.0,24.0,11.0,12.0,17.0,17.0,24.0,11.0,32.0,36.0,27.0,33.0,101.0,130.0,166.0,1.0
"4.(22.376,24.748]",41.0,51.0,29.0,29.0,36.0,67.0,92.0,43.0,75.0,120.0,64.0,103.0,135.0,299.0,370.0,11.0
"5.(20.508,22.376]",69.0,106.0,63.0,89.0,100.0,134.0,214.0,144.0,265.0,341.0,188.0,210.0,304.0,438.0,511.0,15.0
"6.(18.653,20.508]",74.0,133.0,68.0,122.0,135.0,197.0,226.0,166.0,283.0,350.0,210.0,208.0,278.0,419.0,512.0,15.0
"7.(16.515,18.653]",112.0,191.0,153.0,179.0,227.0,276.0,322.0,212.0,360.0,472.0,273.0,276.0,372.0,487.0,613.0,18.0
"8.(14.524,16.515]",126.0,279.0,247.0,264.0,322.0,361.0,448.0,308.0,505.0,646.0,359.0,313.0,403.0,601.0,640.0,14.0
"9.(12.142,14.524]",144.0,313.0,310.0,378.0,411.0,462.0,504.0,352.0,517.0,654.0,324.0,308.0,377.0,582.0,622.0,8.0


In [151]:
# （3-3）model_score_2401
df_psi_js['model_score_2401'] = df_psi_js['model_score_2401'].fillna(-999)
df_psi_js['model_score_2401'] = np.where(df_psi_js.model_score_2401 <= 0, -999, df_psi_js.model_score_2401)
df_bin_js = df_psi_js[(df_psi_js.下单日期 >= '2024-10-01') & (df_psi_js.下单日期 <= '2024-10-31')]

df_bin_js.shape
train, test = train_test_split(df_bin_js, test_size = 0.2, random_state = 0)
train['target'] = 1
test['target'] = 0
df_bin_js = pd.concat([train, test])

df_bin_js.shape
# df_bin_js[:1]


cutOff = df_qcut_me(df_bin_js, 'model_score_2401', 'target', max_interval=10, special_attribute = [-1111, -999])
cutOff


# （1-3）根据业务应用做相应的调整
# 蚁盾分1.0
# cutOff[10] = 70
# cutOff
# cutOff[5] = 4.5
# cutOff[6] = 10.5
# cutOff[8] = 0.95

# # （1-4）映射
var_bin_list_js = []
df = df_psi_js.copy()
var_bin_js = ['model_score_2401']
for col in var_bin_js:
    # print ("{} is in processing".format(col))
    if -1111 not in set(df[col]) and -999 not in set(df[col]): 
        # cutOff = df_qcut_me(df, col, 'target', max_interval=10, special_attribute=[])
        df[col+'_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[]))
        # del df[col]
        # var_bin_list_js.append(col)
    else:
        # cutOff = df_qcut_me(df, col, 'target', max_interval = 12, special_attribute = [-1111, -999])
        df[col + '_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[-1111, -999])) 
        # del df[col]
        # var_bin_list_js.append(col)

df['model_score_2401_Bin'] = df.apply(lambda x : '91.[0,6.899]' if x.model_score_2401 >= 0 and x.model_score_2401 <= 6.899 
                                                  else '9.(6.899,8.786]' if x.model_score_2401 > 6.899  and x.model_score_2401 <= 8.786
                                                  else '8.(8.786,10.028]' if x.model_score_2401 > 8.786 and x.model_score_2401 <= 10.028
                                                  else '7.(10.028,12.388]' if x.model_score_2401 > 10.028 and x.model_score_2401 <= 12.388
                                                  else '6.(12.388,13.485]' if x.model_score_2401 > 12.388 and x.model_score_2401 <= 13.485
                                                  else '5.(13.485,15.787]' if x.model_score_2401 > 13.485 and x.model_score_2401 <= 15.787
                                                  else '4.(15.787,17.466]' if x.model_score_2401 > 15.787 and x.model_score_2401 <= 17.466
                                                  else '3.(17.466,18.845]' if x.model_score_2401 > 17.466 and x.model_score_2401 <= 18.845
                                                  else '2.(18.845,21.449]' if x.model_score_2401 > 18.845 and x.model_score_2401 <= 21.449
                                                  else '1.(21.449,inf)' if x.model_score_2401 > 21.449
                                                  else '92.-999', axis = 1)

# （1-5）统计
# df.审核状态 == '出库' 有可能为拒量商家的出库
# conditions_3 = (df.审核状态 == '出库') & (df.是否出库 == 1)  & (df.是否拒量 == 0)
conditions_3 = (df.是否出库 == 1)
conditions_4 = ((~df.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df.是否拒量 == 0))
conditions_5 = (df.is_model == '3')
conditions_6 = ['单人聊天会话中的小程序消息卡片（分享）',
                '八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']

# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
# conditions_1 = (df_info_ori.is_model == '1')
# conditions_2 = (df_info_ori.is_model == '2')
# conditions_3 = (df_info_ori.is_model == '3')
# conditions_4 = (df_info_ori.is_model == '4')
   

df['model_score_2401_result']= df.apply(lambda x: '1.accept' if x.归属渠道 != '搜索渠道'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 11.04
                                                else '2.review' if x.归属渠道 != '搜索渠道'  and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16
                                                else '3.reject' if x.归属渠道 != '搜索渠道'  and x.model_score_2401 > 16

                                                else '1.accept' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
                                                else '2.review' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 9.35 and x.model_score_2401 <= 14.93
                                                else '3.reject' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 14.93
                                                else 'other'
                                                , axis = 1)
df['model_score_2401_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_2401_result == '2.review'), '3.reject', df.model_score_2401_result)
df['model_score_2401_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_2401_result == '1.accept'), '2.review', df.model_score_2401_result)
df['model_score_2401_result_f'] = np.where(df.model_score_2401_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')
# 进件
pd.pivot_table(df, index='model_score_2401_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的进件
pd.pivot_table(df[conditions_5], index='model_score_2401_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的通过
pd.pivot_table(df[conditions_5 & (df.model_score_2401_result_f == '模型通过')], index='model_score_2401_Bin', columns='search_time', values='order_number',aggfunc='count')
# # 使用该模型的出库
# pd.pivot_table(df[conditions_3 & conditions_4 & conditions_5], index='model_score_2401_Bin', columns='search_time', values='order_number',aggfunc='count')
# 总体出库
pd.pivot_table(df[conditions_3 & conditions_4], index='model_score_2401_Bin', columns='search_time', values='order_number',aggfunc='count')


(13614, 189)

(13614, 190)

[-1111,
 -999,
 6.898587299341213,
 8.785663252698981,
 10.027996013246865,
 12.387846862891433,
 13.48526442400384,
 15.787234823583333,
 17.46616527426384,
 18.8450374094232,
 21.44919773307096]

search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2401_Bin,,,,,,,,,,,,,,,,
"1.(21.449,inf)",422,637,635,676,869,1027,1473,1419,1509,1820,1471,1820,2501,3847,4033,71
"2.(18.845,21.449]",480,683,665,740,923,1046,1487,1332,1306,1787,1357,1748,2402,3526,4196,90
"3.(17.466,18.845]",532,706,693,839,1008,1116,1593,1397,1397,1760,1307,1697,2313,3273,4123,95
"4.(15.787,17.466]",721,981,890,1170,1282,1491,1725,1302,1268,1402,1043,1250,1636,2369,3117,78
"5.(13.485,15.787]",583,823,681,898,1138,1430,1840,1570,1464,1796,1277,1512,1989,2823,3522,94
"6.(12.388,13.485]",538,710,660,863,974,1160,1414,1115,1047,1274,967,1088,1349,1970,2757,47
"7.(10.028,12.388]",697,894,928,1093,1299,1374,1726,1352,1344,1541,1190,1333,1830,2750,3502,84
"8.(8.786,10.028]",555,895,844,998,1197,1261,1556,1353,1335,1899,1369,1510,1973,2799,3613,83
"9.(6.899,8.786]",509,1181,1049,1379,1640,1607,1907,1323,1322,1732,1232,1299,1746,2559,3087,84


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2401_Bin,,,,,,,,,,,,,,,,
"1.(21.449,inf)",159,243,125,112,169,209,333,301,328,330,289,314,341,503,555,9
"2.(18.845,21.449]",199,275,131,149,199,190,312,269,241,376,270,297,320,535,561,19
"3.(17.466,18.845]",209,295,133,163,199,237,317,291,270,330,252,297,332,457,557,10
"4.(15.787,17.466]",285,412,210,236,279,293,364,248,242,252,206,208,216,320,404,10
"5.(13.485,15.787]",232,348,115,192,219,260,382,332,319,351,263,282,255,423,503,19
"6.(12.388,13.485]",205,296,141,178,211,240,303,213,208,264,215,219,181,270,352,7
"7.(10.028,12.388]",257,356,196,223,267,267,337,258,267,326,247,212,267,424,484,13
"8.(8.786,10.028]",224,387,179,184,238,236,325,270,290,377,263,265,277,390,484,12
"9.(6.899,8.786]",185,486,216,279,334,321,377,277,250,339,244,206,239,371,393,12


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2401_Bin,,,,,,,,,,,,,,,,
"4.(15.787,17.466]",60.0,117.0,56.0,85.0,84.0,81.0,80.0,38.0,26.0,23.0,32.0,6.0,17.0,32.0,34.0,NaN
"5.(13.485,15.787]",206.0,321.0,99.0,176.0,202.0,229.0,312.0,252.0,257.0,273.0,206.0,219.0,204.0,319.0,389.0,15.0
"6.(12.388,13.485]",194.0,292.0,138.0,173.0,209.0,223.0,278.0,190.0,182.0,244.0,188.0,205.0,158.0,228.0,304.0,6.0
"7.(10.028,12.388]",250.0,354.0,195.0,218.0,266.0,260.0,316.0,242.0,245.0,303.0,229.0,200.0,242.0,380.0,441.0,13.0
"8.(8.786,10.028]",224.0,387.0,179.0,184.0,238.0,236.0,325.0,270.0,290.0,377.0,263.0,265.0,277.0,390.0,484.0,12.0
"9.(6.899,8.786]",185.0,486.0,216.0,279.0,334.0,321.0,377.0,277.0,250.0,339.0,244.0,206.0,239.0,371.0,393.0,12.0
"91.[0,6.899]",256.0,579.0,246.0,341.0,340.0,308.0,339.0,276.0,283.0,380.0,261.0,260.0,238.0,337.0,388.0,10.0


search_time,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
model_score_2401_Bin,,,,,,,,,,,,,,,,
"1.(21.449,inf)",4.0,6.0,2.0,2.0,2.0,1.0,3.0,NaN,2.0,1.0,1.0,5.0,20.0,34.0,35.0,NaN
"2.(18.845,21.449]",15.0,10.0,8.0,9.0,8.0,9.0,17.0,8.0,21.0,27.0,26.0,37.0,67.0,113.0,133.0,3.0
"3.(17.466,18.845]",15.0,23.0,16.0,27.0,23.0,32.0,41.0,26.0,73.0,93.0,54.0,57.0,136.0,195.0,213.0,6.0
"4.(15.787,17.466]",57.0,79.0,49.0,76.0,94.0,118.0,138.0,71.0,109.0,101.0,62.0,60.0,114.0,187.0,221.0,9.0
"5.(13.485,15.787]",76.0,71.0,54.0,84.0,111.0,168.0,186.0,138.0,231.0,297.0,152.0,165.0,226.0,354.0,427.0,10.0
"6.(12.388,13.485]",88.0,117.0,95.0,143.0,144.0,199.0,259.0,146.0,240.0,291.0,182.0,169.0,220.0,321.0,451.0,7.0
"7.(10.028,12.388]",134.0,181.0,140.0,178.0,208.0,284.0,364.0,183.0,276.0,313.0,183.0,152.0,261.0,380.0,448.0,12.0
"8.(8.786,10.028]",118.0,220.0,162.0,180.0,254.0,312.0,376.0,304.0,430.0,579.0,301.0,332.0,434.0,603.0,686.0,12.0
"9.(6.899,8.786]",140.0,351.0,288.0,339.0,415.0,417.0,504.0,290.0,470.0,565.0,285.0,314.0,336.0,604.0,683.0,16.0


In [153]:
# （3-4）model_score_2501
df_psi_js['model_score_2501'] = df_psi_js['model_score_2501'].fillna(-999)
df_psi_js['model_score_2501'] = np.where(df_psi_js.model_score_2501 <= 0, -999, df_psi_js.model_score_2501)
df_bin_js = df_psi_js[(df_psi_js.下单日期 >= '2025-03-01') & (df_psi_js.下单日期 <= '2025-03-31')]
df_psi_js = df_psi_js[df_psi_js.下单日期 >= '2025-03-01']

df_bin_js.shape
train, test = train_test_split(df_bin_js, test_size = 0.2, random_state = 0)
train['target'] = 1
test['target'] = 0
df_bin_js = pd.concat([train, test])

df_bin_js.shape
# df_bin_js[:1]


cutOff = df_qcut_me(df_bin_js, 'model_score_2501', 'target', max_interval=10, special_attribute = [-1111, -999])
cutOff


# （1-3）根据业务应用做相应的调整
# 蚁盾分1.0
# cutOff[10] = 70
# cutOff
# cutOff[5] = 4.5
# cutOff[6] = 10.5
# cutOff[8] = 0.95

# # （1-4）映射
var_bin_list_js = []
df = df_psi_js.copy()
var_bin_js = ['model_score_2501']
for col in var_bin_js:
    # print ("{} is in processing".format(col))
    if -1111 not in set(df[col]) and -999 not in set(df[col]): 
        # cutOff = df_qcut_me(df, col, 'target', max_interval=10, special_attribute=[])
        df[col+'_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[]))
        # del df[col]
        # var_bin_list_js.append(col)
    else:
        # cutOff = df_qcut_me(df, col, 'target', max_interval = 12, special_attribute = [-1111, -999])
        df[col + '_Bin'] = df[col].map(lambda x: AssignBin_me(x, cutOff, special_attribute=[-1111, -999])) 
        # del df[col]
        # var_bin_list_js.append(col)

df['model_score_2501_Bin'] = df.apply(lambda x : '91.[0,10.375]' if x.model_score_2501 >= 0 and x.model_score_2501 <= 10.375 
                                                  else '9.(10.375,11.932]' if x.model_score_2501 > 10.375  and x.model_score_2501 <= 11.932
                                                  else '8.(11.932,13.268]' if x.model_score_2501 > 11.932 and x.model_score_2501 <= 13.268
                                                  else '7.(13.268,14.536]' if x.model_score_2501 > 13.268 and x.model_score_2501 <= 14.536
                                                  else '6.(14.536,15.896]' if x.model_score_2501 > 14.536 and x.model_score_2501 <= 15.896
                                                  else '5.(15.896,17.425]' if x.model_score_2501 > 15.896 and x.model_score_2501 <= 17.425
                                                  else '4.(17.425,19.216]' if x.model_score_2501 > 17.425 and x.model_score_2501 <= 19.216
                                                  else '3.(19.216,21.648]' if x.model_score_2501 > 19.216 and x.model_score_2501 <= 21.648
                                                  else '2.(21.648,25.345]' if x.model_score_2501 > 21.648 and x.model_score_2501 <= 25.345
                                                  else '1.(25.345,inf)' if x.model_score_2501 > 25.345
                                                  else '92.-999', axis = 1)

# （1-5）统计
# df.审核状态 == '出库' 有可能为拒量商家的出库
# conditions_3 = (df.审核状态 == '出库') & (df.是否出库 == 1)  & (df.是否拒量 == 0)
conditions_3 = (df.是否出库 == 1)
conditions_4 = ((~df.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df.是否拒量 == 0))
conditions_5 = (df.is_model == '4')
conditions_6 = ['单人聊天会话中的小程序消息卡片（分享）',
                '八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']

# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
# conditions_1 = (df_info_ori.is_model == '1')
# conditions_2 = (df_info_ori.is_model == '2')
# conditions_3 = (df_info_ori.is_model == '3')
# conditions_4 = (df_info_ori.is_model == '4')
   

df['model_score_2501_result']= df.apply(lambda x: '1.accept' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 >= 0  and x.model_score_2501 <= 14.104
                                                else '2.review' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 14.104 and x.model_score_2501 <= 18.1312
                                                else '3.reject' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 18.1312

                                                else '1.accept' if x.归属渠道 != '芝麻租物'  and x.model_score_2501 >= 0  and x.model_score_2501 <= 13.79
                                                else '2.review' if x.归属渠道 != '芝麻租物'  and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
                                                else '3.reject' if x.归属渠道 != '芝麻租物'  and x.model_score_2501 > 18.098
                                                else 'other'
                                                , axis = 1)
df['model_score_2501_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_2501_result == '2.review'), '3.reject', df.model_score_2501_result)
df['model_score_2501_result'] = np.where((df.归属渠道.isin(conditions_6)) & (df.model_score_2501_result == '1.accept'), '2.review', df.model_score_2501_result)
df['model_score_2501_result_f'] = np.where(df.model_score_2501_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')
# 进件
pd.pivot_table(df, index='model_score_2501_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的进件
pd.pivot_table(df[conditions_5], index='model_score_2501_Bin', columns='search_time', values='order_number',aggfunc='count')
# 使用该模型的通过
pd.pivot_table(df[conditions_5 & (df.model_score_2501_result_f == '模型通过')], index='model_score_2501_Bin', columns='search_time', values='order_number',aggfunc='count')
# # 使用该模型的出库
# pd.pivot_table(df[conditions_3 & conditions_4 & conditions_5], index='model_score_2501_Bin', columns='search_time', values='order_number',aggfunc='count')
# 总体出库
pd.pivot_table(df[conditions_3 & conditions_4], index='model_score_2501_Bin', columns='search_time', values='order_number',aggfunc='count')


(19630, 189)

(19630, 190)

[-1111,
 -999,
 10.375055047603752,
 11.93171946699468,
 13.268766731017662,
 14.535773491509277,
 15.896061531234928,
 17.42487855012292,
 19.216451165317004,
 21.64777555655504,
 25.34479117447615]

search_time,2025-03,2025-04,2025-05,2025-06
model_score_2501_Bin,,,,
"1.(25.345,inf)",1952,2887,3039,67
"2.(21.648,25.345]",1945,2817,3175,77
"3.(19.216,21.648]",1946,2923,3590,76
"4.(17.425,19.216]",1944,2700,3585,83
"5.(15.896,17.425]",1946,2801,3586,85
"6.(14.536,15.896]",1944,2795,3595,77
"7.(13.268,14.536]",1947,2688,3582,76
"8.(11.932,13.268]",1943,2837,3603,87
"9.(10.375,11.932]",1946,2817,3612,102


search_time,2025-03,2025-04,2025-05,2025-06
model_score_2501_Bin,,,,
"1.(25.345,inf)",593,898,753,13
"2.(21.648,25.345]",607,825,814,25
"3.(19.216,21.648]",583,890,976,20
"4.(17.425,19.216]",603,808,1049,24
"5.(15.896,17.425]",586,867,1042,20
"6.(14.536,15.896]",613,869,1050,22
"7.(13.268,14.536]",567,770,1046,27
"8.(11.932,13.268]",579,893,1095,20
"9.(10.375,11.932]",607,842,1052,34


search_time,2025-03,2025-04,2025-05,2025-06
model_score_2501_Bin,,,,
"4.(17.425,19.216]",207,283,336,7
"5.(15.896,17.425]",517,731,867,16
"6.(14.536,15.896]",540,725,907,20
"7.(13.268,14.536]",528,709,966,25
"8.(11.932,13.268]",579,893,1095,20
"9.(10.375,11.932]",607,842,1052,34
"91.[0,10.375]",559,793,997,24


search_time,2025-03,2025-04,2025-05,2025-06
model_score_2501_Bin,,,,
"1.(25.345,inf)",11.0,15.0,14.0,NaN
"2.(21.648,25.345]",31.0,40.0,50.0,NaN
"3.(19.216,21.648]",50.0,77.0,76.0,1.0
"4.(17.425,19.216]",86.0,118.0,151.0,4.0
"5.(15.896,17.425]",124.0,237.0,304.0,6.0
"6.(14.536,15.896]",209.0,290.0,378.0,9.0
"7.(13.268,14.536]",249.0,393.0,482.0,12.0
"8.(11.932,13.268]",339.0,524.0,631.0,17.0
"9.(10.375,11.932]",462.0,677.0,828.0,18.0


In [124]:
model_score_2311 
model_score_2401 
model_score_yl 
model_score_2501
conditions_1 = (df_info_ori.is_model == '1')
conditions_2 = (df_info_ori.is_model == '2')
conditions_3 = (df_info_ori.is_model == '3')
conditions_4 = (df_info_ori.is_model == '4')


,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred,y_fraud_fpd7,y_fraud_mob3dpd7,y_credit_mob3dpd30,y_credit_mob6dpd30,y_credit,火眸分_Bin,rentOnlineRiskLevelFulfill_Bin,rentOnlineRiskLevelOverall_Bin,rentOnlineDetailLevelOverall_Bin,蚁盾分_Bin,融安分_Bin,my_zl_score2_Bin,br_ys_score_Bin,青云分_Bin,model_score_2311_Bin,model_score_yl_Bin,model_score_2401_Bin,model_score_2501_Bin,grade_Bin,strategy_240703_4_Bin,strategy_240703_10_Bin,strategy_240703_14_Bin,score_201048_Bin,score_202098_Bin,score_202100_Bin,strategy_240801_1_Bin,strategy_240801_2_Bin,strategy_240801_3_Bin,strategy_240801_4_Bin,strategy_240801_5_Bin,strategy_240801_7_Bin,strategy_240801_8_Bin,strategy_240801_9_Bin,strategy_240801_10_Bin,strategy_240801_11_Bin,规则42,履约保护_Bin,td212_Bin,zm_jh_t0_level_Bin,FICO_Bin
0,A202206131732031,2022-06-13,2022-06,未知渠道,0,人审拒绝,None,0,0,0,0,NaN,NaN,18301579950,342923199007100712,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.-999,3.-999,5.-999,8.-999,8.-999,8.-999,8.-999,93.-999,8.-999,other,other,other,other,8.-999,6.-999,5.-999,5.-999,"7.[-999,-999]","7.[-999,-999]","7.[-999,-999]",4.-999,4.-999,4.-999,4.-999,4.-999,4.-999,4.-999,4.-999,4.-999,4.-999,5,92.-999,6.-999,7.-999,9.-999


# 11.相关性分析

# 12.测算

## 12_1.前置拦截

In [148]:
# （1）去重订单数
# 联合运营订单
sql = '''
SELECT  order_number, union_rent_tag
from  db_digua_business.t_order
where union_rent_tag = 'Y'
;
'''
tmp1 = query(sql)
tmp1.shape

# 有3单未关联到，为测试白名单
# 拒绝理由 为前置拒绝理由，union_rent_rejected 为机审拒绝，因为联合运营的机审豁免了，所以单独存放机审拒绝
# ,om.user_mobile,tmu.true_name,tmu.id_card_num
conditions_1 = (df_risk_new.归属渠道 == '芝麻租物')
conditions_2 = ((df_risk_new.下单日期 >= '2025-03-01') & (df_risk_new.下单日期 <= '2025-03-29'))
df_risk_new[conditions_1 & conditions_2].是否拒量.value_counts()
var_1 = ['user_mobile','id_card_num','order_number','归属渠道','是否拒量','下单日期']
tmp2 = df_risk_new[conditions_1 & conditions_2][var_1].merge(tmp1, on = 'order_number', how = 'left')
tmp2 = tmp2[tmp2.union_rent_tag != 'Y']
tmp2.shape

tmp2 = tmp2.merge(df_out_data[['order_number','bj_hm_score']], on = 'order_number', how = 'left')
tmp2.shape

tmp2['火眸分_Bin'] = tmp2.apply(lambda x: '1.(0,449]' if x.bj_hm_score >= 0 and x.bj_hm_score<= 449
                                           else '2.(449,475]' if x.bj_hm_score > 449 and x.bj_hm_score <= 475
                                           else '3.(475,499]' if x.bj_hm_score > 475 and x.bj_hm_score <= 499
                                           else '4.(499,530]' if x.bj_hm_score > 499 and x.bj_hm_score <= 530
                                           else '5.(530,579]' if x.bj_hm_score > 530 and x.bj_hm_score <= 579
                                           else '6.(579,657]' if x.bj_hm_score > 579 and x.bj_hm_score <= 657
                                           else '7.(657,inf)' if x.bj_hm_score > 657
                                           else '8.-999'
                                           , axis = 1)
tmp2.shape
tmp2 = tmp2.drop_duplicates(subset = ['id_card_num','下单日期'])
tmp2 = tmp2.drop_duplicates(subset = ['user_mobile','下单日期'])
tmp2.shape
tmp2[:1]
tmp2.火眸分_Bin.value_counts()
del tmp1
del tmp2

(358, 2)

是否拒量
0    85152
1     1157
Name: count, dtype: int64

(85958, 7)

(85958, 8)

(85958, 9)

(52042, 9)

,user_mobile,id_card_num,order_number,归属渠道,是否拒量,下单日期,union_rent_tag,bj_hm_score,火眸分_Bin
0,13457350017,450332198807152118,A202503010002112,芝麻租物,0,2025-03-01,NaN,543.0,"5.(530,579]"


火眸分_Bin
8.-999         14254
5.(530,579]     9848
6.(579,657]     9323
4.(499,530]     6163
1.(0,449]       4093
3.(475,499]     3772
2.(449,475]     3106
7.(657,inf)     1483
Name: count, dtype: int64

In [149]:
# （2）进件-免人审-出库
df_info_ori['火眸分_new_Bin'] = df_info_ori.apply(lambda x: '1.(0,449]' if x.bj_hm_score >= 0 and x.bj_hm_score<= 449
                                           else '2.(449,475]' if x.bj_hm_score > 449 and x.bj_hm_score <= 475
                                           else '3.(475,499]' if x.bj_hm_score > 475 and x.bj_hm_score <= 499
                                           else '4.(499,530]' if x.bj_hm_score > 499 and x.bj_hm_score <= 530
                                           else '5.(530,579]' if x.bj_hm_score > 530 and x.bj_hm_score <= 579
                                           else '6.(579,657]' if x.bj_hm_score > 579 and x.bj_hm_score <= 657
                                           else '7.(657,inf)' if x.bj_hm_score > 657
                                           else '8.-999'
                                           , axis = 1)

conditions_1 = ((df_info_ori.归属渠道 == '芝麻租物') & (df_info_ori.是否联合运营 != '1'))
# # conditions_1 = ((df_info_ori.归属渠道 == '芝麻租物') & (df_info_ori.是否拒量 == 0))
conditions_2 = ((df_info_ori.下单日期 >= '2025-03-01') & (df_info_ori.下单日期 <= '2025-03-29'))
df_info_ori[conditions_1 & conditions_2].火眸分_new_Bin.value_counts()

df_info_ori[conditions_1 & conditions_2 & (df_info_ori.是否免人审 == 1)].火眸分_new_Bin.value_counts()
df_info_ori[conditions_1 & conditions_2 & (df_info_ori.审核状态 == '出库') & (df_info_ori.是否拒量 == 0)].火眸分_new_Bin.value_counts()

火眸分_new_Bin
5.(530,579]    2725
6.(579,657]    2638
4.(499,530]    1732
8.-999         1616
7.(657,inf)     498
3.(475,499]      16
1.(0,449]        13
2.(449,475]       5
Name: count, dtype: int64

火眸分_new_Bin
6.(579,657]    1058
5.(530,579]     976
8.-999          607
4.(499,530]     510
7.(657,inf)     271
1.(0,449]         3
3.(475,499]       1
Name: count, dtype: int64

火眸分_new_Bin
6.(579,657]    389
5.(530,579]    389
4.(499,530]    229
8.-999         144
7.(657,inf)     85
3.(475,499]      3
2.(449,475]      1
1.(0,449]        1
Name: count, dtype: int64

## 12_2.机审强拒

In [ ]:
# （1）加工 统计

# （A）规则 '命中蚁盾分>=80强拒，非搜索渠道' 调整为 '命中蚁盾分>=90 强拒，芝麻租物’、‘命中蚁盾分>=80强拒，非租物非搜索渠道’
# （B）规则‘命中蚁盾分>50且为搜索渠道 强拒’调整为‘命中蚁盾分>70且为搜索渠道 强拒’
# （C）规则 '命中融安分低于680强拒'调整为‘命中融安分低于650强拒，芝麻租物’、‘命中融安分低于680强拒，非租物非搜索渠道’
# （D）规则‘命中融安分低于700强拒且为搜索渠道’调整为‘命中融安分低于680强拒且为搜索渠道’

var_1 = ['order_number','下单日期','search_time','归属渠道','是否免人审','审核状态','拒绝理由','my_zl_score','br_ra_score','jx_yl_UPPE082','xy_qj_C22180008','jx_fico_score','model_score_2401','bj_qy_score']
# （AB）
condition_1 = ['[蚁盾分>=80]','[命中蚁盾分>50]','命中蚁盾分>50','[命中蚁盾分>60]','[命中蚁盾分>50, 蚁盾分>=80]','[蚁盾分>=80, 命中蚁盾分>50]','[蚁盾分>=80, 命中蚁盾分>60]','命中蚁盾分>50, 蚁盾分>=80','蚁盾分>=80']
df_info_cs = df_info[(df_info.下单日期>='2024-06-01')&(df_info.拒绝理由.isin(condition_1))][var_1]

# 机审仅被该条规则强拒的，被机审强拒了，就不会调取极信FICO分，无法判断是否免审，故只能统计仅被改规则强拒的订单数
# df_info_cs['是否免审'] = df_info_cs.apply(lambda x: '免人审' if x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 > 0.655
#                                    else '免人审' if x.jx_fico_score > 604.5 and x.model_score_2401 <= 8.093
#                                    else '免人审' if x.jx_fico_score > 561.5 and x.model_score_2401 <= 9.849
#                                    else '免人审' if x.model_score_2401  <= 14.326 and x.xy_qj_C22180008 > 10.5
#                                    else '免人审' if x.jx_fico_score > 561.5 and x.bj_qy_score > 433.5

#                                    else '其他', axis = 1)

condition_2 = ((df_info_cs.归属渠道 == '芝麻租物')&(df_info_cs.my_zl_score >= 80)&(df_info_cs.my_zl_score < 90))
df_info_cs[condition_2].groupby(['search_time']).agg({'order_number':'count'}).reset_index()

condition_3 = ((df_info_cs.归属渠道 == '搜索渠道')&(df_info_cs.my_zl_score >= 50)&(df_info_cs.my_zl_score < 70))
df_info_cs[condition_3].groupby(['search_time']).agg({'order_number':'count'}).reset_index()


#（CD）
# 命中融安分低于680强拒 和 评分等级低于D 相关系数为1
condition_1 = ['[命中融安分<700]','命中融安分<700','[命中融安分低于680强拒, 评分等级低于D,命中拒绝]','[命中融安分低于680强拒, 评分等级低于D,命中拒绝, 命中融安分<700]','[命中融安分<700, 命中融安分低于680强拒, 评分等级低于D,命中拒绝]','[命中融安分<700, 命中融安分低于680强拒(芝麻综合风险等级0豁免), 评分等级低于D,命中拒绝]','[命中融安分低于680强拒(芝麻综合风险等级0豁免), 评分等级低于D,命中拒绝, 命中融安分<700]']
df_info_cs = df_info[(df_info.下单日期>='2024-06-01')&(df_info.拒绝理由.isin(condition_1))][var_1]
df_info_cs.shape

condition_2 = ((df_info_cs.归属渠道 == '芝麻租物')&(df_info_cs.br_ra_score >= 650)&(df_info_cs.br_ra_score < 680))
df_info_cs[condition_2].groupby(['search_time']).agg({'order_number':'count'}).reset_index()


condition_3 = ((df_info_cs.归属渠道 == '搜索渠道')&(df_info_cs.br_ra_score >= 680)&(df_info_cs.br_ra_score < 700))
df_info_cs[condition_3].groupby(['search_time']).agg({'order_number':'count'}).reset_index()


In [ ]:
# （2）加工 统计-模型
# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401

# （A）ascore_yl202311 对租物和搜索的cutoff做调整
# （B）ascore_202401 对租物和搜索的cutoff做调整


# var_1 = ['order_number','下单日期','search_time','归属渠道','是否免人审','审核状态','拒绝理由','is_model','model_score_2311','model_score_yl','model_score_2401']
var_1 = ['order_number','下单日期','search_time','归属渠道','是否免人审','审核状态','拒绝理由','is_model','model_score_yl','model_score_2401']
condition_1 = (df_info.拒绝理由.isin(['[命中自有模型强拒]','命中自有模型强拒']))
condition_2 = (df_info.归属渠道.isin(['芝麻租物','搜索渠道']))
condition_3 = (df_info.is_model.isin(['2','3']))
condition_4 = ((df_info.is_model == '2') & (df_info.model_score_yl <= 24.5))
condition_5 = ((df_info.is_model == '3') & (df_info.model_score_2401 <= 17.5))

df_info_cs = df_info[(df_info.下单日期>='2024-06-01') & condition_1 & condition_2 & condition_3][var_1]
df_info_cs.shape
df_info_cs[:2]
df_info_cs[condition_4].groupby(['search_time']).agg({'order_number':'count'}).reset_index()
df_info_cs[condition_5].groupby(['search_time']).agg({'order_number':'count'}).reset_index()
# df_info_cs.to_excel('temp1.xlsx', index = False)

In [163]:
# （3）拒量-蚁盾分2.0
conditions_1 = ((df_info_ori.归属渠道.isin(['芝麻租物'])) & (df_info_ori.是否联合运营 != '1') & (df_info_ori.是否拒量 == 0))
# # conditions_1 = ((df_info_ori.归属渠道 == '芝麻租物') & (df_info_ori.是否拒量 == 0))
conditions_2 = ((df_info_ori.下单日期 >= '2025-03-01') & (df_info_ori.下单日期 <= '2025-03-29'))

df_info_ori[conditions_1 & conditions_2 & (df_info_ori.审核状态 == '机审强拒') & (df_info_ori.my_zl_score2 <= 50) & (df_info_ori.my_zl_score2 > 0)].shape
conditions_1 = ((df_info_ori.归属渠道.isin(['搜索渠道'])) & (df_info_ori.是否联合运营 != '1') & (df_info_ori.是否拒量 == 0))
df_info_ori[conditions_1 & conditions_2 & (df_info_ori.审核状态 == '机审强拒') & (df_info_ori.my_zl_score2 <= 45) & (df_info_ori.my_zl_score2 > 0)].shape

(257, 167)

(24, 167)

In [164]:
# （4）拒量-评分等级
conditions_1 = ((df_info_ori.归属渠道.isin(['芝麻租物'])) & (df_info_ori.是否联合运营 != '1') & (df_info_ori.是否拒量 == 0))
# # conditions_1 = ((df_info_ori.归属渠道 == '芝麻租物') & (df_info_ori.是否拒量 == 0))
conditions_2 = ((df_info_ori.下单日期 >= '2025-03-01') & (df_info_ori.下单日期 <= '2025-03-29'))

df_info_ori[conditions_1 & conditions_2 & (df_info_ori.审核状态 == '机审强拒') & (df_info_ori.grade == 'A')].shape
conditions_1 = ((df_info_ori.归属渠道.isin(['搜索渠道'])) & (df_info_ori.是否联合运营 != '1') & (df_info_ori.是否拒量 == 0))
df_info_ori[conditions_1 & conditions_2 & (df_info_ori.审核状态 == '机审强拒') & (df_info_ori.grade == 'A')].shape

(104, 167)

(80, 167)

## 12_3.出库前强拒

In [ ]:
# （1）加工
# df_info：进件订单的各方面数据
# (1)规则41：’非单人话且非扫一扫渠道：命中fico_S50c067信用分 > 489.5 且 fico_S50c067信用分 <= 537 且青云分 > 450 随机40% 豁免‘
# 可调整为 ’非单人话且非扫一扫渠道：命中fico_S50c067信用分 > 489.5 且 fico_S50c067信用分 <= 537 且青云分 > 450 豁免‘
# 规则42
# (2)规则42：’银联模型ascore_yl2311分>17.55且fico_S50c067信用分 >537 且 fico_S50c067信用分 <=559，命中强拒  但青云分 > 450的话 随机40% 豁免该规则
# ‘调整 为 ’银联模型ascore_yl2311分>17.55且fico_S50c067信用分 >537 且 fico_S50c067信用分 <=559，命中强拒  但青云分 > 450的话  豁免该规则‘
# （3）规则38：’蚂蚁数控风险等级=996强拒且非芝麻租物‘调整为’蚂蚁数控风险等级=996强拒且非芝麻租物非搜索‘

df_info_cs = df_info.copy()
# 1 为 强拒，若有出库，则为随机豁免的
df_info_cs['cs_var_41'] = df_info_cs.apply(lambda x: 1 if x.jx_fico_score > 489.5 and x.jx_fico_score <= 537 and x.bj_qy_score > 450 else 0, axis = 1)
# 1 为 强拒，2为强拒，若有出库，则为随机豁免的
df_info_cs['cs_var_42'] = df_info_cs.apply(lambda x: 1 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl > 17.55 and x.bj_qy_score <= 450 
                                          else 2 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl > 17.55 and x.bj_qy_score > 450
                                          else 0, axis = 1)
df_info_cs.shape
df_info_cs[:1]

In [ ]:
# （2）统计
# 仅被该条规则命中而被拒的分布
# df_info_cs.cs_var_41.value_counts()

# df_info_cs[(df_info_cs.cs_var_41 == 1)&(df_info_cs.是否机审强拒 == 0)].to_excel('tmp1.xlsx', index = False)
# df_info_cs[(df_info_cs.cs_var_42 == 2)&(df_info_cs.是否机审强拒 == 0)].to_excel('tmp2.xlsx', index = False)
# df_info_cs[(df_info_cs.search_time >= '2025-01')&(df_info_cs.是否机审强拒 == 0)&(df_info_cs.归属渠道 == '搜索渠道')].to_excel('tmp3.xlsx', index = False)

conditions_1 = (df_info_cs.cs_var_41 == 1)&(df_info_cs.是否机审强拒 == 0)&(df_info_cs.审核状态 == '出库前风控强拒')&(df_info_cs.拒绝理由.isin(['[极信sc32007分 <=537命中强拒]','[命中极信sc32007分强拒]']))
df_info_cs[conditions_1].search_time.value_counts()

conditions_2 = (df_info_cs.cs_var_42 == 2)&(df_info_cs.是否机审强拒 == 0)&(df_info_cs.审核状态 == '出库前风控强拒')&(df_info_cs.拒绝理由 == '[命中银联模型及Fico联合规则强拒]')&(df_info_cs.归属渠道 != '单人聊天会话中的小程序消息卡片（分享）')
df_info_cs[conditions_2].search_time.value_counts()

conditions_3 = (df_info_cs.search_time >= '2025-01')&(df_info_cs.是否机审强拒 == 0)&(df_info_cs.归属渠道 == '搜索渠道')&(df_info_cs.拒绝理由 == '蚂蚁数控风险等级=996强拒')
df_info_cs[conditions_3].search_time.value_counts()

In [157]:
# （3）FICOSCORE豁免
conditions_1 = ((df_info_ori.归属渠道.isin(['芝麻租物','搜索渠道'])) & (df_info_ori.是否联合运营 != '1') & (df_info_ori.是否拒量 == 0))
# # conditions_1 = ((df_info_ori.归属渠道 == '芝麻租物') & (df_info_ori.是否拒量 == 0))
conditions_2 = ((df_info_ori.下单日期 >= '2025-03-01') & (df_info_ori.下单日期 <= '2025-03-29'))

df_info_ori[conditions_1 & conditions_2 & (df_info_ori.审核状态 == '出库前风控强拒') & (df_info_ori.jx_fico_score > 672)].是否免人审.value_counts()
# 出库前风控强拒     484

是否免人审
1    45
Name: count, dtype: int64

## 12_4.风控数据成本

In [136]:
# （1）百融融安分调整
# 测算逻辑：
# 仅芝麻租物调用&应用，并行-跟火眸分的调用次数一样，串行-先调用火眸分、再调用融安分

# 导出到临时分析，看租物的成本
# var = ['order_number','trace_id','下单日期','search_time','product_name','归属渠道','进件']
# tmp = df_risk_new[(df_risk_new.下单日期 >= '2025-01-01')&(df_risk_new.下单日期 <= '2025-05-21')&(df_risk_new.归属渠道 == '芝麻租物')][var]
# tmp.shape
# tmp.to_excel('df_zw_cb_cesuan_20250522.xlsx', index = False)

# # 出库
# # 含拒量
# df_analy_ks[(df_analy_ks.归属渠道 == '芝麻租物')&(df_analy_ks.下单日期 >= '2025-01-01')&(df_analy_ks.下单日期 <= '2025-05-21')].search_time.value_counts()

# # （1-1）并行

# sql1='''-- 风控-冰鉴-火眸分
# SELECT trace_id,  rs, time
# from db_credit.risk_icekredit_cds
# ;
# '''
# bj_hm_df =query(sql1)
# bj_hm_df['bj_hm_score'] = bj_hm_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('card_device_score22', {}).get('bj_score', None))  
# bj_hm_df['bj_hm_score'] = bj_hm_df['bj_hm_score'].astype(int)
# bj_hm_df['search_time'] = bj_hm_df['time'].dt.strftime('%Y-%m')
# bj_hm_df.shape
# bj_hm_df = bj_hm_df[bj_hm_df.search_time >= '2025-01']
# bj_hm_df.shape
# bj_hm_df = bj_hm_df.drop_duplicates(subset = 'trace_id')
# bj_hm_df.shape



# # # df_cesuan_1 = bj_hm_df.merge(df_out_data[['order_number','trace_id']], on = 'trace_id', how = 'left')
# df_cesuan_1 = bj_hm_df.merge(df_risk_new[df_risk_new.归属渠道 == '芝麻租物'][['order_number','trace_id','下单日期','product_name','归属渠道','进件']], on = 'trace_id', how = 'left')
# df_cesuan_1.shape
# df_cesuan_1 = df_cesuan_1.drop_duplicates(subset = 'trace_id')
# df_cesuan_1.shape
# df_cesuan_1[:1]


# conditions_1 = (df_cesuan_1.search_time == '2025-03')
# conditions_2 = (df_cesuan_1.归属渠道 == '芝麻租物')
# conditions_3 = (df_cesuan_1.bj_hm_score > 0) & (df_cesuan_1.bj_hm_score < 449)

# df_cesuan_1[conditions_1].shape
# df_cesuan_1[conditions_1 & conditions_2].shape
# df_cesuan_1[conditions_1 & conditions_2 & conditions_3].shape


# # # （1-2）串行


df_risk_new[df_risk_new["拒绝理由"].str.contains(pat=r"被执行人|失信人") == True][['order_number','下单日期','search_time','拒绝理由']].shape
df_info_ori[df_info_ori["拒绝理由"].str.contains(pat=r"被执行人|失信人") == True][['order_number','下单日期','search_time','拒绝理由']].shape




(116718, 4)

(8743, 4)

In [145]:
conditions_1 = (df_cesuan_1.search_time == '2025-05')
conditions_2 = (df_cesuan_1.归属渠道 == '芝麻租物')
# conditions_3 = (df_cesuan_1.bj_hm_score > 0) & (df_cesuan_1.bj_hm_score < 449)

df_cesuan_1[conditions_1].shape
df_cesuan_1[conditions_1 & conditions_2].shape
# df_cesuan_1[conditions_1 & conditions_2 & conditions_3].shape

(26289, 10)

(3193, 10)

In [147]:
sql1='''-- 风控-冰鉴-青云分
SELECT trace_id, rs, time
from db_credit.risk_icekredit_qy
;
'''
bj_qy_df =query(sql1)
bj_qy_df['bj_qy_score'] = bj_qy_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('qing_yun22', {}).get('bj_score', None))  
bj_qy_df['bj_qy_score'] = bj_qy_df['bj_qy_score'].astype(int)
bj_qy_df['search_time'] = bj_qy_df['time'].dt.strftime('%Y-%m')
bj_qy_df.shape
bj_qy_df.drop_duplicates(subset = ['trace_id']).shape
bj_qy_df[bj_qy_df.bj_qy_score.isna()].shape
bj_qy_df[:2]
bj_qy_df.groupby(['search_time']).agg({'trace_id':'count'})

df_cesuan_1 = bj_qy_df.merge(df_risk_new[['order_number','trace_id','下单日期','product_name','归属渠道','进件']], on = 'trace_id', how = 'left')
df_cesuan_1.shape
df_cesuan_1 = df_cesuan_1.drop_duplicates(subset = 'trace_id')
df_cesuan_1.shape
df_cesuan_1[:1]


conditions_1 = (df_cesuan_1.search_time == '2025-03')
# conditions_2 = (df_cesuan_1.归属渠道 == '芝麻租物')
# conditions_3 = (df_cesuan_1.bj_hm_score > 0) & (df_cesuan_1.bj_hm_score < 449)

# df_cesuan_1[conditions_1].shape
# df_cesuan_1[conditions_1 & conditions_2].shape
# df_cesuan_1[conditions_1 & conditions_2 & conditions_3].shape
df_cesuan_1[conditions_1].归属渠道.value_counts()

(327616, 5)

(327616, 5)

(0, 5)

,trace_id,rs,time,bj_qy_score,search_time
0,19935d138af3f087b71d587c0d41693a,"{""result"": {""qing_yun22"": {""bj_score"": 463}}, ...",2022-12-31 15:02:37,463,2022-12
1,29fc01aa38c3b251410f925c7698edc1,"{""result"": {""qing_yun22"": {""bj_score"": 594}}, ...",2023-01-04 14:49:44,594,2023-01


,trace_id
search_time,
2022-12,2
2023-01,2202
2023-02,4026
2023-03,4065
2023-04,6164
2023-05,4265
2023-06,4861
2023-07,6644
2023-08,7851


(405134, 10)

(327616, 10)

,trace_id,rs,time,bj_qy_score,search_time,order_number,下单日期,product_name,归属渠道,进件
0,19935d138af3f087b71d587c0d41693a,"{""result"": {""qing_yun22"": {""bj_score"": 463}}, ...",2022-12-31 15:02:37,463,2022-12,NaN,NaT,NaN,NaN,NaN


归属渠道
芝麻租物                              2408
搜索渠道                              1462
单人聊天会话中的小程序消息卡片（分享）                349
八派信息                               186
抖音渠道                               167
我的小程序入口                             63
九州信息                                59
其他渠道场景渠道。                           28
小程序商家消息（服务提醒）                       26
未知渠道                                22
侠客行全域自定义                            20
支付宝直播                               16
订阅消息                                13
小程序快窗                                9
生活号                                  9
支付宝 push 消息（同1014）                   6
聚量派                                  4
支付宝社群                                4
曙光计划                                 3
付费流量(通过商家数字推广平台，灯火等投放的广告)            3
第三方 APP（如钉钉）打开，在跳转链接中传入访问来源参数：       2
宜品花                                  2
98租超                                 2
扫描二维码                                2
收藏动态                                 1
订单综合入口(抖音极速版)       

In [160]:
df_cesuan_1[conditions_1].tail()
# 352429590f7dd577343c4a74034fb43b
df_risk_new[df_risk_new.trace_id == '352429590f7dd577343c4a74034fb43b'][['trace_id','order_id']]
df_risk_new[df_risk_new.order_id == 2328756][['trace_id','order_id']]

# df_risk_new[df_risk_new.search_time == '2025-04'].归属渠道.value_counts()
# df_risk_new[df_risk_new.search_time == '2025-04'].shape

,trace_id,rs,time,bj_qy_score,search_time,order_number,下单日期,product_name,归属渠道,进件
339764,422995285cefb014df1b99756ed8d183,"{""result"": {""qing_yun22"": {""bj_score"": 531}}, ...",2025-03-31 23:45:27,531,2025-03,NaN,NaT,NaN,NaN,NaN
339765,352429590f7dd577343c4a74034fb43b,"{""result"": {""qing_yun22"": {""bj_score"": 478}}, ...",2025-03-31 23:45:28,478,2025-03,NaN,NaT,NaN,NaN,NaN
339766,3d26b3f263b68a77a1d964f8aff3af58,"{""result"": {""qing_yun22"": {""bj_score"": 620}}, ...",2025-03-31 23:45:45,620,2025-03,A20250331234841125,2025-03-31,iPhone 16 Pro Max 非监管机 全新未激活 国行正品 顺丰包邮,搜索渠道,未进件
339767,dae30530d55711e3335f5a901c627ac6,"{""result"": {""qing_yun22"": {""bj_score"": 505}}, ...",2025-03-31 23:52:15,505,2025-03,NaN,NaT,NaN,NaN,NaN
339768,89f07364f39b930fabad3b25e38fec7a,"{""result"": {""qing_yun22"": {""bj_score"": 640}}, ...",2025-03-31 23:54:47,640,2025-03,NaN,NaT,NaN,NaN,NaN


,trace_id,order_id


,trace_id,order_id
2096037,f5b098af890bb5ae2250f948f16d582c,2328756


# 13.

In [181]:
# （1）大表
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT om.create_time
    ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
                      when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
                      when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
from db_digua_business.t_order  om 

'''
tmp = query(sql1)

tmp["下单日期"] = tmp['create_time'].dt.date
tmp["下单日期"] = pd.to_datetime(tmp["下单日期"],errors="coerce")
tmp.loc[:,"进件"]=np.where((tmp["status2"]=="待支付")|(tmp["status2"]=="订单取消"),"未进件","进件")

tmp[(tmp.下单日期 >= '2025-05-25')&((tmp.进件 == '进件'))].下单日期.value_counts()
del tmp

下单日期
2025-05-26    1482
2025-05-28    1417
2025-05-27    1353
2025-05-25    1203
2025-05-29    1147
Name: count, dtype: int64

# 14.

# 15.

In [ ]:
def model_group_qcut_score(df1, df, target, score, number_of_groups = 10):
    '''
    :param df_tmp: 包含目标变量与预测值的数据集，一般是训练集
    :param df_tmp1: 需要分组统计的数据集，一般是训练集、验证集、测试集，分组和训练集的标准保持一致
    :param score: 得分或者概率；
    :param target: 目标变量
    '''
    df_tmp1 = df1[[target,score]].copy()
    df_tmp = df.copy()
    
    df_tmp1 = df_tmp1[df_tmp1[target].notnull()]
    score_max = df_tmp1[score].max()
    score_min = df_tmp1[score].min()
    n = (score_max - score_min)/number_of_groups
     
    cutOffPoints = []
    for i in range(number_of_groups + 1):
        bins = i * n + score_min
        cutOffPoints.append(bins)
    cutOffPoints = cutOffPoints[:-1]
    cutOffPoints = cutOffPoints + [score_max]
    
    df_tmp[(df_tmp[score] > score_max)] = score_max
    df_tmp[(df_tmp[score] < score_min)] = score_min
    
    cutOffPoints = sorted(list(set(cutOffPoints)))

    bins = pd.DataFrame(pd.cut(df_tmp[score], cutOffPoints)).rename(columns = {score: 'bin_group'})
        
    df_tmp = df_tmp.merge(bins, how = 'left', left_index = True, right_index = True)
    
    df_tmp['cnt_group_total'] = 1
    df_tmp['cnt_bad'] = df_tmp[target].apply(lambda x: 1 if x == 1 else 0)
    df_tmp['cnt_good'] = df_tmp[target].apply(lambda x: 1 if x == 0 else 0)
    
    df_tmp_grouped = df_tmp.groupby('bin_group').agg({'cnt_bad': np.sum, 'cnt_good': np.sum, 'cnt_group_total':np.sum})
    df_tmp_grouped = df_tmp_grouped.reset_index()
    
#     if df_tmp[score].max() > 1:
#         df_tmp_grouped = df_tmp_grouped.sort_values(by = 'bin_group', ascending = True)
#     else:
#         df_tmp_grouped = df_tmp_grouped.sort_values(by = 'bin_group', ascending = True)
        
    df_tmp_grouped['cum_bad'] = df_tmp_grouped['cnt_bad'].cumsum()
    df_tmp_grouped['cum_good'] = df_tmp_grouped['cnt_good'].cumsum()
    B = df_tmp_grouped['cnt_bad'].sum()
    G = df_tmp_grouped['cnt_good'].sum()
    df_tmp_grouped['group_bad_rate'] = df_tmp_grouped['cnt_bad'] / df_tmp_grouped['cnt_group_total']
    df_tmp_grouped['cum_bad_pct'] = df_tmp_grouped['cum_bad'] / B
    df_tmp_grouped['cum_good_pct'] = df_tmp_grouped['cum_good'] / G
    df_tmp_grouped['KS'] = df_tmp_grouped['cum_bad_pct'] - df_tmp_grouped['cum_good_pct']
    df_tmp_grouped['KS'] = df_tmp_grouped['KS'].apply(lambda x: abs(x))
    
    return df_tmp_grouped

In [ ]:
br_ra_df["下单日期"] = br_ra_df['query_time'].dt.date
br_ra_df["下单日期"] = pd.to_datetime(br_ra_df["下单日期"],errors="coerce")
br_ra_df['search_time'] = pd.to_datetime(br_ra_df["下单日期"]).dt.strftime('%Y-%m')

df.drop(['label'],axis=1)

<!-- （1）每个mob 观察日 对应的逾期天数
（2）实付日期逻辑：如实付日期大于观察日，实付日期清空
（3）达到表现期逻辑：应付日期小于观察日，且应付日期小于当前日期
（4）逾期天数判断逻辑:如状态为已取消状态则0，如实付日期不为空则 0，如实付日期为空且观察日期大于当前日期则 当前日期-应付日期，否则 观察日期-应付日期 -->